<a href="https://colab.research.google.com/github/danielrodriguez007/Data-Science/blob/main/Script_RIPS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Import Libraries 📚**

In [ ]:
pip install pyodbc

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
pip install plyer

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import pyodbc
import pandas as pd
import os
from datetime import datetime
from plyer import notification

# **Create SQL Connection🗄️**

In [ ]:
# Create a SQL connection

connection = pyodbc.connect(
    SERVER='192.168.10.13',
    DRIVER = '{ODBC Driver 17 for SQL Server}',
    host = 'ATENEA',
    DATABASE = "HISMEDICAL",
    UID='adm_drodriguez',
    PWD='0r13nt4m3_4**',
    TrustServerCertificate='yes'
)

# **Queries of Extranjeras 🌍**

En esta sección se encuentran las 4 consultas correspondientes a "Extranjeras" cada una de las consultas se guardan como un archivo .csv por aparte en la carpeta C:\Users\ddrodriguez\Documents\RIPS\Extranjeras

## Consultas

In [ ]:
# SQL command to read the data
sqlQuery = '''

--CONSULTAS EXTRANJERAS

DECLARE @hoy DATETIME = GETDATE()
DECLARE @hasta DATETIME = EOMONTH(@hoy, -1);
DECLARE @desde DATETIME =  DATETRUNC(month,@hasta);

SELECT	DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		S.CodigoHabilitacion [Codigo Prestador],
		SUBSTRING(pac.TipoIdentificacion,1,2) [Tipo Documento],
		pais.ISO+''+pac.Identificacion Documento,
		CAST(FechaFactura AS DATE) Fecha,
		C.NumeroAutorizacion [# Autorización],
		ACT.CodigoNacional [Codigo CUP],
		10 Finalidad,
		13 Causa,
		'' [DX Principal],
		'' [DX Relacionado 1],
		'' [DX Relacionado 2],
		'' [DX Relacionado 3],
		2  [Tipo DX],
		 ((B.ValorFactura*B.IndicadorFacturable)-B.ValorIvaFactura)*A.EstadoActivo ValorFactura1,
		'' ValorCopago,
		 ((B.ValorFactura*B.IndicadorFacturable)-B.ValorIvaFactura)*A.EstadoActivo ValorFactura,
		 ACT.Nombre
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
	left join GEN_CNF_Parametros ciudad on ciudad.ParametroId = MunicipioResidenciaId
	left join GEN_CNF_Parametros nacimiento on nacimiento.ParametroId = LugarNacimientoId
	left join GEN_CNF_Parametros departamento on departamento.ParametroId = nacimiento.PadreId
	left join (select ParametroId,Valor,Interfase1 ISO from GEN_CNF_Parametros where TipoParametro = 'PAISES') pais on pais.ParametroId = departamento.PadreId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('CONSULTA')
AND pais.Valor <> 'Colombia'
AND SUBSTRING(pac.TipoIdentificacion,1,2) NOT IN ('CC','TI')
and  NOT EXISTS (SELECT  *
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaId = A.FacturaId AND NOTAS.FechaFactura > @desde)

'''

# Information converted into a DataFrame with pandas
df = pd.read_sql(sql=sqlQuery, con=connection)

# Export the data on the excel into a Desktop
df.to_csv(os.environ["userprofile"] + "\\Documents\\Rips\\Extranjeras\\" + "Extranjeras_Consultas " +
            datetime.now().strftime("%d-%b-%Y %H%M%S") + ".csv", index=False)


C:\Users\ddrodriguez\AppData\Local\Temp\ipykernel_19832\53706393.py:85: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=sqlQuery, con=connection)


## Facturas

In [ ]:
# SQL command to read the data
sqlQuery = '''

--FACTURAS EXTRANJERAS

DECLARE @hoy DATETIME = GETDATE()
DECLARE @hasta DATETIME = EOMONTH(@hoy, -1);
DECLARE @desde DATETIME =  DATETRUNC(month,@hasta);


SELECT
		[Codigo Prestador],
		[Razon Social],
		[Tipo Identificacion],
		[Identificacion],
		Factura,
		Fecha,
		FechaInicial,
		FechaDesde,
		CodigoResponsable,
		NombreResponsable,
		[# Contrato],
		[Plan Beneficios],
		[# Poliza],
		[Valor Copago],
		[Valor Descuento],
		sum(ValorFactura1) Valor
FROM (
SELECT  S.CodigoHabilitacion [Codigo Prestador],
		'FUNDACION ORIENTAME' [Razon Social],
		'NI' [Tipo Identificacion],
		'860049972' [Identificacion],
		DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		CAST(FechaFactura AS DATE) Fecha,
		CAST(@desde AS DATE) FechaInicial,
		CAST(@hasta AS DATE) FechaDesde,
		CASE
			WHEN M.CodigoResponsable = '900156264' THEN 'EPS037'
			WHEN M.CodigoResponsable = '830003564' THEN 'EPS017'
			WHEN M.CodigoResponsable = '800088702' THEN 'EPS018'
			WHEN M.CodigoResponsable = '830002272' THEN 'EPS019'
			WHEN M.CodigoResponsable = '901127521' THEN 'EPS020'
			ELSE 'SDS001' END CodigoResponsable,

		CASE
			WHEN M.CodigoResponsable = '900156264' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '830003564' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '800088702' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '830002272' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '901127521' THEN A.NombreResponsable
			ELSE 'FUNDACION ORIENTAME' END NombreResponsable,
		'' [# Contrato],
		'IVE' [Plan Beneficios],
		'' [# Poliza],
		'' [Valor Copago],
		'' [Valor Descuento],
		((B.ValorFactura * B.IndicadorFacturable ) - B.ValorIvaFactura ) * A.EstadoActivo ValorFactura1
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
	left join GEN_CNF_Parametros ciudad on ciudad.ParametroId = MunicipioResidenciaId
	left join GEN_CNF_Parametros nacimiento on nacimiento.ParametroId = LugarNacimientoId
	left join GEN_CNF_Parametros departamento on departamento.ParametroId = nacimiento.PadreId
	left join (select ParametroId,Valor,Interfase1 ISO from GEN_CNF_Parametros where TipoParametro = 'PAISES') pais on pais.ParametroId = departamento.PadreId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('CONSULTA')
AND pais.Valor <> 'Colombia'
AND SUBSTRING(pac.TipoIdentificacion,1,2) NOT IN ('CC','TI')
and  NOT EXISTS (SELECT  *
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaId = A.FacturaId AND NOTAS.FechaFactura > @desde)
UNION ALL
select   [Codigo Prestador],
		'FUNDACION ORIENTAME' [Razon Social],
		'NI' [Tipo Identificacion],
		'860049972' [Identificacion],
		Factura,
		Fecha,
		CAST(@desde AS DATE) FechaInicial,
		CAST(@hasta AS DATE) FechaDesde,
				CASE

			WHEN CodigoResponsable = '900156264' THEN 'EPS037'
			WHEN CodigoResponsable = '830003564' THEN 'EPS017'
			WHEN CodigoResponsable = '800088702' THEN 'EPS018'
			WHEN CodigoResponsable = '830002272' THEN 'EPS019'
			WHEN CodigoResponsable = '901127521' THEN 'EPS020' ELSE 'SDS001' END CodigoResponsable, --	P6.Valor ,
				CASE

			WHEN CodigoResponsable = '900156264' THEN NombreResponsable
			WHEN CodigoResponsable = '830003564' THEN NombreResponsable
			WHEN CodigoResponsable = '800088702' THEN NombreResponsable
			WHEN CodigoResponsable = '830002272' THEN NombreResponsable
			WHEN CodigoResponsable = '901127521' THEN NombreResponsable ELSE 'FUNDACION ORIENTAME' END NombreResponsable	,
		'' [# Contrato],
		'IVE' [Plan Beneficios],
		'' [# Poliza],
		'' [Valor Copago],
		'' [Valor Descuento],
		ValorFactura1
from(
 SELECT	Row_number() over(partition by pac.Identificacion,ACT.ActividadId order by pac.Identificacion,A.Numero ASC) fila,
		DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		S.CodigoHabilitacion [Codigo Prestador],
		SUBSTRING(pac.TipoIdentificacion,1,2) [Tipo Documento],
		pais.ISO+''+pac.Identificacion Documento,
		CAST(FechaFactura AS DATE) Fecha,
		'' [# Autorización],
		ACT.CodigoNacional [Codigo CUP],
		1 Ambito,
		2 Finalidad,
		'' Personal,
		'' [DX Principal],
		'' [DX Relacionado 1],
		2  [DX Complicaciones],
		1 [Realizacion],
		M.CodigoResponsable,
		A.NombreResponsable	,
		 ((B.ValorFactura*B.IndicadorFacturable)-B.ValorIvaFactura)*A.EstadoActivo ValorFactura1,
		 ACT.Nombre
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
	left join GEN_CNF_Parametros ciudad on ciudad.ParametroId = MunicipioResidenciaId
	left join GEN_CNF_Parametros nacimiento on nacimiento.ParametroId = LugarNacimientoId
	left join GEN_CNF_Parametros departamento on departamento.ParametroId = nacimiento.PadreId
	left join (select ParametroId,Valor,Interfase1 ISO from GEN_CNF_Parametros where TipoParametro = 'PAISES') pais on pais.ParametroId = departamento.PadreId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('PROCEDIMIENTOS')
AND pais.Valor <> 'Colombia'
AND SUBSTRING(pac.TipoIdentificacion,1,2) NOT IN ('CC','TI')
and  NOT EXISTS (SELECT  NOTAS.FacturaId,NOtas.Numero,  B.FacturaAfectadaId
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					LEFT JOIN FAC_MOV_FacturasAnexos B WITH (NOLOCK) ON NOTAS.FacturaId = B.FacturaEncabezadoId
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaAfectadaId = A.FacturaId  AND NOTAS.FechaFactura> @desde )
				) tabla
				where fila in (1,2)
	)	TABLA
group by
		[Codigo Prestador],
		[Razon Social],
		[Tipo Identificacion],
		[Identificacion],
		Factura,
		Fecha,
		FechaInicial,
		FechaDesde,
		CodigoResponsable,
		NombreResponsable,
		[# Contrato],
		[Plan Beneficios],
		[# Poliza],
		[Valor Copago],
		[Valor Descuento]
'''

# Information converted into a DataFrame with pandas
df = pd.read_sql(sql=sqlQuery, con=connection)

# Export the data on the excel into a Desktop
df.to_csv(os.environ["userprofile"] + "\\Documents\\Rips\\Extranjeras\\" + "Extranjeras_Facturas " +
            datetime.now().strftime("%d-%b-%Y %H%M%S") + ".csv", index=False)




C:\Users\ddrodriguez\AppData\Local\Temp\ipykernel_19832\1849221375.py:236: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=sqlQuery, con=connection)


## Procedimientos

In [ ]:
# SQL command to read the data
sqlQuery = '''

--PROCEDIMIENTOS EXTRANJERAS

DECLARE @hoy DATETIME = GETDATE()
DECLARE @hasta DATETIME = EOMONTH(@hoy, -1);
DECLARE @desde DATETIME =  DATETRUNC(month,@hasta);


SELECT *
FROM(
SELECT	Row_number() over(partition by pac.Identificacion,ACT.ActividadId order by pac.Identificacion,FechaFactura) fila,
		DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		 S.CodigoHabilitacion [Codigo Prestador],
		SUBSTRING(pac.TipoIdentificacion,1,2) [Tipo Documento],
		pais.ISO+''+pac.Identificacion Documento,
		CAST(FechaFactura AS DATE) Fecha,
		C.NumeroAutorizacion [# Autorización],
		ACT.CodigoNacional [Codigo CUP],
		1 Ambito,
		2 Finalidad,
		'' Personal,
		'' [DX Principal],
		'' [DX Relacionado 1],
		2  [DX Complicaciones],
		1 [Realizacion],
		 ((B.ValorFactura*B.IndicadorFacturable)-B.ValorIvaFactura)*A.EstadoActivo ValorFactura1,
		 ACT.Nombre
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
	left join GEN_CNF_Parametros ciudad on ciudad.ParametroId = MunicipioResidenciaId
	left join GEN_CNF_Parametros nacimiento on nacimiento.ParametroId = LugarNacimientoId
	left join GEN_CNF_Parametros departamento on departamento.ParametroId = nacimiento.PadreId
	left join (select ParametroId,Valor,Interfase1 ISO from GEN_CNF_Parametros where TipoParametro = 'PAISES') pais on pais.ParametroId = departamento.PadreId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('PROCEDIMIENTOS')
AND pais.Valor <> 'Colombia'
AND SUBSTRING(pac.TipoIdentificacion,1,2) NOT IN ('CC','TI')
and  NOT EXISTS (SELECT  NOTAS.FacturaId,NOtas.Numero,  B.FacturaAfectadaId
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					LEFT JOIN FAC_MOV_FacturasAnexos B WITH (NOLOCK) ON NOTAS.FacturaId = B.FacturaEncabezadoId
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaAfectadaId = A.FacturaId  AND NOTAS.FechaFactura> @desde )
				) tabla
				where fila IN (1,2)
'''

# Information converted into a DataFrame with pandas
df = pd.read_sql(sql=sqlQuery, con=connection)

# Export the data on the excel into a Desktop
df.to_csv(os.environ["userprofile"] + "\\Documents\\Rips\\Extranjeras\\" + "Extranjeras_Procedimientos " +
            datetime.now().strftime("%d-%b-%Y %H%M%S") + ".csv", index=False)

C:\Users\ddrodriguez\AppData\Local\Temp\ipykernel_19832\2183055988.py:89: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=sqlQuery, con=connection)


## Usuarios

In [ ]:
# SQL command to read the data
sqlQuery = '''

--USUARIOS EXTRANJERAS

DECLARE @hoy DATETIME = GETDATE()
DECLARE @hasta DATETIME = EOMONTH(@hoy, -1);
DECLARE @desde DATETIME =  DATETRUNC(month,@hasta);

SELECT
	DISTINCT
		TABLA.[Tipo Documento],
		TABLA.Documento,
		CodigoResponsable,
		case when CodigoResponsable = 'SDS001' THEN 4 ELSE 1 END TipoUsuaurio,
		COALESCE(pac.PrimerApellido,'') PrimerApellido,
		COALESCE(pac.SegundoApellido,'') SegundoApellido,
		COALESCE(pac.PrimerNombre,'') PrimerNombre,
		COALESCE(pac.SegundoNombre,'') SegundoNombre,
		pac.FechaNacimiento,
		[dbo].[Fun_ORI_CalculaEdad](pac.FechaNacimiento,Fecha) Edad,
		1 Unidad,
		substring(Genero,1,1) Sexo,
		substring(ciudad.Referencia1,1,2) Depertamento,
		substring(ciudad.Referencia1,3,3) Ciudad,
		'U' Zona,
		case when pais.Valor = 'Colombia' or TABLA.[Tipo Documento] in ('CC','TI') then 'Colombia'

		else pais.Valor end Nacionalidad,
		case when pais.Valor = 'Colombia' or TABLA.[Tipo Documento] in ('CC','TI') then 'Nacional'

		else 'Extranjera' end TipoNacionalidad
FROM (
SELECT   S.CodigoHabilitacion [Codigo Prestador],
		'FUNDACION ORIENTAME' [Razon Social],
		'NI' [TipoNit],
		'860049972' Nit,
		SUBSTRING(pac.TipoIdentificacion,1,2) [Tipo Documento],
		pais.ISO+''+pac.Identificacion Documento,
		pac.Identificacion DocumentoReal,
		DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		CAST(FechaFactura AS DATE) Fecha,
		CAST(@desde AS DATE) FechaInicial,
		CAST(@hasta AS DATE) FechaDesde,
		CASE
			WHEN M.CodigoResponsable = '900156264' THEN 'EPS037'
			WHEN M.CodigoResponsable = '830003564' THEN 'EPS017'
			WHEN M.CodigoResponsable = '800088702' THEN 'EPS018'
			WHEN M.CodigoResponsable = '830002272' THEN 'EPS019'
			WHEN M.CodigoResponsable = '901127521' THEN 'EPS020'
			ELSE 'SDS001' END CodigoResponsable,

		CASE
			WHEN M.CodigoResponsable = '900156264' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '830003564' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '800088702' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '830002272' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '901127521' THEN A.NombreResponsable
			ELSE 'FUNDACION ORIENTAME' END NombreResponsable,
		'' [# Contrato],
		'IVE' [Plan Beneficios],
		'' [# Poliza],
		'' [Valor Copago],
		'' [Valor Descuento],
		((B.ValorFactura * B.IndicadorFacturable ) - B.ValorIvaFactura ) * A.EstadoActivo ValorFactura1
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
left join GEN_CNF_Parametros ciudad on ciudad.ParametroId = MunicipioResidenciaId
	left join GEN_CNF_Parametros nacimiento on nacimiento.ParametroId = LugarNacimientoId
	left join GEN_CNF_Parametros departamento on departamento.ParametroId = nacimiento.PadreId
	left join (select ParametroId,Valor,Interfase1 ISO from GEN_CNF_Parametros where TipoParametro = 'PAISES') pais on pais.ParametroId = departamento.PadreId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('CONSULTA')
AND pais.Valor <> 'Colombia'
AND SUBSTRING(pac.TipoIdentificacion,1,2) NOT IN ('CC','TI')
and  NOT EXISTS (SELECT  *
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaId = A.FacturaId AND NOTAS.FechaFactura > @desde)
UNION ALL
select  [Codigo Prestador],
		'FUNDACION ORIENTAME' [Razon Social],
		'NI' [TipoNit],
		'860049972' Nit,
		[Tipo Documento],
		Documento,
		DocumentoReal,
		Factura,
		Fecha,
		CAST(@desde AS DATE) FechaInicial,
		CAST(@hasta AS DATE) FechaDesde,
				CASE

			WHEN CodigoResponsable = '900156264' THEN 'EPS037'
			WHEN CodigoResponsable = '830003564' THEN 'EPS017'
			WHEN CodigoResponsable = '800088702' THEN 'EPS018'
			WHEN CodigoResponsable = '830002272' THEN 'EPS019'
			WHEN CodigoResponsable = '901127521' THEN 'EPS020' ELSE 'SDS001' END CodigoResponsable, --	P6.Valor ,
				CASE

			WHEN CodigoResponsable = '900156264' THEN NombreResponsable
			WHEN CodigoResponsable = '830003564' THEN NombreResponsable
			WHEN CodigoResponsable = '800088702' THEN NombreResponsable
			WHEN CodigoResponsable = '830002272' THEN NombreResponsable
			WHEN CodigoResponsable = '901127521' THEN NombreResponsable ELSE 'FUNDACION ORIENTAME' END NombreResponsable	,
		'' [# Contrato],
		'IVE' [Plan Beneficios],
		'' [# Poliza],
		'' [Valor Copago],
		'' [Valor Descuento],
		ValorFactura1
from(
 SELECT	Row_number() over(partition by pac.Identificacion,ACT.ActividadId order by pac.Identificacion,A.Numero ASC) fila,
		DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		 S.CodigoHabilitacion [Codigo Prestador],
		SUBSTRING(pac.TipoIdentificacion,1,2) [Tipo Documento],
		pais.ISO+''+pac.Identificacion Documento,
		pac.Identificacion DocumentoReal,
		CAST(FechaFactura AS DATE) Fecha,
		'' [# Autorizacion],
		ACT.CodigoNacional [Codigo CUP],
		1 Ambito,
		2 Finalidad,
		'' Personal,
		'' [DX Principal],
		'' [DX Relacionado 1],
		2  [DX Complicaciones],
		1 [Realizacion],
			M.CodigoResponsable, --	P6.Valor ,
		A.NombreResponsable	,
		 ((B.ValorFactura*B.IndicadorFacturable)-B.ValorIvaFactura)*A.EstadoActivo ValorFactura1,
		 ACT.Nombre
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
	left join GEN_CNF_Parametros ciudad on ciudad.ParametroId = MunicipioResidenciaId
	left join GEN_CNF_Parametros nacimiento on nacimiento.ParametroId = LugarNacimientoId
	left join GEN_CNF_Parametros departamento on departamento.ParametroId = nacimiento.PadreId
	left join (select ParametroId,Valor,Interfase1 ISO from GEN_CNF_Parametros where TipoParametro = 'PAISES') pais on pais.ParametroId = departamento.PadreId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('PROCEDIMIENTOS')
AND pais.Valor <> 'Colombia'
AND SUBSTRING(pac.TipoIdentificacion,1,2) NOT IN ('CC','TI')
and  NOT EXISTS (SELECT  NOTAS.FacturaId,NOtas.Numero,  B.FacturaAfectadaId
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					LEFT JOIN FAC_MOV_FacturasAnexos B WITH (NOLOCK) ON NOTAS.FacturaId = B.FacturaEncabezadoId
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaAfectadaId = A.FacturaId  AND NOTAS.FechaFactura> @desde )
				) tabla
					where fila in (1,2)
	)	TABLA
	INNER JOIN ADT_VIS_Pacientes pac with (nolock) ON pac.Identificacion = TABLA.DocumentoReal and SUBSTRING(pac.TipoIdentificacion,1,2) = TABLA.[Tipo Documento]
	left join GEN_CNF_Parametros ciudad on ciudad.ParametroId = MunicipioResidenciaId
	left join GEN_CNF_Parametros nacimiento on nacimiento.ParametroId = LugarNacimientoId
	left join GEN_CNF_Parametros departamento on departamento.ParametroId = nacimiento.PadreId
	left join GEN_CNF_Parametros  pais on pais.ParametroId = departamento.PadreId
'''

# Information converted into a DataFrame with pandas
df = pd.read_sql(sql=sqlQuery, con=connection)

# Export the data on the excel into a Desktop
df.to_csv(os.environ["userprofile"] + "\\Documents\\Rips\\Extranjeras\\" + "Extranjeras_Usuarios " +
            datetime.now().strftime("%d-%b-%Y %H%M%S") + ".csv", index=False)

#Display notification to user
notification.notify(title="✅Report Status!!!",
                    message= f"Queries of Extranjeras has been succesfully saved into .csv\
                        \n Queries Executed:  \n 1. Consultas, 2.Facturas, 3.Procedimientos, 4.Usuarios",
                        timeout=10)

C:\Users\ddrodriguez\AppData\Local\Temp\ipykernel_19832\1856946553.py:237: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=sqlQuery, con=connection)


# **Queries of Bogotá IVE🐸**

En esta sección se encuentran las 4 consultas correspondientes a "Bogota IVE" cada una de las consultas se guardan como un archivo .csv por aparte en la carpeta C:\Users\ddrodriguez\Documents\RIPS\Bogota\IVE

## Consultas

In [ ]:
# SQL command to read the data
sqlQuery = '''
--CONSULTAS BOGOTA IVE

DECLARE @hoy DATETIME = GETDATE()
DECLARE @hasta DATETIME = EOMONTH(@hoy, -1);
DECLARE @desde DATETIME =  DATETRUNC(month,@hasta),
		@tipo_atencion INT = 6081,
		@codigo_ciudad VARCHAR(5) = '11001'
/*
6081	IVE-TAI- Interrupción Voluntaria Embarazo

11001	BOGOTA
05001	MEDELLIN
08001	BARRANQUILLA
66170	DOSQUEBRADAS
54001	CUCUTA
52356	IPIALES
50001	VILLAVICENCI
20001	VALLEDUPAR
*/

SELECT	DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		'110010893004' [Codigo Prestador],
		SUBSTRING(pac.TipoIdentificacion,1,2) [Tipo Documento],
		pac.Identificacion Documento,
		CAST(FechaFactura AS DATE) Fecha,
		C.NumeroAutorizacion [# Autorización],
		ACT.CodigoNacional [Codigo CUP],
		10 Finalidad,
		13 Causa,
		'' [DX Principal],
		'' [DX Relacionado 1],
		'' [DX Relacionado 2],
		'' [DX Relacionado 3],
		2  [Tipo DX],
		 ((B.ValorFactura*B.IndicadorFacturable)-B.ValorIvaFactura)*A.EstadoActivo ValorFactura1,
		'' ValorCopago,
		 ((B.ValorFactura*B.IndicadorFacturable)-B.ValorIvaFactura)*A.EstadoActivo ValorFactura,
		 ACT.Nombre
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('CONSULTA')
AND S.Codigo in (SELECT Codigo FROM GEN_CNF_Sedes WHERE CodigoTributarioCiudad = @codigo_ciudad)
AND  P5.ParametroId = @tipo_atencion
and  NOT EXISTS (SELECT  *
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaId = A.FacturaId AND NOTAS.FechaFactura > @desde)
'''

# Information converted into a DataFrame with pandas
df = pd.read_sql(sql=sqlQuery, con=connection)

# Export the data on the excel into a Desktop
df.to_csv(os.environ["userprofile"] + "\\Documents\\Rips\\Bogota\\IVE\\" + "Bogota_IVE_Consultas " +
            datetime.now().strftime("%d-%b-%Y %H%M%S") + ".csv", index=False)



C:\Users\ddrodriguez\AppData\Local\Temp\ipykernel_19832\639777439.py:93: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=sqlQuery, con=connection)


## Facturas

In [ ]:
# SQL command to read the data
sqlQuery = '''
--Facturas
DECLARE @hoy DATETIME = GETDATE()
DECLARE @hasta DATETIME = EOMONTH(@hoy, -1);
DECLARE @desde DATETIME =  DATETRUNC(month,@hasta),
		@tipo_atencion INT = 6081,
		@codigo_ciudad VARCHAR(5) = '11001'
/*
6081	IVE-TAI- Interrupción Voluntaria Embarazo

11001	BOGOTA
*/



SELECT
		[Codigo Prestador],
		[Razon Social],
		[Tipo Identificacion],
		[Identificacion],
		Factura,
		Fecha,
		FechaInicial,
		FechaDesde,
		CodigoResponsable,
		NombreResponsable,
		[# Contrato],
		[Plan Beneficios],
		[# Poliza],
		[Valor Copago],
		[Valor Descuento],
		sum(ValorFactura1) Valor
FROM (
SELECT  '110010893004' [Codigo Prestador],
		'FUNDACION ORIENTAME' [Razon Social],
		'NI' [Tipo Identificacion],
		'860049972' [Identificacion],
		DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		CAST(FechaFactura AS DATE) Fecha,
		CAST(@desde AS DATE) FechaInicial,
		CAST(@hasta AS DATE) FechaDesde,
		CASE
			WHEN M.CodigoResponsable = '900156264' THEN 'EPS037'
			WHEN M.CodigoResponsable = '830003564' THEN 'EPS017'
			WHEN M.CodigoResponsable = '800088702' THEN 'EPS018'
			WHEN M.CodigoResponsable = '830002272' THEN 'EPS019'
			WHEN M.CodigoResponsable = '901127521' THEN 'EPS020'
			ELSE 'SDS001' END CodigoResponsable,

		CASE
			WHEN M.CodigoResponsable = '900156264' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '830003564' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '800088702' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '830002272' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '901127521' THEN A.NombreResponsable
			ELSE 'FUNDACION ORIENTAME' END NombreResponsable,
		'' [# Contrato],
		'IVE' [Plan Beneficios],
		'' [# Poliza],
		'' [Valor Copago],
		'' [Valor Descuento],
		((B.ValorFactura * B.IndicadorFacturable ) - B.ValorIvaFactura ) * A.EstadoActivo ValorFactura1
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('CONSULTA')
AND S.Codigo in (SELECT Codigo FROM GEN_CNF_Sedes WHERE CodigoTributarioCiudad = @codigo_ciudad)
AND  P5.ParametroId = @tipo_atencion
and  NOT EXISTS (SELECT  *
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaId = A.FacturaId AND NOTAS.FechaFactura > @desde)
UNION ALL
select  '110010893004' [Codigo Prestador],
		'FUNDACION ORIENTAME' [Razon Social],
		'NI' [Tipo Identificacion],
		'860049972' [Identificacion],
		Factura,
		Fecha,
		CAST(@desde AS DATE) FechaInicial,
		CAST(@hasta AS DATE) FechaDesde,
				CASE

			WHEN CodigoResponsable = '900156264' THEN 'EPS037'
			WHEN CodigoResponsable = '830003564' THEN 'EPS017'
			WHEN CodigoResponsable = '800088702' THEN 'EPS018'
			WHEN CodigoResponsable = '830002272' THEN 'EPS019'
			WHEN CodigoResponsable = '901127521' THEN 'EPS020' ELSE 'SDS001' END CodigoResponsable, --	P6.Valor ,
				CASE

			WHEN CodigoResponsable = '900156264' THEN NombreResponsable
			WHEN CodigoResponsable = '830003564' THEN NombreResponsable
			WHEN CodigoResponsable = '800088702' THEN NombreResponsable
			WHEN CodigoResponsable = '830002272' THEN NombreResponsable
			WHEN CodigoResponsable = '901127521' THEN NombreResponsable ELSE 'FUNDACION ORIENTAME' END NombreResponsable	,
		'' [# Contrato],
		'IVE' [Plan Beneficios],
		'' [# Poliza],
		'' [Valor Copago],
		'' [Valor Descuento],
		ValorFactura1
from(
 SELECT	Row_number() over(partition by pac.Identificacion,ACT.ActividadId order by pac.Identificacion,A.Numero ASC) fila,
		DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		'110010893004' [Codigo Prestador],
		SUBSTRING(pac.TipoIdentificacion,1,2) [Tipo Documento],
		pac.Identificacion Documento,
		CAST(FechaFactura AS DATE) Fecha,
		'' [# Autorización],
		ACT.CodigoNacional [Codigo CUP],
		1 Ambito,
		2 Finalidad,
		'' Personal,
		'' [DX Principal],
		'' [DX Relacionado 1],
		2  [DX Complicaciones],
		1 [Realizacion],
		M.CodigoResponsable,
		A.NombreResponsable	,
		 ((B.ValorFactura*B.IndicadorFacturable)-B.ValorIvaFactura)*A.EstadoActivo ValorFactura1,
		 ACT.Nombre
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('PROCEDIMIENTOS')
AND S.Codigo in (SELECT Codigo FROM GEN_CNF_Sedes WHERE CodigoTributarioCiudad = @codigo_ciudad)
AND  P5.ParametroId = @tipo_atencion
and  NOT EXISTS (SELECT  NOTAS.FacturaId,NOtas.Numero,  B.FacturaAfectadaId
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					LEFT JOIN FAC_MOV_FacturasAnexos B WITH (NOLOCK) ON NOTAS.FacturaId = B.FacturaEncabezadoId
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaAfectadaId = A.FacturaId  AND NOTAS.FechaFactura> @desde )
				) tabla
				where fila in (1,2)
	)	TABLA
group by
		[Codigo Prestador],
		[Razon Social],
		[Tipo Identificacion],
		[Identificacion],
		Factura,
		Fecha,
		FechaInicial,
		FechaDesde,
		CodigoResponsable,
		NombreResponsable,
		[# Contrato],
		[Plan Beneficios],
		[# Poliza],
		[Valor Copago],
		[Valor Descuento]
'''

# Information converted into a DataFrame with pandas
df = pd.read_sql(sql=sqlQuery, con=connection)

# Export the data on the excel into a Desktop
df.to_csv(os.environ["userprofile"] + "\\Documents\\Rips\\Bogota\\IVE\\" + "Bogota_IVE_Facturas " +
            datetime.now().strftime("%d-%b-%Y %H%M%S") + ".csv", index=False)



C:\Users\ddrodriguez\AppData\Local\Temp\ipykernel_19832\2223920359.py:234: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=sqlQuery, con=connection)


## Procedimientos

In [ ]:
# SQL command to read the data
sqlQuery = '''
DECLARE @hoy DATETIME = GETDATE()
DECLARE @hasta DATETIME = EOMONTH(@hoy, -1);
DECLARE @desde DATETIME =  DATETRUNC(month,@hasta),
		@tipo_atencion INT = 6081,
		@codigo_ciudad VARCHAR(5) = '11001'
/*
6081	IVE-TAI- Interrupción Voluntaria Embarazo

11001	BOGOTA
*/


select *
from(
SELECT	Row_number() over(partition by pac.Identificacion,ACT.ActividadId order by pac.Identificacion,FechaFactura) fila,
		DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		'110010893004' [Codigo Prestador],
		SUBSTRING(pac.TipoIdentificacion,1,2) [Tipo Documento],
		pac.Identificacion Documento,
		CAST(FechaFactura AS DATE) Fecha,
		C.NumeroAutorizacion [# Autorizaci�n],
		ACT.CodigoNacional [Codigo CUP],
		1 Ambito,
		2 Finalidad,
		'' Personal,
		'' [DX Principal],
		'' [DX Relacionado 1],
		2  [DX Complicaciones],
		1 [Realizacion],
		 ((B.ValorFactura*B.IndicadorFacturable)-B.ValorIvaFactura)*A.EstadoActivo ValorFactura1,
		 ACT.Nombre
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('PROCEDIMIENTOS')
AND S.Codigo in (SELECT Codigo FROM GEN_CNF_Sedes WHERE CodigoTributarioCiudad = @codigo_ciudad)
AND  P5.ParametroId = @tipo_atencion
and  NOT EXISTS (SELECT  NOTAS.FacturaId,NOtas.Numero,  B.FacturaAfectadaId
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					LEFT JOIN FAC_MOV_FacturasAnexos B WITH (NOLOCK) ON NOTAS.FacturaId = B.FacturaEncabezadoId
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaAfectadaId = A.FacturaId  AND NOTAS.FechaFactura> @desde )
				) tabla
				where fila in (1,2)
'''

# Information converted into a DataFrame with pandas
df = pd.read_sql(sql=sqlQuery, con=connection)

# Export the data on the excel into a Desktop
df.to_csv(os.environ["userprofile"] + "\\Documents\\Rips\\Bogota\\IVE\\" + "Bogota_IVE_Procedimientos " +
            datetime.now().strftime("%d-%b-%Y %H%M%S") + ".csv", index=False)



C:\Users\ddrodriguez\AppData\Local\Temp\ipykernel_19832\1218606089.py:89: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=sqlQuery, con=connection)


## Usuarios

In [ ]:
# SQL command to read the data
sqlQuery = '''

--USUARIOS BOGOTA IVE

DECLARE @hoy DATETIME = GETDATE()
DECLARE @hasta DATETIME = EOMONTH(@hoy, -1);
DECLARE @desde DATETIME =  DATETRUNC(month,@hasta),
		@tipo_atencion INT = 6081,
		@codigo_ciudad VARCHAR(5) = '11001'
/*
6081	IVE-TAI- Interrupción Voluntaria Embarazo

11001	BOGOTA
*/


SELECT
	DISTINCT
		TABLA.[Tipo Documento],
		TABLA.Documento,
		CodigoResponsable,
		case when CodigoResponsable = 'SDS001' THEN 4 ELSE 1 END TipoUsuaurio,
		COALESCE(pac.PrimerApellido,'') PrimerApellido,
		COALESCE(pac.SegundoApellido,'') SegundoApellido,
		COALESCE(pac.PrimerNombre,'') PrimerNombre,
		COALESCE(pac.SegundoNombre,'') SegundoNombre,
		pac.FechaNacimiento,
		[dbo].[Fun_ORI_CalculaEdad](pac.FechaNacimiento,Fecha) Edad,
		1 Unidad,
		substring(Genero,1,1) Sexo,
		substring(ciudad.Referencia1,1,2) Depertamento,
		substring(ciudad.Referencia1,3,3) Ciudad,
		'U' Zona,
		case when pais.Valor = 'Colombia' or TABLA.[Tipo Documento] in ('CC','TI') then 'Colombia'

		else pais.Valor end Nacionalidad,
		case when pais.Valor = 'Colombia' or TABLA.[Tipo Documento] in ('CC','TI') then 'Nacional'

		else 'Extranjera' end TipoNacionalidad
FROM (
SELECT  '110010893004' [Codigo Prestador],
		'FUNDACION ORIENTAME' [Razon Social],
		'NI' [TipoNit],
		'860049972' Nit,
		SUBSTRING(pac.TipoIdentificacion,1,2) [Tipo Documento],
		pac.Identificacion Documento,
		DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		CAST(FechaFactura AS DATE) Fecha,
		CAST(@desde AS DATE) FechaInicial,
		CAST(@hasta AS DATE) FechaDesde,
		CASE
			WHEN M.CodigoResponsable = '900156264' THEN 'EPS037'
			WHEN M.CodigoResponsable = '830003564' THEN 'EPS017'
			WHEN M.CodigoResponsable = '800088702' THEN 'EPS018'
			WHEN M.CodigoResponsable = '830002272' THEN 'EPS019'
			WHEN M.CodigoResponsable = '901127521' THEN 'EPS020'
			ELSE 'SDS001' END CodigoResponsable,

		CASE
			WHEN M.CodigoResponsable = '900156264' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '830003564' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '800088702' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '830002272' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '901127521' THEN A.NombreResponsable
			ELSE 'FUNDACION ORIENTAME' END NombreResponsable,
		'' [# Contrato],
		'IVE' [Plan Beneficios],
		'' [# Poliza],
		'' [Valor Copago],
		'' [Valor Descuento],
		((B.ValorFactura * B.IndicadorFacturable ) - B.ValorIvaFactura ) * A.EstadoActivo ValorFactura1
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('CONSULTA')
AND S.Codigo in (SELECT Codigo FROM GEN_CNF_Sedes WHERE CodigoTributarioCiudad = @codigo_ciudad)
AND  P5.ParametroId = @tipo_atencion
and  NOT EXISTS (SELECT  *
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaId = A.FacturaId AND NOTAS.FechaFactura > @desde)
UNION ALL
select  '110010893004' [Codigo Prestador],
		'FUNDACION ORIENTAME' [Razon Social],
		'NI' [TipoNit],
		'860049972' Nit,
		[Tipo Documento],
		Documento,
		Factura,
		Fecha,
		CAST(@desde AS DATE) FechaInicial,
		CAST(@hasta AS DATE) FechaDesde,
				CASE

			WHEN CodigoResponsable = '900156264' THEN 'EPS037'
			WHEN CodigoResponsable = '830003564' THEN 'EPS017'
			WHEN CodigoResponsable = '800088702' THEN 'EPS018'
			WHEN CodigoResponsable = '830002272' THEN 'EPS019'
			WHEN CodigoResponsable = '901127521' THEN 'EPS020' ELSE 'SDS001' END CodigoResponsable, --	P6.Valor ,
				CASE

			WHEN CodigoResponsable = '900156264' THEN NombreResponsable
			WHEN CodigoResponsable = '830003564' THEN NombreResponsable
			WHEN CodigoResponsable = '800088702' THEN NombreResponsable
			WHEN CodigoResponsable = '830002272' THEN NombreResponsable
			WHEN CodigoResponsable = '901127521' THEN NombreResponsable ELSE 'FUNDACION ORIENTAME' END NombreResponsable	,
		'' [# Contrato],
		'IVE' [Plan Beneficios],
		'' [# Poliza],
		'' [Valor Copago],
		'' [Valor Descuento],
		ValorFactura1
from(
 SELECT	Row_number() over(partition by pac.Identificacion,ACT.ActividadId order by pac.Identificacion,A.Numero ASC) fila,
		DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		'110010893004' [Codigo Prestador],
		SUBSTRING(pac.TipoIdentificacion,1,2) [Tipo Documento],
		pac.Identificacion Documento,
		CAST(FechaFactura AS DATE) Fecha,
		'' [# Autorizaci�n],
		ACT.CodigoNacional [Codigo CUP],
		1 Ambito,
		2 Finalidad,
		'' Personal,
		'' [DX Principal],
		'' [DX Relacionado 1],
		2  [DX Complicaciones],
		1 [Realizacion],
			M.CodigoResponsable, --	P6.Valor ,
		A.NombreResponsable	,
		 ((B.ValorFactura*B.IndicadorFacturable)-B.ValorIvaFactura)*A.EstadoActivo ValorFactura1,
		 ACT.Nombre
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('PROCEDIMIENTOS')
AND S.Codigo in (SELECT Codigo FROM GEN_CNF_Sedes WHERE CodigoTributarioCiudad = @codigo_ciudad)
AND  P5.ParametroId = @tipo_atencion
and  NOT EXISTS (SELECT  NOTAS.FacturaId,NOtas.Numero,  B.FacturaAfectadaId
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					LEFT JOIN FAC_MOV_FacturasAnexos B WITH (NOLOCK) ON NOTAS.FacturaId = B.FacturaEncabezadoId
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaAfectadaId = A.FacturaId  AND NOTAS.FechaFactura> @desde )
				) tabla
					where fila in (1,2)
	)	TABLA
	INNER JOIN ADT_VIS_Pacientes pac with (nolock) ON pac.Identificacion = TABLA.Documento and SUBSTRING(pac.TipoIdentificacion,1,2) = TABLA.[Tipo Documento]
	left join GEN_CNF_Parametros ciudad on ciudad.ParametroId = MunicipioResidenciaId
	left join GEN_CNF_Parametros nacimiento on nacimiento.ParametroId = LugarNacimientoId
	left join GEN_CNF_Parametros departamento on departamento.ParametroId = nacimiento.PadreId
	left join GEN_CNF_Parametros pais on pais.ParametroId = departamento.PadreId
'''

# Information converted into a DataFrame with pandas
df = pd.read_sql(sql=sqlQuery, con=connection)

# Export the data on the excel into a Desktop
df.to_csv(os.environ["userprofile"] + "\\Documents\\Rips\\Bogota\\IVE\\" + "Bogota_IVE_Usuarios " +
            datetime.now().strftime("%d-%b-%Y %H%M%S") + ".csv", index=False)

#Display notification to user
notification.notify(title="✅Report Status!!!",
                    message= f"Queries of Bogota IVE has been succesfully saved into .csv\
                        \n Queries Executed:  \n 1. Consultas, 2.Facturas, 3.Procedimientos, 4.Usuarios",
                        timeout=10)


C:\Users\ddrodriguez\AppData\Local\Temp\ipykernel_19832\3327516119.py:234: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=sqlQuery, con=connection)


# **Queries of Bogota DIV🐳**

En esta sección se encuentran las 4 consultas correspondientes a "Bogota DIV" cada una de las consultas se guardan como un archivo .csv por aparte en la carpeta C:\Users\ddrodriguez\Documents\RIPS\Bogota\DIV.

## Consultas

In [ ]:
# SQL command to read the data
sqlQuery = '''
--CONSULTAS BOGOTA DIV

DECLARE @hoy DATETIME = GETDATE()
DECLARE @hasta DATETIME = EOMONTH(@hoy, -1);
DECLARE @desde DATETIME =  DATETRUNC(month,@hasta),
		@tipo_atencion INT = 6081,
		@codigo_ciudad VARCHAR(5) = '11001'

/*
6081	IVE-TAI- Interrupción Voluntaria Embarazo

11001	BOGOTA
*/

SELECT	DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		'110010893004' [Codigo Prestador],
		SUBSTRING(pac.TipoIdentificacion,1,2) [Tipo Documento],
		pac.Identificacion Documento,
		CAST(FechaFactura AS DATE) Fecha,
		C.NumeroAutorizacion [# Autorización],
		ACT.CodigoNacional [Codigo CUP],
		10 Finalidad,
		13 Causa,
		'' [DX Principal],
		'' [DX Relacionado 1],
		'' [DX Relacionado 2],
		'' [DX Relacionado 3],
		2  [Tipo DX],
		 ((B.ValorFactura*B.IndicadorFacturable)-B.ValorIvaFactura)*A.EstadoActivo ValorFactura1,
		'' ValorCopago,
		 ((B.ValorFactura*B.IndicadorFacturable)-B.ValorIvaFactura)*A.EstadoActivo ValorFactura,
		 ACT.Nombre
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('CONSULTA')
AND S.Codigo in (SELECT Codigo FROM GEN_CNF_Sedes WHERE CodigoTributarioCiudad = @codigo_ciudad)
AND  P5.ParametroId <> @tipo_atencion
and  NOT EXISTS (SELECT  *
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaId = A.FacturaId AND NOTAS.FechaFactura > @desde)
'''

# Information converted into a DataFrame with pandas
df = pd.read_sql(sql=sqlQuery, con=connection)

# Export the data on the excel into a Desktop
df.to_csv(os.environ["userprofile"] + "\\Documents\\Rips\\Bogota\\DIV\\" + "Bogota_DIV_Consultas " +
            datetime.now().strftime("%d-%b-%Y %H%M%S") + ".csv", index=False)

C:\Users\ddrodriguez\AppData\Local\Temp\ipykernel_19832\2863674225.py:87: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=sqlQuery, con=connection)


## Facturas

In [ ]:
# SQL command to read the data
sqlQuery = '''
--FACTURAS
DECLARE @hoy DATETIME = GETDATE()
DECLARE @hasta DATETIME = EOMONTH(@hoy, -1);
DECLARE @desde DATETIME =  DATETRUNC(month,@hasta),
		@tipo_atencion INT = 6081,
		@codigo_ciudad VARCHAR(5) = '11001'
/*
6081	IVE-TAI- Interrupción Voluntaria Embarazo

11001	BOGOTA
*/



SELECT
		[Codigo Prestador],
		[Razon Social],
		[Tipo Identificacion],
		[Identificacion],
		Factura,
		Fecha,
		FechaInicial,
		FechaDesde,
		CodigoResponsable,
		NombreResponsable,
		[# Contrato],
		[Plan Beneficios],
		[# Poliza],
		[Valor Copago],
		[Valor Descuento],
		sum(ValorFactura1) Valor
FROM (
SELECT  '110010893004' [Codigo Prestador],
		'FUNDACION ORIENTAME' [Razon Social],
		'NI' [Tipo Identificacion],
		'860049972' [Identificacion],
		DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		CAST(FechaFactura AS DATE) Fecha,
		CAST(@desde AS DATE) FechaInicial,
		CAST(@hasta AS DATE) FechaDesde,
		CASE
			WHEN M.CodigoResponsable = '900156264' THEN 'EPS037'
			WHEN M.CodigoResponsable = '830003564' THEN 'EPS017'
			WHEN M.CodigoResponsable = '800088702' THEN 'EPS018'
			WHEN M.CodigoResponsable = '830002272' THEN 'EPS019'
			WHEN M.CodigoResponsable = '901127521' THEN 'EPS020'
			ELSE 'SDS001' END CodigoResponsable,

		CASE
			WHEN M.CodigoResponsable = '900156264' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '830003564' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '800088702' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '830002272' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '901127521' THEN A.NombreResponsable
			ELSE 'FUNDACION ORIENTAME' END NombreResponsable,
		'' [# Contrato],
		'IVE' [Plan Beneficios],
		'' [# Poliza],
		'' [Valor Copago],
		'' [Valor Descuento],
		((B.ValorFactura * B.IndicadorFacturable ) - B.ValorIvaFactura ) * A.EstadoActivo ValorFactura1
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('CONSULTA')
AND S.Codigo in (SELECT Codigo FROM GEN_CNF_Sedes WHERE CodigoTributarioCiudad = @codigo_ciudad)
AND  P5.ParametroId <> @tipo_atencion
and  NOT EXISTS (SELECT  *
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaId = A.FacturaId AND NOTAS.FechaFactura > @desde)
UNION ALL
select  '110010893004' [Codigo Prestador],
		'FUNDACION ORIENTAME' [Razon Social],
		'NI' [Tipo Identificacion],
		'860049972' [Identificacion],
		Factura,
		Fecha,
		CAST(@desde AS DATE) FechaInicial,
		CAST(@hasta AS DATE) FechaDesde,
				CASE

			WHEN CodigoResponsable = '900156264' THEN 'EPS037'
			WHEN CodigoResponsable = '830003564' THEN 'EPS017'
			WHEN CodigoResponsable = '800088702' THEN 'EPS018'
			WHEN CodigoResponsable = '830002272' THEN 'EPS019'
			WHEN CodigoResponsable = '901127521' THEN 'EPS020' ELSE 'SDS001' END CodigoResponsable, --	P6.Valor ,
				CASE

			WHEN CodigoResponsable = '900156264' THEN NombreResponsable
			WHEN CodigoResponsable = '830003564' THEN NombreResponsable
			WHEN CodigoResponsable = '800088702' THEN NombreResponsable
			WHEN CodigoResponsable = '830002272' THEN NombreResponsable
			WHEN CodigoResponsable = '901127521' THEN NombreResponsable ELSE 'FUNDACION ORIENTAME' END NombreResponsable	,
		'' [# Contrato],
		'IVE' [Plan Beneficios],
		'' [# Poliza],
		'' [Valor Copago],
		'' [Valor Descuento],
		ValorFactura1
from(
 SELECT	Row_number() over(partition by pac.Identificacion,ACT.ActividadId order by pac.Identificacion,A.Numero ASC) fila,
		DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		'110010893004' [Codigo Prestador],
		SUBSTRING(pac.TipoIdentificacion,1,2) [Tipo Documento],
		pac.Identificacion Documento,
		CAST(FechaFactura AS DATE) Fecha,
		'' [# Autorización],
		ACT.CodigoNacional [Codigo CUP],
		1 Ambito,
		2 Finalidad,
		'' Personal,
		'' [DX Principal],
		'' [DX Relacionado 1],
		2  [DX Complicaciones],
		1 [Realizacion],
		M.CodigoResponsable,
		A.NombreResponsable	,
		 ((B.ValorFactura*B.IndicadorFacturable)-B.ValorIvaFactura)*A.EstadoActivo ValorFactura1,
		 ACT.Nombre
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('PROCEDIMIENTOS')
AND S.Codigo in (SELECT Codigo FROM GEN_CNF_Sedes WHERE CodigoTributarioCiudad = @codigo_ciudad)
AND  P5.ParametroId <> @tipo_atencion
and  NOT EXISTS (SELECT  NOTAS.FacturaId,NOtas.Numero,  B.FacturaAfectadaId
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					LEFT JOIN FAC_MOV_FacturasAnexos B WITH (NOLOCK) ON NOTAS.FacturaId = B.FacturaEncabezadoId
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaAfectadaId = A.FacturaId  AND NOTAS.FechaFactura> @desde )
				) tabla
				where fila in (1,2)
	)	TABLA
group by
		[Codigo Prestador],
		[Razon Social],
		[Tipo Identificacion],
		[Identificacion],
		Factura,
		Fecha,
		FechaInicial,
		FechaDesde,
		CodigoResponsable,
		NombreResponsable,
		[# Contrato],
		[Plan Beneficios],
		[# Poliza],
		[Valor Copago],
		[Valor Descuento]
'''

# Information converted into a DataFrame with pandas
df = pd.read_sql(sql=sqlQuery, con=connection)

# Export the data on the excel into a Desktop
df.to_csv(os.environ["userprofile"] + "\\Documents\\Rips\\Bogota\\DIV\\" + "Bogota_DIV_Facturas " +
            datetime.now().strftime("%d-%b-%Y %H%M%S") + ".csv", index=False)

C:\Users\ddrodriguez\AppData\Local\Temp\ipykernel_19832\2241811764.py:234: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=sqlQuery, con=connection)


## Procedimientos

In [ ]:
# SQL command to read the data
sqlQuery = '''

--PROCEDIMIENTOS BOGOTA DIV

DECLARE @hoy DATETIME = GETDATE()
DECLARE @hasta DATETIME = EOMONTH(@hoy, -1);
DECLARE @desde DATETIME =  DATETRUNC(month,@hasta),
		@tipo_atencion INT = 6081,
		@codigo_ciudad VARCHAR(5) = '11001'
/*
6081	IVE-TAI- Interrupción Voluntaria Embarazo

11001	BOGOTA
*/


select *
from(
SELECT	Row_number() over(partition by pac.Identificacion,ACT.ActividadId order by pac.Identificacion,FechaFactura) fila,
		DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		'110010893004' [Codigo Prestador],
		SUBSTRING(pac.TipoIdentificacion,1,2) [Tipo Documento],
		pac.Identificacion Documento,
		CAST(FechaFactura AS DATE) Fecha,
		C.NumeroAutorizacion [# Autorizaci�n],
		ACT.CodigoNacional [Codigo CUP],
		1 Ambito,
		2 Finalidad,
		'' Personal,
		'' [DX Principal],
		'' [DX Relacionado 1],
		2  [DX Complicaciones],
		1 [Realizacion],
		 ((B.ValorFactura*B.IndicadorFacturable)-B.ValorIvaFactura)*A.EstadoActivo ValorFactura1,
		 ACT.Nombre
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('PROCEDIMIENTOS')
AND S.Codigo in (SELECT Codigo FROM GEN_CNF_Sedes WHERE CodigoTributarioCiudad = @codigo_ciudad)
AND  P5.ParametroId <> @tipo_atencion

and  NOT EXISTS (SELECT  NOTAS.FacturaId,NOtas.Numero,  B.FacturaAfectadaId
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					LEFT JOIN FAC_MOV_FacturasAnexos B WITH (NOLOCK) ON NOTAS.FacturaId = B.FacturaEncabezadoId
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaAfectadaId = A.FacturaId  AND NOTAS.FechaFactura> @desde )
				) tabla
				where fila IN (1,2)
'''

# Information converted into a DataFrame with pandas
df = pd.read_sql(sql=sqlQuery, con=connection)

# Export the data on the excel into a Desktop
df.to_csv(os.environ["userprofile"] + "\\Documents\\Rips\\Bogota\\DIV\\" + "Bogota_DIV_Procedimientos " +
            datetime.now().strftime("%d-%b-%Y %H%M%S") + ".csv", index=False)

C:\Users\ddrodriguez\AppData\Local\Temp\ipykernel_19832\1389990529.py:93: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=sqlQuery, con=connection)


## Usuarios

In [ ]:
# SQL command to read the data
sqlQuery = '''

--USUARIOS BOGOTA DIV

DECLARE @hoy DATETIME = GETDATE()
DECLARE @hasta DATETIME = EOMONTH(@hoy, -1);
DECLARE @desde DATETIME =  DATETRUNC(month,@hasta),
		@tipo_atencion INT = 6081,
		@codigo_ciudad VARCHAR(5) = '11001'
/*
6081	IVE-TAI- Interrupción Voluntaria Embarazo

11001	BOGOTA
*/


SELECT
	DISTINCT
		TABLA.[Tipo Documento],
		TABLA.Documento,
		CodigoResponsable,
		case when CodigoResponsable = 'SDS001' THEN 4 ELSE 1 END TipoUsuaurio,
		COALESCE(pac.PrimerApellido,'') PrimerApellido,
		COALESCE(pac.SegundoApellido,'') SegundoApellido,
		COALESCE(pac.PrimerNombre,'') PrimerNombre,
		COALESCE(pac.SegundoNombre,'') SegundoNombre,
		pac.FechaNacimiento,
		[dbo].[Fun_ORI_CalculaEdad](pac.FechaNacimiento,Fecha) Edad,
		1 Unidad,
		substring(Genero,1,1) Sexo,
		substring(ciudad.Referencia1,1,2) Depertamento,
		substring(ciudad.Referencia1,3,3) Ciudad,
		'U' Zona,
		case when pais.Valor = 'Colombia' or TABLA.[Tipo Documento] in ('CC','TI') then 'Colombia'

		else pais.Valor end Nacionalidad,
		case when pais.Valor = 'Colombia' or TABLA.[Tipo Documento] in ('CC','TI') then 'Nacional'

		else 'Extranjera' end TipoNacionalidad
FROM (
SELECT  '110010893004' [Codigo Prestador],
		'FUNDACION ORIENTAME' [Razon Social],
		'NI' [TipoNit],
		'860049972' Nit,
		SUBSTRING(pac.TipoIdentificacion,1,2) [Tipo Documento],
		pac.Identificacion Documento,
		DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		CAST(FechaFactura AS DATE) Fecha,
		CAST(@desde AS DATE) FechaInicial,
		CAST(@hasta AS DATE) FechaDesde,
		CASE
			WHEN M.CodigoResponsable = '900156264' THEN 'EPS037'
			WHEN M.CodigoResponsable = '830003564' THEN 'EPS017'
			WHEN M.CodigoResponsable = '800088702' THEN 'EPS018'
			WHEN M.CodigoResponsable = '830002272' THEN 'EPS019'
			WHEN M.CodigoResponsable = '901127521' THEN 'EPS020'
			ELSE 'SDS001' END CodigoResponsable,

		CASE
			WHEN M.CodigoResponsable = '900156264' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '830003564' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '800088702' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '830002272' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '901127521' THEN A.NombreResponsable
			ELSE 'FUNDACION ORIENTAME' END NombreResponsable,
		'' [# Contrato],
		'IVE' [Plan Beneficios],
		'' [# Poliza],
		'' [Valor Copago],
		'' [Valor Descuento],
		((B.ValorFactura * B.IndicadorFacturable ) - B.ValorIvaFactura ) * A.EstadoActivo ValorFactura1
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('CONSULTA')
AND S.Codigo in (SELECT Codigo FROM GEN_CNF_Sedes WHERE CodigoTributarioCiudad = @codigo_ciudad)
AND  P5.ParametroId <> @tipo_atencion
and  NOT EXISTS (SELECT  *
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaId = A.FacturaId AND NOTAS.FechaFactura > @desde)
UNION ALL
select  '110010893004' [Codigo Prestador],
		'FUNDACION ORIENTAME' [Razon Social],
		'NI' [TipoNit],
		'860049972' Nit,
		[Tipo Documento],
		Documento,
		Factura,
		Fecha,
		CAST(@desde AS DATE) FechaInicial,
		CAST(@hasta AS DATE) FechaDesde,
				CASE

			WHEN CodigoResponsable = '900156264' THEN 'EPS037'
			WHEN CodigoResponsable = '830003564' THEN 'EPS017'
			WHEN CodigoResponsable = '800088702' THEN 'EPS018'
			WHEN CodigoResponsable = '830002272' THEN 'EPS019'
			WHEN CodigoResponsable = '901127521' THEN 'EPS020' ELSE 'SDS001' END CodigoResponsable, --	P6.Valor ,
				CASE

			WHEN CodigoResponsable = '900156264' THEN NombreResponsable
			WHEN CodigoResponsable = '830003564' THEN NombreResponsable
			WHEN CodigoResponsable = '800088702' THEN NombreResponsable
			WHEN CodigoResponsable = '830002272' THEN NombreResponsable
			WHEN CodigoResponsable = '901127521' THEN NombreResponsable ELSE 'FUNDACION ORIENTAME' END NombreResponsable	,
		'' [# Contrato],
		'IVE' [Plan Beneficios],
		'' [# Poliza],
		'' [Valor Copago],
		'' [Valor Descuento],
		ValorFactura1
from(
 SELECT	Row_number() over(partition by pac.Identificacion,ACT.ActividadId order by pac.Identificacion,A.Numero ASC) fila,
		DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		'110010893004' [Codigo Prestador],
		SUBSTRING(pac.TipoIdentificacion,1,2) [Tipo Documento],
		pac.Identificacion Documento,
		CAST(FechaFactura AS DATE) Fecha,
		'' [# Autorizaci�n],
		ACT.CodigoNacional [Codigo CUP],
		1 Ambito,
		2 Finalidad,
		'' Personal,
		'' [DX Principal],
		'' [DX Relacionado 1],
		2  [DX Complicaciones],
		1 [Realizacion],
			M.CodigoResponsable, --	P6.Valor ,
		A.NombreResponsable	,
		 ((B.ValorFactura*B.IndicadorFacturable)-B.ValorIvaFactura)*A.EstadoActivo ValorFactura1,
		 ACT.Nombre
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('PROCEDIMIENTOS')
AND S.Codigo in (SELECT Codigo FROM GEN_CNF_Sedes WHERE CodigoTributarioCiudad = @codigo_ciudad)
AND  P5.ParametroId <> @tipo_atencion
and  NOT EXISTS (SELECT  NOTAS.FacturaId,NOtas.Numero,  B.FacturaAfectadaId
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					LEFT JOIN FAC_MOV_FacturasAnexos B WITH (NOLOCK) ON NOTAS.FacturaId = B.FacturaEncabezadoId
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaAfectadaId = A.FacturaId  AND NOTAS.FechaFactura> @desde )
				) tabla
					where fila in (1,2)
	)	TABLA
	INNER JOIN ADT_VIS_Pacientes pac with (nolock) ON pac.Identificacion = TABLA.Documento and SUBSTRING(pac.TipoIdentificacion,1,2) = TABLA.[Tipo Documento]
	left join GEN_CNF_Parametros ciudad on ciudad.ParametroId = MunicipioResidenciaId
	left join GEN_CNF_Parametros nacimiento on nacimiento.ParametroId = LugarNacimientoId
	left join GEN_CNF_Parametros departamento on departamento.ParametroId = nacimiento.PadreId
	left join GEN_CNF_Parametros pais on pais.ParametroId = departamento.PadreId
'''

# Information converted into a DataFrame with pandas
df = pd.read_sql(sql=sqlQuery, con=connection)

# Export the data on the excel into a Desktop
df.to_csv(os.environ["userprofile"] + "\\Documents\\Rips\\Bogota\\DIV\\" + "Bogota_DIV_Usuarios " +
            datetime.now().strftime("%d-%b-%Y %H%M%S") + ".csv", index=False)

#Display notification to user
notification.notify(title="✅Report Status!!!",
                    message= f"Queries of Bogota DIV has been succesfully saved into .csv\
                        \n Queries Executed:  \n 1. Consultas, 2.Facturas, 3.Procedimientos, 4.Usuarios",
                        timeout=10)

C:\Users\ddrodriguez\AppData\Local\Temp\ipykernel_19832\1072102481.py:234: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=sqlQuery, con=connection)


# **Queries of Empresas🏢**

En esta sección se encuentran las consultas correspondientes a "EMPRESAS" cada una de las consultas se guardan como un archivo .csv por aparte en la carpeta C:\Users\ddrodriguez\Documents\RIPS\Empresas. y separadas por empresa prestadora

<font color='Gold' size="6">FAMISANAR</font>
##

###
<font color='Gold' size="5"> Consultas Famisanar </font>

In [ ]:
# SQL command to read the data
sqlQuery = '''

--CONSULTAS FAMISANAR

DECLARE @hoy DATETIME = GETDATE()
DECLARE @hasta DATETIME = EOMONTH(@hoy, -1);
DECLARE @desde DATETIME =  DATETRUNC(month,@hasta),
		@empresa INT = 1

/*
1	FAMISANAR SAS
2	NUEVA EPS SA
4	SERVICIO OCCIDENTAL DE SALUD S.O.S
5	SURA EPS
6	UT SERVISALUD SAN JOSE
7	UNISALUD
10	SERVIMED IPS SA
13	EPS COMFAMILIAR NARI�O
16	DEPARTAMENTO DE CUNDINAMARCA
20	WOMENS LINK WORLDWIDE
25	FONDO DE POBLACIONES DE LAS NACIONES UNIDAS
29	COLEGIO COLOMBIANO DE PSICOLOGIA
43	I E D LLANO ORIENTAL
*/



SELECT	DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		'110010893004' [Codigo Prestador],
		SUBSTRING(pac.TipoIdentificacion,1,2) [Tipo Documento],
		pac.Identificacion Documento,
		CAST(FechaFactura AS DATE) Fecha,
		C.NumeroAutorizacion [# Autorizaci�n],
		ACT.CodigoNacional [Codigo CUP],
		10 Finalidad,
		13 Causa,
		'' [DX Principal],
		'' [DX Relacionado 1],
		'' [DX Relacionado 2],
		'' [DX Relacionado 3],
		2  [Tipo DX],
		 ((B.ValorFactura*B.IndicadorFacturable)-B.ValorIvaFactura)*A.EstadoActivo ValorFactura1,
		'' ValorCopago,
		 ((B.ValorFactura*B.IndicadorFacturable)-B.ValorIvaFactura)*A.EstadoActivo ValorFactura,
		 ACT.Nombre
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('CONSULTA')
AND A.EmpresaId = @empresa
and  NOT EXISTS (SELECT  *
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaId = A.FacturaId AND NOTAS.FechaFactura > @desde)
'''

# Information converted into a DataFrame with pandas
df = pd.read_sql(sql=sqlQuery, con=connection)

# Export the data on the excel into a Desktop
df.to_csv(os.environ["userprofile"] + "\\Documents\\Rips\\Empresas\\Famisanar\\" + "Famisanar_Consultas " +
            datetime.now().strftime("%d-%b-%Y %H%M%S") + ".csv", index=False)

C:\Users\ddrodriguez\AppData\Local\Temp\ipykernel_19832\2507567680.py:98: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=sqlQuery, con=connection)


###
<font color='Gold' size="5"> Facturas Famisanar </font>

In [ ]:
# SQL command to read the data
sqlQuery = '''

--FACTURAS FAMISANAR

DECLARE @hoy DATETIME = GETDATE()
DECLARE @hasta DATETIME = EOMONTH(@hoy, -1);
DECLARE @desde DATETIME =  DATETRUNC(month,@hasta),
		@empresa INT = 1,
		@fila int = 1
/*
1	FAMISANAR SAS
2	NUEVA EPS SA
4	SERVICIO OCCIDENTAL DE SALUD S.O.S
5	SURA EPS
6	UT SERVISALUD SAN JOSE
7	UNISALUD
10	SERVIMED IPS SA
13	EPS COMFAMILIAR NARIÑO
16	DEPARTAMENTO DE CUNDINAMARCA
20	WOMENS LINK WORLDWIDE
25	FONDO DE POBLACIONES DE LAS NACIONES UNIDAS
29	COLEGIO COLOMBIANO DE PSICOLOGIA
43	I E D LLANO ORIENTAL
*/


SELECT
		[Codigo Prestador],
		[Razon Social],
		[Tipo Identificacion],
		[Identificacion],
		Factura,
		Fecha,
		FechaInicial,
		FechaDesde,
		CodigoResponsable,
		NombreResponsable,
		[# Contrato],
		[Plan Beneficios],
		[# Poliza],
		[Valor Copago],
		[Valor Descuento],
		sum(ValorFactura1) Valor
FROM (
SELECT  '110010893004' [Codigo Prestador],
		'FUNDACION ORIENTAME' [Razon Social],
		'NI' [Tipo Identificacion],
		'860049972' [Identificacion],
		DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		CAST(FechaFactura AS DATE) Fecha,
		CAST(@desde AS DATE) FechaInicial,
		CAST(@hasta AS DATE) FechaDesde,
		CASE
			WHEN M.CodigoResponsable = '900156264' THEN 'EPS037'
			WHEN M.CodigoResponsable = '830003564' THEN 'EPS017'
			WHEN M.CodigoResponsable = '800088702' THEN 'EPS018'
			WHEN M.CodigoResponsable = '830002272' THEN 'EPS019'
			WHEN M.CodigoResponsable = '901127521' THEN 'EPS020'
			ELSE 'SDS001' END CodigoResponsable,

		CASE
			WHEN M.CodigoResponsable = '900156264' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '830003564' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '800088702' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '830002272' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '901127521' THEN A.NombreResponsable
			ELSE 'FUNDACION ORIENTAME' END NombreResponsable,
		'' [# Contrato],
		'IVE' [Plan Beneficios],
		'' [# Poliza],
		'' [Valor Copago],
		'' [Valor Descuento],
		((B.ValorFactura * B.IndicadorFacturable ) - B.ValorIvaFactura ) * A.EstadoActivo ValorFactura1
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('CONSULTA')
AND A.EmpresaId = @empresa
and  NOT EXISTS (SELECT  *
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaId = A.FacturaId AND NOTAS.FechaFactura > @desde)
UNION ALL
select  '110010893004' [Codigo Prestador],
		'FUNDACION ORIENTAME' [Razon Social],
		'NI' [Tipo Identificacion],
		'860049972' [Identificacion],
		Factura,
		Fecha,
		CAST(@desde AS DATE) FechaInicial,
		CAST(@hasta AS DATE) FechaDesde,
				CASE

			WHEN CodigoResponsable = '900156264' THEN 'EPS037'
			WHEN CodigoResponsable = '830003564' THEN 'EPS017'
			WHEN CodigoResponsable = '800088702' THEN 'EPS018'
			WHEN CodigoResponsable = '830002272' THEN 'EPS019'
			WHEN CodigoResponsable = '901127521' THEN 'EPS020' ELSE 'SDS001' END CodigoResponsable, --	P6.Valor ,
				CASE

			WHEN CodigoResponsable = '900156264' THEN NombreResponsable
			WHEN CodigoResponsable = '830003564' THEN NombreResponsable
			WHEN CodigoResponsable = '800088702' THEN NombreResponsable
			WHEN CodigoResponsable = '830002272' THEN NombreResponsable
			WHEN CodigoResponsable = '901127521' THEN NombreResponsable ELSE 'FUNDACION ORIENTAME' END NombreResponsable	,
		'' [# Contrato],
		'IVE' [Plan Beneficios],
		'' [# Poliza],
		'' [Valor Copago],
		'' [Valor Descuento],
		ValorFactura1
from(
 SELECT	Row_number() over(partition by pac.Identificacion,ACT.ActividadId order by pac.Identificacion,A.Numero ASC) fila,
		DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		'110010893004' [Codigo Prestador],
		SUBSTRING(pac.TipoIdentificacion,1,2) [Tipo Documento],
		pac.Identificacion Documento,
		CAST(FechaFactura AS DATE) Fecha,
		'' [# Autorizaci�n],
		ACT.CodigoNacional [Codigo CUP],
		1 Ambito,
		2 Finalidad,
		'' Personal,
		'' [DX Principal],
		'' [DX Relacionado 1],
		2  [DX Complicaciones],
		1 [Realizacion],
		M.CodigoResponsable,
		A.NombreResponsable	,
		 ((B.ValorFactura*B.IndicadorFacturable)-B.ValorIvaFactura)*A.EstadoActivo ValorFactura1,
		 ACT.Nombre
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('PROCEDIMIENTOS')
AND M.EmpresaId = @empresa
and  NOT EXISTS (SELECT  NOTAS.FacturaId,NOtas.Numero,  B.FacturaAfectadaId
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					LEFT JOIN FAC_MOV_FacturasAnexos B WITH (NOLOCK) ON NOTAS.FacturaId = B.FacturaEncabezadoId
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaAfectadaId = A.FacturaId  AND NOTAS.FechaFactura> @desde )
				) tabla
				where fila =@fila
	)	TABLA
group by
		[Codigo Prestador],
		[Razon Social],
		[Tipo Identificacion],
		[Identificacion],
		Factura,
		Fecha,
		FechaInicial,
		FechaDesde,
		CodigoResponsable,
		NombreResponsable,
		[# Contrato],
		[Plan Beneficios],
		[# Poliza],
		[Valor Copago],
		[Valor Descuento]
'''

# Information converted into a DataFrame with pandas
df = pd.read_sql(sql=sqlQuery, con=connection)

# Export the data on the excel into a Desktop
df.to_csv(os.environ["userprofile"] + "\\Documents\\Rips\\Empresas\\Famisanar\\" + "Famisanar_Facturas " +
            datetime.now().strftime("%d-%b-%Y %H%M%S") + ".csv", index=False)

C:\Users\ddrodriguez\AppData\Local\Temp\ipykernel_19832\2053471990.py:243: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=sqlQuery, con=connection)


###
<font color='Gold' size="5"> Procedimientos Famisanar </font>

In [ ]:
# SQL command to read the data
sqlQuery = '''

--PROCEDIMIENTOS FAMISANAR

DECLARE @hoy DATETIME = GETDATE()
DECLARE @hasta DATETIME = EOMONTH(@hoy, -1);
DECLARE @desde DATETIME =  DATETRUNC(month,@hasta),
		@empresa INT = 1,
		@fila int = 1
/*
1	FAMISANAR SAS
2	NUEVA EPS SA
4	SERVICIO OCCIDENTAL DE SALUD S.O.S
5	SURA EPS
6	UT SERVISALUD SAN JOSE
7	UNISALUD
10	SERVIMED IPS SA
13	EPS COMFAMILIAR NARIÑO
16	DEPARTAMENTO DE CUNDINAMARCA
20	WOMENS LINK WORLDWIDE
25	FONDO DE POBLACIONES DE LAS NACIONES UNIDAS
29	COLEGIO COLOMBIANO DE PSICOLOGIA
43	I E D LLANO ORIENTAL
*/

select *
from(
SELECT	Row_number() over(partition by pac.Identificacion,ACT.ActividadId order by pac.Identificacion,FechaFactura) fila,
		DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		'110010893004' [Codigo Prestador],
		SUBSTRING(pac.TipoIdentificacion,1,2) [Tipo Documento],
		pac.Identificacion Documento,
		CAST(FechaFactura AS DATE) Fecha,
		C.NumeroAutorizacion [# Autorizaci�n],
		ACT.CodigoNacional [Codigo CUP],
		1 Ambito,
		2 Finalidad,
		'' Personal,
		'' [DX Principal],
		'' [DX Relacionado 1],
		2  [DX Complicaciones],
		1 [Realizacion],
		 ((B.ValorFactura*B.IndicadorFacturable)-B.ValorIvaFactura)*A.EstadoActivo ValorFactura1,
		 ACT.Nombre
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('PROCEDIMIENTOS')
AND M.EmpresaId = @empresa
and  NOT EXISTS (SELECT  NOTAS.FacturaId,NOtas.Numero,  B.FacturaAfectadaId
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					LEFT JOIN FAC_MOV_FacturasAnexos B WITH (NOLOCK) ON NOTAS.FacturaId = B.FacturaEncabezadoId
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaAfectadaId = A.FacturaId  AND NOTAS.FechaFactura> @desde )
				) tabla
				where fila =@fila
'''

# Information converted into a DataFrame with pandas
df = pd.read_sql(sql=sqlQuery, con=connection)

# Export the data on the excel into a Desktop
df.to_csv(os.environ["userprofile"] + "\\Documents\\Rips\\Empresas\\Famisanar\\" + "Famisanar_Procedimientos " +
            datetime.now().strftime("%d-%b-%Y %H%M%S") + ".csv", index=False)

C:\Users\ddrodriguez\AppData\Local\Temp\ipykernel_19832\318310003.py:100: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=sqlQuery, con=connection)


###
<font color='Gold' size="5"> Usuarios Famisanar </font>

In [ ]:
# SQL command to read the data
sqlQuery = '''

--USUARIOS FAMISANAR

DECLARE @hoy DATETIME = GETDATE()
DECLARE @hasta DATETIME = EOMONTH(@hoy, -1);
DECLARE @desde DATETIME =  DATETRUNC(month,@hasta),
		@empresa INT = 1,
		@fila int = 1
/*
1	FAMISANAR SAS
2	NUEVA EPS SA
4	SERVICIO OCCIDENTAL DE SALUD S.O.S
5	SURA EPS
6	UT SERVISALUD SAN JOSE
7	UNISALUD
13	EPS COMFAMILIAR NARIÑO
16	DEPARTAMENTO DE CUNDINAMARCA
20	WOMENS LINK WORLDWIDE
25	FONDO DE POBLACIONES DE LAS NACIONES UNIDAS
29	COLEGIO COLOMBIANO DE PSICOLOGIA
43	I E D LLANO ORIENTAL
*/

SELECT
	DISTINCT
		TABLA.[Tipo Documento],
		TABLA.Documento,
		CodigoResponsable,
		case when CodigoResponsable = 'SDS001' THEN 4 ELSE 1 END TipoUsuaurio,
		COALESCE(pac.PrimerApellido,'') PrimerApellido,
		COALESCE(pac.SegundoApellido,'') SegundoApellido,
		COALESCE(pac.PrimerNombre,'') PrimerNombre,
		COALESCE(pac.SegundoNombre,'') SegundoNombre,
		pac.FechaNacimiento,
		[dbo].[Fun_ORI_CalculaEdad](pac.FechaNacimiento,Fecha) Edad,
		1 Unidad,
		substring(Genero,1,1) Sexo,
		substring(ciudad.Referencia1,1,2) Depertamento,
		substring(ciudad.Referencia1,3,3) Ciudad,
		'U' Zona,
		case when pais.Valor = 'Colombia' or TABLA.[Tipo Documento] in ('CC','TI') then 'Colombia'

		else pais.Valor end Nacionalidad,
		case when pais.Valor = 'Colombia' or TABLA.[Tipo Documento] in ('CC','TI') then 'Nacional'

		else 'Extranjera' end TipoNacionalidad
FROM (
SELECT  '110010893004' [Codigo Prestador],
		'FUNDACION ORIENTAME' [Razon Social],
		'NI' [TipoNit],
		'860049972' Nit,
		SUBSTRING(pac.TipoIdentificacion,1,2) [Tipo Documento],
		pac.Identificacion Documento,
		DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		CAST(FechaFactura AS DATE) Fecha,
		CAST(@desde AS DATE) FechaInicial,
		CAST(@hasta AS DATE) FechaDesde,
		CASE
			WHEN M.CodigoResponsable = '900156264' THEN 'EPS037'
			WHEN M.CodigoResponsable = '830003564' THEN 'EPS017'
			WHEN M.CodigoResponsable = '800088702' THEN 'EPS018'
			WHEN M.CodigoResponsable = '830002272' THEN 'EPS019'
			WHEN M.CodigoResponsable = '901127521' THEN 'EPS020'
			ELSE 'SDS001' END CodigoResponsable,

		CASE
			WHEN M.CodigoResponsable = '900156264' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '830003564' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '800088702' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '830002272' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '901127521' THEN A.NombreResponsable
			ELSE 'FUNDACION ORIENTAME' END NombreResponsable,
		'' [# Contrato],
		'IVE' [Plan Beneficios],
		'' [# Poliza],
		'' [Valor Copago],
		'' [Valor Descuento],
		((B.ValorFactura * B.IndicadorFacturable ) - B.ValorIvaFactura ) * A.EstadoActivo ValorFactura1
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('CONSULTA')
AND A.EmpresaId = @empresa
and  NOT EXISTS (SELECT  *
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaId = A.FacturaId AND NOTAS.FechaFactura > @desde)
UNION ALL
select  '110010893004' [Codigo Prestador],
		'FUNDACION ORIENTAME' [Razon Social],
		'NI' [TipoNit],
		'860049972' Nit,
		[Tipo Documento],
		Documento,
		Factura,
		Fecha,
		CAST(@desde AS DATE) FechaInicial,
		CAST(@hasta AS DATE) FechaDesde,
				CASE

			WHEN CodigoResponsable = '900156264' THEN 'EPS037'
			WHEN CodigoResponsable = '830003564' THEN 'EPS017'
			WHEN CodigoResponsable = '800088702' THEN 'EPS018'
			WHEN CodigoResponsable = '830002272' THEN 'EPS019'
			WHEN CodigoResponsable = '901127521' THEN 'EPS020' ELSE 'SDS001' END CodigoResponsable, --	P6.Valor ,
				CASE

			WHEN CodigoResponsable = '900156264' THEN NombreResponsable
			WHEN CodigoResponsable = '830003564' THEN NombreResponsable
			WHEN CodigoResponsable = '800088702' THEN NombreResponsable
			WHEN CodigoResponsable = '830002272' THEN NombreResponsable
			WHEN CodigoResponsable = '901127521' THEN NombreResponsable ELSE 'FUNDACION ORIENTAME' END NombreResponsable	,
		'' [# Contrato],
		'IVE' [Plan Beneficios],
		'' [# Poliza],
		'' [Valor Copago],
		'' [Valor Descuento],
		ValorFactura1
from(
 SELECT	Row_number() over(partition by pac.Identificacion,ACT.ActividadId order by pac.Identificacion,A.Numero ASC) fila,
		DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		'110010893004' [Codigo Prestador],
		SUBSTRING(pac.TipoIdentificacion,1,2) [Tipo Documento],
		pac.Identificacion Documento,
		CAST(FechaFactura AS DATE) Fecha,
		'' [# Autorizaci�n],
		ACT.CodigoNacional [Codigo CUP],
		1 Ambito,
		2 Finalidad,
		'' Personal,
		'' [DX Principal],
		'' [DX Relacionado 1],
		2  [DX Complicaciones],
		1 [Realizacion],
			M.CodigoResponsable, --	P6.Valor ,
		A.NombreResponsable	,
		 ((B.ValorFactura*B.IndicadorFacturable)-B.ValorIvaFactura)*A.EstadoActivo ValorFactura1,
		 ACT.Nombre
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('PROCEDIMIENTOS')
AND M.EmpresaId = @empresa
and  NOT EXISTS (SELECT  NOTAS.FacturaId,NOtas.Numero,  B.FacturaAfectadaId
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					LEFT JOIN FAC_MOV_FacturasAnexos B WITH (NOLOCK) ON NOTAS.FacturaId = B.FacturaEncabezadoId
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaAfectadaId = A.FacturaId  AND NOTAS.FechaFactura> @desde )
				) tabla
				where fila =@fila
	)	TABLA
	INNER JOIN ADT_VIS_Pacientes pac with (nolock) ON pac.Identificacion = TABLA.Documento and SUBSTRING(pac.TipoIdentificacion,1,2) = TABLA.[Tipo Documento]
	left join GEN_CNF_Parametros ciudad on ciudad.ParametroId = MunicipioResidenciaId
	left join GEN_CNF_Parametros nacimiento on nacimiento.ParametroId = LugarNacimientoId
	left join GEN_CNF_Parametros departamento on departamento.ParametroId = nacimiento.PadreId
	left join GEN_CNF_Parametros pais on pais.ParametroId = departamento.PadreId
'''

# Information converted into a DataFrame with pandas
df = pd.read_sql(sql=sqlQuery, con=connection)

# Export the data on the excel into a Desktop
df.to_csv(os.environ["userprofile"] + "\\Documents\\Rips\\Empresas\\Famisanar\\" + "Famisanar_Usuarios " +
            datetime.now().strftime("%d-%b-%Y %H%M%S") + ".csv", index=False)

#Display notification to user
notification.notify(title="✅Report Status!!!",
                    message= f"Queries Famisanar has been succesfully saved into .csv\
                        \n Queries Executed:  \n 1. Consultas, 2.Facturas, 3.Procedimientos, 4.Usuarios",
                        timeout=10)

C:\Users\ddrodriguez\AppData\Local\Temp\ipykernel_19832\241848140.py:240: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=sqlQuery, con=connection)


##
<font color='Tomato' size="6"> NUEVA EPS </font>

###
<font color='Tomato' size="5"> Consultas Nueva EPS </font>

In [ ]:
# SQL command to read the data
sqlQuery = '''

--CONSULTAS Nueva EPS

DECLARE @hoy DATETIME = GETDATE()
DECLARE @hasta DATETIME = EOMONTH(@hoy, -1);
DECLARE @desde DATETIME =  DATETRUNC(month,@hasta),
		@empresa INT = 2

/*
1	FAMISANAR SAS
2	NUEVA EPS SA
4	SERVICIO OCCIDENTAL DE SALUD S.O.S
5	SURA EPS
6	UT SERVISALUD SAN JOSE
7	UNISALUD
10	SERVIMED IPS SA
13	EPS COMFAMILIAR NARI�O
16	DEPARTAMENTO DE CUNDINAMARCA
20	WOMENS LINK WORLDWIDE
25	FONDO DE POBLACIONES DE LAS NACIONES UNIDAS
29	COLEGIO COLOMBIANO DE PSICOLOGIA
43	I E D LLANO ORIENTAL
*/



SELECT	DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		'110010893004' [Codigo Prestador],
		SUBSTRING(pac.TipoIdentificacion,1,2) [Tipo Documento],
		pac.Identificacion Documento,
		CAST(FechaFactura AS DATE) Fecha,
		C.NumeroAutorizacion [# Autorizaci�n],
		ACT.CodigoNacional [Codigo CUP],
		10 Finalidad,
		13 Causa,
		'' [DX Principal],
		'' [DX Relacionado 1],
		'' [DX Relacionado 2],
		'' [DX Relacionado 3],
		2  [Tipo DX],
		 ((B.ValorFactura*B.IndicadorFacturable)-B.ValorIvaFactura)*A.EstadoActivo ValorFactura1,
		'' ValorCopago,
		 ((B.ValorFactura*B.IndicadorFacturable)-B.ValorIvaFactura)*A.EstadoActivo ValorFactura,
		 ACT.Nombre
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('CONSULTA')
AND A.EmpresaId = @empresa
and  NOT EXISTS (SELECT  *
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaId = A.FacturaId AND NOTAS.FechaFactura > @desde)

'''

# Information converted into a DataFrame with pandas
df = pd.read_sql(sql=sqlQuery, con=connection)

# Export the data on the excel into a Desktop
df.to_csv(os.environ["userprofile"] + "\\Documents\\Rips\\Empresas\\Nueva EPS\\" + "Nueva_EPS_Consultas " +
            datetime.now().strftime("%d-%b-%Y %H%M%S") + ".csv", index=False)

C:\Users\ddrodriguez\AppData\Local\Temp\ipykernel_19832\1137950601.py:99: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=sqlQuery, con=connection)


###
<font color='Tomato' size="5"> Facturas Nueva EPS </font>

In [ ]:
# SQL command to read the data
sqlQuery = '''

--FACTURAS NUEVA EPS

DECLARE @hoy DATETIME = GETDATE()
DECLARE @hasta DATETIME = EOMONTH(@hoy, -1);
DECLARE @desde DATETIME =  DATETRUNC(month,@hasta),
		@empresa INT = 2,
		@fila int = 1
/*
1	FAMISANAR SAS
2	NUEVA EPS SA
4	SERVICIO OCCIDENTAL DE SALUD S.O.S
5	SURA EPS
6	UT SERVISALUD SAN JOSE
7	UNISALUD
10	SERVIMED IPS SA
13	EPS COMFAMILIAR NARIÑO
16	DEPARTAMENTO DE CUNDINAMARCA
20	WOMENS LINK WORLDWIDE
25	FONDO DE POBLACIONES DE LAS NACIONES UNIDAS
29	COLEGIO COLOMBIANO DE PSICOLOGIA
43	I E D LLANO ORIENTAL
*/


SELECT
		[Codigo Prestador],
		[Razon Social],
		[Tipo Identificacion],
		[Identificacion],
		Factura,
		Fecha,
		FechaInicial,
		FechaDesde,
		CodigoResponsable,
		NombreResponsable,
		[# Contrato],
		[Plan Beneficios],
		[# Poliza],
		[Valor Copago],
		[Valor Descuento],
		sum(ValorFactura1) Valor
FROM (
SELECT  '110010893004' [Codigo Prestador],
		'FUNDACION ORIENTAME' [Razon Social],
		'NI' [Tipo Identificacion],
		'860049972' [Identificacion],
		DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		CAST(FechaFactura AS DATE) Fecha,
		CAST(@desde AS DATE) FechaInicial,
		CAST(@hasta AS DATE) FechaDesde,
		CASE
			WHEN M.CodigoResponsable = '900156264' THEN 'EPS037'
			WHEN M.CodigoResponsable = '830003564' THEN 'EPS017'
			WHEN M.CodigoResponsable = '800088702' THEN 'EPS018'
			WHEN M.CodigoResponsable = '830002272' THEN 'EPS019'
			WHEN M.CodigoResponsable = '901127521' THEN 'EPS020'
			ELSE 'SDS001' END CodigoResponsable,

		CASE
			WHEN M.CodigoResponsable = '900156264' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '830003564' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '800088702' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '830002272' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '901127521' THEN A.NombreResponsable
			ELSE 'FUNDACION ORIENTAME' END NombreResponsable,
		'' [# Contrato],
		'IVE' [Plan Beneficios],
		'' [# Poliza],
		'' [Valor Copago],
		'' [Valor Descuento],
		((B.ValorFactura * B.IndicadorFacturable ) - B.ValorIvaFactura ) * A.EstadoActivo ValorFactura1
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('CONSULTA')
AND A.EmpresaId = @empresa
and  NOT EXISTS (SELECT  *
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaId = A.FacturaId AND NOTAS.FechaFactura > @desde)
UNION ALL
select  '110010893004' [Codigo Prestador],
		'FUNDACION ORIENTAME' [Razon Social],
		'NI' [Tipo Identificacion],
		'860049972' [Identificacion],
		Factura,
		Fecha,
		CAST(@desde AS DATE) FechaInicial,
		CAST(@hasta AS DATE) FechaDesde,
				CASE

			WHEN CodigoResponsable = '900156264' THEN 'EPS037'
			WHEN CodigoResponsable = '830003564' THEN 'EPS017'
			WHEN CodigoResponsable = '800088702' THEN 'EPS018'
			WHEN CodigoResponsable = '830002272' THEN 'EPS019'
			WHEN CodigoResponsable = '901127521' THEN 'EPS020' ELSE 'SDS001' END CodigoResponsable, --	P6.Valor ,
				CASE

			WHEN CodigoResponsable = '900156264' THEN NombreResponsable
			WHEN CodigoResponsable = '830003564' THEN NombreResponsable
			WHEN CodigoResponsable = '800088702' THEN NombreResponsable
			WHEN CodigoResponsable = '830002272' THEN NombreResponsable
			WHEN CodigoResponsable = '901127521' THEN NombreResponsable ELSE 'FUNDACION ORIENTAME' END NombreResponsable	,
		'' [# Contrato],
		'IVE' [Plan Beneficios],
		'' [# Poliza],
		'' [Valor Copago],
		'' [Valor Descuento],
		ValorFactura1
from(
 SELECT	Row_number() over(partition by pac.Identificacion,ACT.ActividadId order by pac.Identificacion,A.Numero ASC) fila,
		DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		'110010893004' [Codigo Prestador],
		SUBSTRING(pac.TipoIdentificacion,1,2) [Tipo Documento],
		pac.Identificacion Documento,
		CAST(FechaFactura AS DATE) Fecha,
		'' [# Autorizaci�n],
		ACT.CodigoNacional [Codigo CUP],
		1 Ambito,
		2 Finalidad,
		'' Personal,
		'' [DX Principal],
		'' [DX Relacionado 1],
		2  [DX Complicaciones],
		1 [Realizacion],
		M.CodigoResponsable,
		A.NombreResponsable	,
		 ((B.ValorFactura*B.IndicadorFacturable)-B.ValorIvaFactura)*A.EstadoActivo ValorFactura1,
		 ACT.Nombre
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('PROCEDIMIENTOS')
AND M.EmpresaId = @empresa
and  NOT EXISTS (SELECT  NOTAS.FacturaId,NOtas.Numero,  B.FacturaAfectadaId
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					LEFT JOIN FAC_MOV_FacturasAnexos B WITH (NOLOCK) ON NOTAS.FacturaId = B.FacturaEncabezadoId
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaAfectadaId = A.FacturaId  AND NOTAS.FechaFactura> @desde )
				) tabla
				where fila =@fila
	)	TABLA
group by
		[Codigo Prestador],
		[Razon Social],
		[Tipo Identificacion],
		[Identificacion],
		Factura,
		Fecha,
		FechaInicial,
		FechaDesde,
		CodigoResponsable,
		NombreResponsable,
		[# Contrato],
		[Plan Beneficios],
		[# Poliza],
		[Valor Copago],
		[Valor Descuento]
'''

# Information converted into a DataFrame with pandas
df = pd.read_sql(sql=sqlQuery, con=connection)

# Export the data on the excel into a Desktop
df.to_csv(os.environ["userprofile"] + "\\Documents\\Rips\\Empresas\\Nueva EPS\\" + "Nueva_EPS_Facturas " +
            datetime.now().strftime("%d-%b-%Y %H%M%S") + ".csv", index=False)

C:\Users\ddrodriguez\AppData\Local\Temp\ipykernel_19832\136192753.py:243: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=sqlQuery, con=connection)


###
<font color='Tomato' size="5"> Procedimientos Nueva EPS </font>

In [ ]:
# SQL command to read the data
sqlQuery = '''

--PROCEDIMIENTOS Nueva EPS

DECLARE @desde datetime ='20231201',
		@hasta DATETIME ='20231231 23:59:59',
		@empresa INT = 2,
		@fila int = 1
/*
1	FAMISANAR SAS
2	NUEVA EPS SA
4	SERVICIO OCCIDENTAL DE SALUD S.O.S
5	SURA EPS
6	UT SERVISALUD SAN JOSE
7	UNISALUD
10	SERVIMED IPS SA
13	EPS COMFAMILIAR NARIÑO
16	DEPARTAMENTO DE CUNDINAMARCA
20	WOMENS LINK WORLDWIDE
25	FONDO DE POBLACIONES DE LAS NACIONES UNIDAS
29	COLEGIO COLOMBIANO DE PSICOLOGIA
43	I E D LLANO ORIENTAL
*/

select *
from(
SELECT	Row_number() over(partition by pac.Identificacion,ACT.ActividadId order by pac.Identificacion,FechaFactura) fila,
		DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		'110010893004' [Codigo Prestador],
		SUBSTRING(pac.TipoIdentificacion,1,2) [Tipo Documento],
		pac.Identificacion Documento,
		CAST(FechaFactura AS DATE) Fecha,
		C.NumeroAutorizacion [# Autorizaci�n],
		ACT.CodigoNacional [Codigo CUP],
		1 Ambito,
		2 Finalidad,
		'' Personal,
		'' [DX Principal],
		'' [DX Relacionado 1],
		2  [DX Complicaciones],
		1 [Realizacion],
		 ((B.ValorFactura*B.IndicadorFacturable)-B.ValorIvaFactura)*A.EstadoActivo ValorFactura1,
		 ACT.Nombre
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('PROCEDIMIENTOS')
AND M.EmpresaId = @empresa
and  NOT EXISTS (SELECT  NOTAS.FacturaId,NOtas.Numero,  B.FacturaAfectadaId
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					LEFT JOIN FAC_MOV_FacturasAnexos B WITH (NOLOCK) ON NOTAS.FacturaId = B.FacturaEncabezadoId
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaAfectadaId = A.FacturaId  AND NOTAS.FechaFactura> @desde )
				) tabla
				where fila =@fila
'''

# Information converted into a DataFrame with pandas
df = pd.read_sql(sql=sqlQuery, con=connection)

# Export the data on the excel into a Desktop
df.to_csv(os.environ["userprofile"] + "\\Documents\\Rips\\Empresas\\Nueva EPS\\" + "Nueva_EPS_Procedimientos " +
            datetime.now().strftime("%d-%b-%Y %H%M%S") + ".csv", index=False)

C:\Users\ddrodriguez\AppData\Local\Temp\ipykernel_19832\3059553990.py:99: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=sqlQuery, con=connection)


###
<font color='Tomato' size="5"> Usuarios Nueva EPS </font>

In [ ]:
# SQL command to read the data
sqlQuery = '''

--USUARIOS NUEVA EPS

DECLARE @hoy DATETIME = GETDATE()
DECLARE @hasta DATETIME = EOMONTH(@hoy, -1);
DECLARE @desde DATETIME =  DATETRUNC(month,@hasta),
		@empresa INT = 2,
		@fila int = 1
/*
1	FAMISANAR SAS
2	NUEVA EPS SA
4	SERVICIO OCCIDENTAL DE SALUD S.O.S
5	SURA EPS
6	UT SERVISALUD SAN JOSE
7	UNISALUD
13	EPS COMFAMILIAR NARIÑO
16	DEPARTAMENTO DE CUNDINAMARCA
20	WOMENS LINK WORLDWIDE
25	FONDO DE POBLACIONES DE LAS NACIONES UNIDAS
29	COLEGIO COLOMBIANO DE PSICOLOGIA
43	I E D LLANO ORIENTAL
*/

SELECT
	DISTINCT
		TABLA.[Tipo Documento],
		TABLA.Documento,
		CodigoResponsable,
		case when CodigoResponsable = 'SDS001' THEN 4 ELSE 1 END TipoUsuaurio,
		COALESCE(pac.PrimerApellido,'') PrimerApellido,
		COALESCE(pac.SegundoApellido,'') SegundoApellido,
		COALESCE(pac.PrimerNombre,'') PrimerNombre,
		COALESCE(pac.SegundoNombre,'') SegundoNombre,
		pac.FechaNacimiento,
		[dbo].[Fun_ORI_CalculaEdad](pac.FechaNacimiento,Fecha) Edad,
		1 Unidad,
		substring(Genero,1,1) Sexo,
		substring(ciudad.Referencia1,1,2) Depertamento,
		substring(ciudad.Referencia1,3,3) Ciudad,
		'U' Zona,
		case when pais.Valor = 'Colombia' or TABLA.[Tipo Documento] in ('CC','TI') then 'Colombia'

		else pais.Valor end Nacionalidad,
		case when pais.Valor = 'Colombia' or TABLA.[Tipo Documento] in ('CC','TI') then 'Nacional'

		else 'Extranjera' end TipoNacionalidad
FROM (
SELECT  '110010893004' [Codigo Prestador],
		'FUNDACION ORIENTAME' [Razon Social],
		'NI' [TipoNit],
		'860049972' Nit,
		SUBSTRING(pac.TipoIdentificacion,1,2) [Tipo Documento],
		pac.Identificacion Documento,
		DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		CAST(FechaFactura AS DATE) Fecha,
		CAST(@desde AS DATE) FechaInicial,
		CAST(@hasta AS DATE) FechaDesde,
		CASE
			WHEN M.CodigoResponsable = '900156264' THEN 'EPS037'
			WHEN M.CodigoResponsable = '830003564' THEN 'EPS017'
			WHEN M.CodigoResponsable = '800088702' THEN 'EPS018'
			WHEN M.CodigoResponsable = '830002272' THEN 'EPS019'
			WHEN M.CodigoResponsable = '901127521' THEN 'EPS020'
			ELSE 'SDS001' END CodigoResponsable,

		CASE
			WHEN M.CodigoResponsable = '900156264' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '830003564' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '800088702' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '830002272' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '901127521' THEN A.NombreResponsable
			ELSE 'FUNDACION ORIENTAME' END NombreResponsable,
		'' [# Contrato],
		'IVE' [Plan Beneficios],
		'' [# Poliza],
		'' [Valor Copago],
		'' [Valor Descuento],
		((B.ValorFactura * B.IndicadorFacturable ) - B.ValorIvaFactura ) * A.EstadoActivo ValorFactura1
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('CONSULTA')
AND A.EmpresaId = @empresa
and  NOT EXISTS (SELECT  *
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaId = A.FacturaId AND NOTAS.FechaFactura > @desde)
UNION ALL
select  '110010893004' [Codigo Prestador],
		'FUNDACION ORIENTAME' [Razon Social],
		'NI' [TipoNit],
		'860049972' Nit,
		[Tipo Documento],
		Documento,
		Factura,
		Fecha,
		CAST(@desde AS DATE) FechaInicial,
		CAST(@hasta AS DATE) FechaDesde,
				CASE

			WHEN CodigoResponsable = '900156264' THEN 'EPS037'
			WHEN CodigoResponsable = '830003564' THEN 'EPS017'
			WHEN CodigoResponsable = '800088702' THEN 'EPS018'
			WHEN CodigoResponsable = '830002272' THEN 'EPS019'
			WHEN CodigoResponsable = '901127521' THEN 'EPS020' ELSE 'SDS001' END CodigoResponsable, --	P6.Valor ,
				CASE

			WHEN CodigoResponsable = '900156264' THEN NombreResponsable
			WHEN CodigoResponsable = '830003564' THEN NombreResponsable
			WHEN CodigoResponsable = '800088702' THEN NombreResponsable
			WHEN CodigoResponsable = '830002272' THEN NombreResponsable
			WHEN CodigoResponsable = '901127521' THEN NombreResponsable ELSE 'FUNDACION ORIENTAME' END NombreResponsable	,
		'' [# Contrato],
		'IVE' [Plan Beneficios],
		'' [# Poliza],
		'' [Valor Copago],
		'' [Valor Descuento],
		ValorFactura1
from(
 SELECT	Row_number() over(partition by pac.Identificacion,ACT.ActividadId order by pac.Identificacion,A.Numero ASC) fila,
		DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		'110010893004' [Codigo Prestador],
		SUBSTRING(pac.TipoIdentificacion,1,2) [Tipo Documento],
		pac.Identificacion Documento,
		CAST(FechaFactura AS DATE) Fecha,
		'' [# Autorizaci�n],
		ACT.CodigoNacional [Codigo CUP],
		1 Ambito,
		2 Finalidad,
		'' Personal,
		'' [DX Principal],
		'' [DX Relacionado 1],
		2  [DX Complicaciones],
		1 [Realizacion],
			M.CodigoResponsable, --	P6.Valor ,
		A.NombreResponsable	,
		 ((B.ValorFactura*B.IndicadorFacturable)-B.ValorIvaFactura)*A.EstadoActivo ValorFactura1,
		 ACT.Nombre
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('PROCEDIMIENTOS')
AND M.EmpresaId = @empresa
and  NOT EXISTS (SELECT  NOTAS.FacturaId,NOtas.Numero,  B.FacturaAfectadaId
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					LEFT JOIN FAC_MOV_FacturasAnexos B WITH (NOLOCK) ON NOTAS.FacturaId = B.FacturaEncabezadoId
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaAfectadaId = A.FacturaId  AND NOTAS.FechaFactura> @desde )
				) tabla
				where fila =@fila
	)	TABLA
	INNER JOIN ADT_VIS_Pacientes pac with (nolock) ON pac.Identificacion = TABLA.Documento and SUBSTRING(pac.TipoIdentificacion,1,2) = TABLA.[Tipo Documento]
	left join GEN_CNF_Parametros ciudad on ciudad.ParametroId = MunicipioResidenciaId
	left join GEN_CNF_Parametros nacimiento on nacimiento.ParametroId = LugarNacimientoId
	left join GEN_CNF_Parametros departamento on departamento.ParametroId = nacimiento.PadreId
	left join GEN_CNF_Parametros pais on pais.ParametroId = departamento.PadreId
'''

# Information converted into a DataFrame with pandas
df = pd.read_sql(sql=sqlQuery, con=connection)

# Export the data on the excel into a Desktop
df.to_csv(os.environ["userprofile"] + "\\Documents\\Rips\\Empresas\\Nueva EPS\\" + "Nueva_EPS_Usuarios " +
            datetime.now().strftime("%d-%b-%Y %H%M%S") + ".csv", index=False)

#Display notification to user
notification.notify(title="✅Report Status!!!",
                    message= f"Queries of Nueva EPS has been succesfully saved into .csv\
                        \n Queries Executed:  \n 1. Consultas, 2.Facturas, 3.Procedimientos, 4.Usuarios",
                        timeout=10)

C:\Users\ddrodriguez\AppData\Local\Temp\ipykernel_19832\279580920.py:240: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=sqlQuery, con=connection)


##
<font color='SpringGreen' size="6">SERVICIO DE SALUD </font>

###
<font color='SpringGreen' size="5"> Consultas Servicio de salud </font>

In [ ]:
# SQL command to read the data
sqlQuery = '''

--CONSULTAS SERVICIO DE SALUD

DECLARE @hoy DATETIME = GETDATE()
DECLARE @hasta DATETIME = EOMONTH(@hoy, -1);
DECLARE @desde DATETIME =  DATETRUNC(month,@hasta),
		@empresa INT = 6
/*
1	FAMISANAR SAS
2	NUEVA EPS SA
4	SERVICIO OCCIDENTAL DE SALUD S.O.S
5	SURA EPS
6	UT SERVISALUD SAN JOSE
7	UNISALUD
10	SERVIMED IPS SA
13	EPS COMFAMILIAR NARI�O
16	DEPARTAMENTO DE CUNDINAMARCA
20	WOMENS LINK WORLDWIDE
25	FONDO DE POBLACIONES DE LAS NACIONES UNIDAS
29	COLEGIO COLOMBIANO DE PSICOLOGIA
43	I E D LLANO ORIENTAL
*/



SELECT	DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		'110010893004' [Codigo Prestador],
		SUBSTRING(pac.TipoIdentificacion,1,2) [Tipo Documento],
		pac.Identificacion Documento,
		CAST(FechaFactura AS DATE) Fecha,
		C.NumeroAutorizacion [# Autorizaci�n],
		ACT.CodigoNacional [Codigo CUP],
		10 Finalidad,
		13 Causa,
		'' [DX Principal],
		'' [DX Relacionado 1],
		'' [DX Relacionado 2],
		'' [DX Relacionado 3],
		2  [Tipo DX],
		 ((B.ValorFactura*B.IndicadorFacturable)-B.ValorIvaFactura)*A.EstadoActivo ValorFactura1,
		'' ValorCopago,
		 ((B.ValorFactura*B.IndicadorFacturable)-B.ValorIvaFactura)*A.EstadoActivo ValorFactura,
		 ACT.Nombre
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('CONSULTA')
AND A.EmpresaId = @empresa
and  NOT EXISTS (SELECT  *
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaId = A.FacturaId AND NOTAS.FechaFactura > @desde)
'''

# Information converted into a DataFrame with pandas
df = pd.read_sql(sql=sqlQuery, con=connection)

# Export the data on the excel into a Desktop
df.to_csv(os.environ["userprofile"] + "\\Documents\\Rips\\Empresas\\Servicio de salud\\" + "Servicio_de_salud_Consultas " +
            datetime.now().strftime("%d-%b-%Y %H%M%S") + ".csv", index=False)

C:\Users\ddrodriguez\AppData\Local\Temp\ipykernel_19832\1665062919.py:97: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=sqlQuery, con=connection)


###
<font color='SpringGreen' size="5"> Facturas Servicios de salud </font>

In [ ]:
sqlQuery = '''

--FACTURAS SERVICIOS DE SALUD

DECLARE @hoy DATETIME = GETDATE()
DECLARE @hasta DATETIME = EOMONTH(@hoy, -1);
DECLARE @desde DATETIME =  DATETRUNC(month,@hasta),
		@empresa INT = 6,
		@fila int = 1
/*
1	FAMISANAR SAS
2	NUEVA EPS SA
4	SERVICIO OCCIDENTAL DE SALUD S.O.S
5	SURA EPS
6	UT SERVISALUD SAN JOSE
7	UNISALUD
10	SERVIMED IPS SA
13	EPS COMFAMILIAR NARIÑO
16	DEPARTAMENTO DE CUNDINAMARCA
20	WOMENS LINK WORLDWIDE
25	FONDO DE POBLACIONES DE LAS NACIONES UNIDAS
29	COLEGIO COLOMBIANO DE PSICOLOGIA
43	I E D LLANO ORIENTAL
*/


SELECT
		[Codigo Prestador],
		[Razon Social],
		[Tipo Identificacion],
		[Identificacion],
		Factura,
		Fecha,
		FechaInicial,
		FechaDesde,
		CodigoResponsable,
		NombreResponsable,
		[# Contrato],
		[Plan Beneficios],
		[# Poliza],
		[Valor Copago],
		[Valor Descuento],
		sum(ValorFactura1) Valor
FROM (
SELECT  '110010893004' [Codigo Prestador],
		'FUNDACION ORIENTAME' [Razon Social],
		'NI' [Tipo Identificacion],
		'860049972' [Identificacion],
		DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		CAST(FechaFactura AS DATE) Fecha,
		CAST(@desde AS DATE) FechaInicial,
		CAST(@hasta AS DATE) FechaDesde,
		CASE
			WHEN M.CodigoResponsable = '900156264' THEN 'EPS037'
			WHEN M.CodigoResponsable = '830003564' THEN 'EPS017'
			WHEN M.CodigoResponsable = '800088702' THEN 'EPS018'
			WHEN M.CodigoResponsable = '830002272' THEN 'EPS019'
			WHEN M.CodigoResponsable = '901127521' THEN 'EPS020'
			ELSE 'SDS001' END CodigoResponsable,

		CASE
			WHEN M.CodigoResponsable = '900156264' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '830003564' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '800088702' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '830002272' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '901127521' THEN A.NombreResponsable
			ELSE 'FUNDACION ORIENTAME' END NombreResponsable,
		'' [# Contrato],
		'IVE' [Plan Beneficios],
		'' [# Poliza],
		'' [Valor Copago],
		'' [Valor Descuento],
		((B.ValorFactura * B.IndicadorFacturable ) - B.ValorIvaFactura ) * A.EstadoActivo ValorFactura1
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('CONSULTA')
AND A.EmpresaId = @empresa
and  NOT EXISTS (SELECT  *
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaId = A.FacturaId AND NOTAS.FechaFactura > @desde)
UNION ALL
select  '110010893004' [Codigo Prestador],
		'FUNDACION ORIENTAME' [Razon Social],
		'NI' [Tipo Identificacion],
		'860049972' [Identificacion],
		Factura,
		Fecha,
		CAST(@desde AS DATE) FechaInicial,
		CAST(@hasta AS DATE) FechaDesde,
				CASE

			WHEN CodigoResponsable = '900156264' THEN 'EPS037'
			WHEN CodigoResponsable = '830003564' THEN 'EPS017'
			WHEN CodigoResponsable = '800088702' THEN 'EPS018'
			WHEN CodigoResponsable = '830002272' THEN 'EPS019'
			WHEN CodigoResponsable = '901127521' THEN 'EPS020' ELSE 'SDS001' END CodigoResponsable, --	P6.Valor ,
				CASE

			WHEN CodigoResponsable = '900156264' THEN NombreResponsable
			WHEN CodigoResponsable = '830003564' THEN NombreResponsable
			WHEN CodigoResponsable = '800088702' THEN NombreResponsable
			WHEN CodigoResponsable = '830002272' THEN NombreResponsable
			WHEN CodigoResponsable = '901127521' THEN NombreResponsable ELSE 'FUNDACION ORIENTAME' END NombreResponsable	,
		'' [# Contrato],
		'IVE' [Plan Beneficios],
		'' [# Poliza],
		'' [Valor Copago],
		'' [Valor Descuento],
		ValorFactura1
from(
 SELECT	Row_number() over(partition by pac.Identificacion,ACT.ActividadId order by pac.Identificacion,A.Numero ASC) fila,
		DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		'110010893004' [Codigo Prestador],
		SUBSTRING(pac.TipoIdentificacion,1,2) [Tipo Documento],
		pac.Identificacion Documento,
		CAST(FechaFactura AS DATE) Fecha,
		'' [# Autorizaci�n],
		ACT.CodigoNacional [Codigo CUP],
		1 Ambito,
		2 Finalidad,
		'' Personal,
		'' [DX Principal],
		'' [DX Relacionado 1],
		2  [DX Complicaciones],
		1 [Realizacion],
		M.CodigoResponsable,
		A.NombreResponsable	,
		 ((B.ValorFactura*B.IndicadorFacturable)-B.ValorIvaFactura)*A.EstadoActivo ValorFactura1,
		 ACT.Nombre
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('PROCEDIMIENTOS')
AND M.EmpresaId = @empresa
and  NOT EXISTS (SELECT  NOTAS.FacturaId,NOtas.Numero,  B.FacturaAfectadaId
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					LEFT JOIN FAC_MOV_FacturasAnexos B WITH (NOLOCK) ON NOTAS.FacturaId = B.FacturaEncabezadoId
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaAfectadaId = A.FacturaId  AND NOTAS.FechaFactura> @desde )
				) tabla
				where fila =@fila
	)	TABLA
group by
		[Codigo Prestador],
		[Razon Social],
		[Tipo Identificacion],
		[Identificacion],
		Factura,
		Fecha,
		FechaInicial,
		FechaDesde,
		CodigoResponsable,
		NombreResponsable,
		[# Contrato],
		[Plan Beneficios],
		[# Poliza],
		[Valor Copago],
		[Valor Descuento]
'''

# Information converted into a DataFrame with pandas
df = pd.read_sql(sql=sqlQuery, con=connection)

# Export the data on the excel into a Desktop
df.to_csv(os.environ["userprofile"] + "\\Documents\\Rips\\Empresas\\Servicio de salud\\" + "Servicio_de_salud_Facturas " +
            datetime.now().strftime("%d-%b-%Y %H%M%S") + ".csv", index=False)

C:\Users\ddrodriguez\AppData\Local\Temp\ipykernel_19832\1611547533.py:242: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=sqlQuery, con=connection)


###
<font color='SpringGreen' size="5"> Procedimientos Servicio de salud </font>

In [ ]:
# SQL command to read the data
sqlQuery = '''

--PROCEDIMIENTOS Servicios de salud

DECLARE @hoy DATETIME = GETDATE()
DECLARE @hasta DATETIME = EOMONTH(@hoy, -1);
DECLARE @desde DATETIME =  DATETRUNC(month,@hasta),
		@empresa INT = 6,
		@fila int = 1
/*
1	FAMISANAR SAS
2	NUEVA EPS SA
4	SERVICIO OCCIDENTAL DE SALUD S.O.S
5	SURA EPS
6	UT SERVISALUD SAN JOSE
7	UNISALUD
10	SERVIMED IPS SA
13	EPS COMFAMILIAR NARIÑO
16	DEPARTAMENTO DE CUNDINAMARCA
20	WOMENS LINK WORLDWIDE
25	FONDO DE POBLACIONES DE LAS NACIONES UNIDAS
29	COLEGIO COLOMBIANO DE PSICOLOGIA
43	I E D LLANO ORIENTAL
*/

select *
from(
SELECT	Row_number() over(partition by pac.Identificacion,ACT.ActividadId order by pac.Identificacion,FechaFactura) fila,
		DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		'110010893004' [Codigo Prestador],
		SUBSTRING(pac.TipoIdentificacion,1,2) [Tipo Documento],
		pac.Identificacion Documento,
		CAST(FechaFactura AS DATE) Fecha,
		C.NumeroAutorizacion [# Autorizaci�n],
		ACT.CodigoNacional [Codigo CUP],
		1 Ambito,
		2 Finalidad,
		'' Personal,
		'' [DX Principal],
		'' [DX Relacionado 1],
		2  [DX Complicaciones],
		1 [Realizacion],
		 ((B.ValorFactura*B.IndicadorFacturable)-B.ValorIvaFactura)*A.EstadoActivo ValorFactura1,
		 ACT.Nombre
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('PROCEDIMIENTOS')
AND M.EmpresaId = @empresa
and  NOT EXISTS (SELECT  NOTAS.FacturaId,NOtas.Numero,  B.FacturaAfectadaId
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					LEFT JOIN FAC_MOV_FacturasAnexos B WITH (NOLOCK) ON NOTAS.FacturaId = B.FacturaEncabezadoId
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaAfectadaId = A.FacturaId  AND NOTAS.FechaFactura> @desde )
				) tabla
				where fila =@fila
'''

# Information converted into a DataFrame with pandas
df = pd.read_sql(sql=sqlQuery, con=connection)

# Export the data on the excel into a Desktop
df.to_csv(os.environ["userprofile"] + "\\Documents\\Rips\\Empresas\\Servicio de salud\\" + "Servicio_de_salud_Procedimientos " +
            datetime.now().strftime("%d-%b-%Y %H%M%S") + ".csv", index=False)

C:\Users\ddrodriguez\AppData\Local\Temp\ipykernel_19832\3911485664.py:100: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=sqlQuery, con=connection)


###
<font color='SpringGreen' size="5"> Usuarios Servicio de salud </font>

In [ ]:
sqlQuery = '''

--USUARIOS SERVICIO DE SALUD

DECLARE @hoy DATETIME = GETDATE()
DECLARE @hasta DATETIME = EOMONTH(@hoy, -1);
DECLARE @desde DATETIME =  DATETRUNC(month,@hasta),
		@empresa INT = 6,
		@fila int = 1
/*
1	FAMISANAR SAS
2	NUEVA EPS SA
4	SERVICIO OCCIDENTAL DE SALUD S.O.S
5	SURA EPS
6	UT SERVISALUD SAN JOSE
7	UNISALUD
13	EPS COMFAMILIAR NARIÑO
16	DEPARTAMENTO DE CUNDINAMARCA
20	WOMENS LINK WORLDWIDE
25	FONDO DE POBLACIONES DE LAS NACIONES UNIDAS
29	COLEGIO COLOMBIANO DE PSICOLOGIA
43	I E D LLANO ORIENTAL
*/

SELECT
	DISTINCT
		TABLA.[Tipo Documento],
		TABLA.Documento,
		CodigoResponsable,
		case when CodigoResponsable = 'SDS001' THEN 4 ELSE 1 END TipoUsuaurio,
		COALESCE(pac.PrimerApellido,'') PrimerApellido,
		COALESCE(pac.SegundoApellido,'') SegundoApellido,
		COALESCE(pac.PrimerNombre,'') PrimerNombre,
		COALESCE(pac.SegundoNombre,'') SegundoNombre,
		pac.FechaNacimiento,
		[dbo].[Fun_ORI_CalculaEdad](pac.FechaNacimiento,Fecha) Edad,
		1 Unidad,
		substring(Genero,1,1) Sexo,
		substring(ciudad.Referencia1,1,2) Depertamento,
		substring(ciudad.Referencia1,3,3) Ciudad,
		'U' Zona,
		case when pais.Valor = 'Colombia' or TABLA.[Tipo Documento] in ('CC','TI') then 'Colombia'

		else pais.Valor end Nacionalidad,
		case when pais.Valor = 'Colombia' or TABLA.[Tipo Documento] in ('CC','TI') then 'Nacional'

		else 'Extranjera' end TipoNacionalidad
FROM (
SELECT  '110010893004' [Codigo Prestador],
		'FUNDACION ORIENTAME' [Razon Social],
		'NI' [TipoNit],
		'860049972' Nit,
		SUBSTRING(pac.TipoIdentificacion,1,2) [Tipo Documento],
		pac.Identificacion Documento,
		DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		CAST(FechaFactura AS DATE) Fecha,
		CAST(@desde AS DATE) FechaInicial,
		CAST(@hasta AS DATE) FechaDesde,
		CASE
			WHEN M.CodigoResponsable = '900156264' THEN 'EPS037'
			WHEN M.CodigoResponsable = '830003564' THEN 'EPS017'
			WHEN M.CodigoResponsable = '800088702' THEN 'EPS018'
			WHEN M.CodigoResponsable = '830002272' THEN 'EPS019'
			WHEN M.CodigoResponsable = '901127521' THEN 'EPS020'
			ELSE 'SDS001' END CodigoResponsable,

		CASE
			WHEN M.CodigoResponsable = '900156264' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '830003564' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '800088702' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '830002272' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '901127521' THEN A.NombreResponsable
			ELSE 'FUNDACION ORIENTAME' END NombreResponsable,
		'' [# Contrato],
		'IVE' [Plan Beneficios],
		'' [# Poliza],
		'' [Valor Copago],
		'' [Valor Descuento],
		((B.ValorFactura * B.IndicadorFacturable ) - B.ValorIvaFactura ) * A.EstadoActivo ValorFactura1
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('CONSULTA')
AND A.EmpresaId = @empresa
and  NOT EXISTS (SELECT  *
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaId = A.FacturaId AND NOTAS.FechaFactura > @desde)
UNION ALL
select  '110010893004' [Codigo Prestador],
		'FUNDACION ORIENTAME' [Razon Social],
		'NI' [TipoNit],
		'860049972' Nit,
		[Tipo Documento],
		Documento,
		Factura,
		Fecha,
		CAST(@desde AS DATE) FechaInicial,
		CAST(@hasta AS DATE) FechaDesde,
				CASE

			WHEN CodigoResponsable = '900156264' THEN 'EPS037'
			WHEN CodigoResponsable = '830003564' THEN 'EPS017'
			WHEN CodigoResponsable = '800088702' THEN 'EPS018'
			WHEN CodigoResponsable = '830002272' THEN 'EPS019'
			WHEN CodigoResponsable = '901127521' THEN 'EPS020' ELSE 'SDS001' END CodigoResponsable, --	P6.Valor ,
				CASE

			WHEN CodigoResponsable = '900156264' THEN NombreResponsable
			WHEN CodigoResponsable = '830003564' THEN NombreResponsable
			WHEN CodigoResponsable = '800088702' THEN NombreResponsable
			WHEN CodigoResponsable = '830002272' THEN NombreResponsable
			WHEN CodigoResponsable = '901127521' THEN NombreResponsable ELSE 'FUNDACION ORIENTAME' END NombreResponsable	,
		'' [# Contrato],
		'IVE' [Plan Beneficios],
		'' [# Poliza],
		'' [Valor Copago],
		'' [Valor Descuento],
		ValorFactura1
from(
 SELECT	Row_number() over(partition by pac.Identificacion,ACT.ActividadId order by pac.Identificacion,A.Numero ASC) fila,
		DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		'110010893004' [Codigo Prestador],
		SUBSTRING(pac.TipoIdentificacion,1,2) [Tipo Documento],
		pac.Identificacion Documento,
		CAST(FechaFactura AS DATE) Fecha,
		'' [# Autorizaci�n],
		ACT.CodigoNacional [Codigo CUP],
		1 Ambito,
		2 Finalidad,
		'' Personal,
		'' [DX Principal],
		'' [DX Relacionado 1],
		2  [DX Complicaciones],
		1 [Realizacion],
			M.CodigoResponsable, --	P6.Valor ,
		A.NombreResponsable	,
		 ((B.ValorFactura*B.IndicadorFacturable)-B.ValorIvaFactura)*A.EstadoActivo ValorFactura1,
		 ACT.Nombre
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('PROCEDIMIENTOS')
AND M.EmpresaId = @empresa
and  NOT EXISTS (SELECT  NOTAS.FacturaId,NOtas.Numero,  B.FacturaAfectadaId
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					LEFT JOIN FAC_MOV_FacturasAnexos B WITH (NOLOCK) ON NOTAS.FacturaId = B.FacturaEncabezadoId
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaAfectadaId = A.FacturaId  AND NOTAS.FechaFactura> @desde )
				) tabla
				where fila =@fila
	)	TABLA
	INNER JOIN ADT_VIS_Pacientes pac with (nolock) ON pac.Identificacion = TABLA.Documento and SUBSTRING(pac.TipoIdentificacion,1,2) = TABLA.[Tipo Documento]
	left join GEN_CNF_Parametros ciudad on ciudad.ParametroId = MunicipioResidenciaId
	left join GEN_CNF_Parametros nacimiento on nacimiento.ParametroId = LugarNacimientoId
	left join GEN_CNF_Parametros departamento on departamento.ParametroId = nacimiento.PadreId
	left join GEN_CNF_Parametros pais on pais.ParametroId = departamento.PadreId
'''

# Information converted into a DataFrame with pandas
df = pd.read_sql(sql=sqlQuery, con=connection)

# Export the data on the excel into a Desktop
df.to_csv(os.environ["userprofile"] + "\\Documents\\Rips\\Empresas\\Servicio de salud\\" + "Servicio_de_salud_Usuarios " +
            datetime.now().strftime("%d-%b-%Y %H%M%S") + ".csv", index=False)

#Display notification to user
notification.notify(title="✅Report Status!!!",
                    message= f"Queries of Servicio de Salud has been succesfully saved into .csv\
                        \n Queries Executed:  \n 1. Consultas, 2.Facturas, 3.Procedimientos, 4.Usuarios",
                        timeout=10)

C:\Users\ddrodriguez\AppData\Local\Temp\ipykernel_19832\1092378650.py:239: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=sqlQuery, con=connection)


##
<font color='SteelBlue' size="6"> SURA </font>

###
<font color='SteelBlue' size="5"> Consultas Sura </font>

In [ ]:
# SQL command to read the data
sqlQuery = '''

--CONSULTAS SURA

DECLARE @hoy DATETIME = GETDATE()
DECLARE @hasta DATETIME = EOMONTH(@hoy, -1);
DECLARE @desde DATETIME =  DATETRUNC(month,@hasta),
		@empresa INT = 5

/*
1	FAMISANAR SAS
2	NUEVA EPS SA
4	SERVICIO OCCIDENTAL DE SALUD S.O.S
5	SURA EPS
6	UT SERVISALUD SAN JOSE
7	UNISALUD
10	SERVIMED IPS SA
13	EPS COMFAMILIAR NARI�O
16	DEPARTAMENTO DE CUNDINAMARCA
20	WOMENS LINK WORLDWIDE
25	FONDO DE POBLACIONES DE LAS NACIONES UNIDAS
29	COLEGIO COLOMBIANO DE PSICOLOGIA
43	I E D LLANO ORIENTAL
*/



SELECT	DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		'110010893004' [Codigo Prestador],
		SUBSTRING(pac.TipoIdentificacion,1,2) [Tipo Documento],
		pac.Identificacion Documento,
		CAST(FechaFactura AS DATE) Fecha,
		C.NumeroAutorizacion [# Autorizaci�n],
		ACT.CodigoNacional [Codigo CUP],
		10 Finalidad,
		13 Causa,
		'' [DX Principal],
		'' [DX Relacionado 1],
		'' [DX Relacionado 2],
		'' [DX Relacionado 3],
		2  [Tipo DX],
		 ((B.ValorFactura*B.IndicadorFacturable)-B.ValorIvaFactura)*A.EstadoActivo ValorFactura1,
		'' ValorCopago,
		 ((B.ValorFactura*B.IndicadorFacturable)-B.ValorIvaFactura)*A.EstadoActivo ValorFactura,
		 ACT.Nombre
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('CONSULTA')
AND A.EmpresaId = @empresa
and  NOT EXISTS (SELECT  *
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaId = A.FacturaId AND NOTAS.FechaFactura > @desde)
'''

# Information converted into a DataFrame with pandas
df = pd.read_sql(sql=sqlQuery, con=connection)

# Export the data on the excel into a Desktop
df.to_csv(os.environ["userprofile"] + "\\Documents\\Rips\\Empresas\\Sura\\" + "Sura_Consultas " +
            datetime.now().strftime("%d-%b-%Y %H%M%S") + ".csv", index=False)

C:\Users\ddrodriguez\AppData\Local\Temp\ipykernel_19832\326606209.py:98: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=sqlQuery, con=connection)


###
<font color='SteelBlue' size="5"> Facturas Sura </font>

In [ ]:
# SQL command to read the data
sqlQuery = '''
DECLARE @desde datetime ='20231201',
		@hasta DATETIME ='20231231 23:59:59',
		@empresa INT = 5,
		@fila int = 1
/*
1	FAMISANAR SAS
2	NUEVA EPS SA
4	SERVICIO OCCIDENTAL DE SALUD S.O.S
5	SURA EPS
6	UT SERVISALUD SAN JOSE
7	UNISALUD
10	SERVIMED IPS SA
13	EPS COMFAMILIAR NARIÑO
16	DEPARTAMENTO DE CUNDINAMARCA
20	WOMENS LINK WORLDWIDE
25	FONDO DE POBLACIONES DE LAS NACIONES UNIDAS
29	COLEGIO COLOMBIANO DE PSICOLOGIA
43	I E D LLANO ORIENTAL
*/


SELECT
		[Codigo Prestador],
		[Razon Social],
		[Tipo Identificacion],
		[Identificacion],
		Factura,
		Fecha,
		FechaInicial,
		FechaDesde,
		CodigoResponsable,
		NombreResponsable,
		[# Contrato],
		[Plan Beneficios],
		[# Poliza],
		[Valor Copago],
		[Valor Descuento],
		sum(ValorFactura1) Valor
FROM (
SELECT  '110010893004' [Codigo Prestador],
		'FUNDACION ORIENTAME' [Razon Social],
		'NI' [Tipo Identificacion],
		'860049972' [Identificacion],
		DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		CAST(FechaFactura AS DATE) Fecha,
		CAST(@desde AS DATE) FechaInicial,
		CAST(@hasta AS DATE) FechaDesde,
		CASE
			WHEN M.CodigoResponsable = '900156264' THEN 'EPS037'
			WHEN M.CodigoResponsable = '830003564' THEN 'EPS017'
			WHEN M.CodigoResponsable = '800088702' THEN 'EPS018'
			WHEN M.CodigoResponsable = '830002272' THEN 'EPS019'
			WHEN M.CodigoResponsable = '901127521' THEN 'EPS020'
			ELSE 'SDS001' END CodigoResponsable,

		CASE
			WHEN M.CodigoResponsable = '900156264' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '830003564' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '800088702' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '830002272' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '901127521' THEN A.NombreResponsable
			ELSE 'FUNDACION ORIENTAME' END NombreResponsable,
		'' [# Contrato],
		'IVE' [Plan Beneficios],
		'' [# Poliza],
		'' [Valor Copago],
		'' [Valor Descuento],
		((B.ValorFactura * B.IndicadorFacturable ) - B.ValorIvaFactura ) * A.EstadoActivo ValorFactura1
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('CONSULTA')
AND A.EmpresaId = @empresa
and  NOT EXISTS (SELECT  *
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaId = A.FacturaId AND NOTAS.FechaFactura > @desde)
UNION ALL
select  '110010893004' [Codigo Prestador],
		'FUNDACION ORIENTAME' [Razon Social],
		'NI' [Tipo Identificacion],
		'860049972' [Identificacion],
		Factura,
		Fecha,
		CAST(@desde AS DATE) FechaInicial,
		CAST(@hasta AS DATE) FechaDesde,
				CASE

			WHEN CodigoResponsable = '900156264' THEN 'EPS037'
			WHEN CodigoResponsable = '830003564' THEN 'EPS017'
			WHEN CodigoResponsable = '800088702' THEN 'EPS018'
			WHEN CodigoResponsable = '830002272' THEN 'EPS019'
			WHEN CodigoResponsable = '901127521' THEN 'EPS020' ELSE 'SDS001' END CodigoResponsable, --	P6.Valor ,
				CASE

			WHEN CodigoResponsable = '900156264' THEN NombreResponsable
			WHEN CodigoResponsable = '830003564' THEN NombreResponsable
			WHEN CodigoResponsable = '800088702' THEN NombreResponsable
			WHEN CodigoResponsable = '830002272' THEN NombreResponsable
			WHEN CodigoResponsable = '901127521' THEN NombreResponsable ELSE 'FUNDACION ORIENTAME' END NombreResponsable	,
		'' [# Contrato],
		'IVE' [Plan Beneficios],
		'' [# Poliza],
		'' [Valor Copago],
		'' [Valor Descuento],
		ValorFactura1
from(
 SELECT	Row_number() over(partition by pac.Identificacion,ACT.ActividadId order by pac.Identificacion,A.Numero ASC) fila,
		DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		'110010893004' [Codigo Prestador],
		SUBSTRING(pac.TipoIdentificacion,1,2) [Tipo Documento],
		pac.Identificacion Documento,
		CAST(FechaFactura AS DATE) Fecha,
		'' [# Autorizaci�n],
		ACT.CodigoNacional [Codigo CUP],
		1 Ambito,
		2 Finalidad,
		'' Personal,
		'' [DX Principal],
		'' [DX Relacionado 1],
		2  [DX Complicaciones],
		1 [Realizacion],
		M.CodigoResponsable,
		A.NombreResponsable	,
		 ((B.ValorFactura*B.IndicadorFacturable)-B.ValorIvaFactura)*A.EstadoActivo ValorFactura1,
		 ACT.Nombre
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('PROCEDIMIENTOS')
AND M.EmpresaId = @empresa
and  NOT EXISTS (SELECT  NOTAS.FacturaId,NOtas.Numero,  B.FacturaAfectadaId
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					LEFT JOIN FAC_MOV_FacturasAnexos B WITH (NOLOCK) ON NOTAS.FacturaId = B.FacturaEncabezadoId
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaAfectadaId = A.FacturaId  AND NOTAS.FechaFactura> @desde )
				) tabla
				where fila =@fila
	)	TABLA
group by
		[Codigo Prestador],
		[Razon Social],
		[Tipo Identificacion],
		[Identificacion],
		Factura,
		Fecha,
		FechaInicial,
		FechaDesde,
		CodigoResponsable,
		NombreResponsable,
		[# Contrato],
		[Plan Beneficios],
		[# Poliza],
		[Valor Copago],
		[Valor Descuento]
'''

# Information converted into a DataFrame with pandas
df = pd.read_sql(sql=sqlQuery, con=connection)

# Export the data on the excel into a Desktop
df.to_csv(os.environ["userprofile"] + "\\Documents\\Rips\\Empresas\\Sura\\" + "Sura_Consultas " +
            datetime.now().strftime("%d-%b-%Y %H%M%S") + ".csv", index=False)

C:\Users\ddrodriguez\AppData\Local\Temp\ipykernel_19832\4087892175.py:239: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=sqlQuery, con=connection)


###
<font color='SteelBlue' size="5"> Procedimientos Sura </font>

In [ ]:
# SQL command to read the data
sqlQuery = '''
--PROCEDIMIENTOS SURA

DECLARE @hoy DATETIME = GETDATE()
DECLARE @hasta DATETIME = EOMONTH(@hoy, -1);
DECLARE @desde DATETIME =  DATETRUNC(month,@hasta),
		@empresa INT = 5,
		@fila int = 1

/*
1	FAMISANAR SAS
2	NUEVA EPS SA
4	SERVICIO OCCIDENTAL DE SALUD S.O.S
5	SURA EPS
6	UT SERVISALUD SAN JOSE
7	UNISALUD
10	SERVIMED IPS SA
13	EPS COMFAMILIAR NARIÑO
16	DEPARTAMENTO DE CUNDINAMARCA
20	WOMENS LINK WORLDWIDE
25	FONDO DE POBLACIONES DE LAS NACIONES UNIDAS
29	COLEGIO COLOMBIANO DE PSICOLOGIA
43	I E D LLANO ORIENTAL
*/

select *
from(
SELECT	Row_number() over(partition by pac.Identificacion,ACT.ActividadId order by pac.Identificacion,FechaFactura) fila,
		DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		'110010893004' [Codigo Prestador],
		SUBSTRING(pac.TipoIdentificacion,1,2) [Tipo Documento],
		pac.Identificacion Documento,
		CAST(FechaFactura AS DATE) Fecha,
		C.NumeroAutorizacion [# Autorizaci�n],
		ACT.CodigoNacional [Codigo CUP],
		1 Ambito,
		2 Finalidad,
		'' Personal,
		'' [DX Principal],
		'' [DX Relacionado 1],
		2  [DX Complicaciones],
		1 [Realizacion],
		 ((B.ValorFactura*B.IndicadorFacturable)-B.ValorIvaFactura)*A.EstadoActivo ValorFactura1,
		 ACT.Nombre
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('PROCEDIMIENTOS')
AND M.EmpresaId = @empresa
and  NOT EXISTS (SELECT  NOTAS.FacturaId,NOtas.Numero,  B.FacturaAfectadaId
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					LEFT JOIN FAC_MOV_FacturasAnexos B WITH (NOLOCK) ON NOTAS.FacturaId = B.FacturaEncabezadoId
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaAfectadaId = A.FacturaId  AND NOTAS.FechaFactura> @desde )
				) tabla
				where fila =@fila
'''

# Information converted into a DataFrame with pandas
df = pd.read_sql(sql=sqlQuery, con=connection)

# Export the data on the excel into a Desktop
df.to_csv(os.environ["userprofile"] + "\\Documents\\Rips\\Empresas\\Sura\\" + "Sura_Procedimientos " +
            datetime.now().strftime("%d-%b-%Y %H%M%S") + ".csv", index=False)

C:\Users\ddrodriguez\AppData\Local\Temp\ipykernel_19832\218376897.py:100: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=sqlQuery, con=connection)


###
<font color='SteelBlue' size="5"> Usuarios Sura </font>

In [ ]:
# SQL command to read the data
sqlQuery = '''

--USUARIOS SURA

DECLARE @hoy DATETIME = GETDATE()
DECLARE @hasta DATETIME = EOMONTH(@hoy, -1);
DECLARE @desde DATETIME =  DATETRUNC(month,@hasta),
		@empresa INT = 5,
		@fila int = 1
/*
1	FAMISANAR SAS
2	NUEVA EPS SA
4	SERVICIO OCCIDENTAL DE SALUD S.O.S
5	SURA EPS
6	UT SERVISALUD SAN JOSE
7	UNISALUD
13	EPS COMFAMILIAR NARIÑO
16	DEPARTAMENTO DE CUNDINAMARCA
20	WOMENS LINK WORLDWIDE
25	FONDO DE POBLACIONES DE LAS NACIONES UNIDAS
29	COLEGIO COLOMBIANO DE PSICOLOGIA
43	I E D LLANO ORIENTAL
*/

SELECT
	DISTINCT
		TABLA.[Tipo Documento],
		TABLA.Documento,
		CodigoResponsable,
		case when CodigoResponsable = 'SDS001' THEN 4 ELSE 1 END TipoUsuaurio,
		COALESCE(pac.PrimerApellido,'') PrimerApellido,
		COALESCE(pac.SegundoApellido,'') SegundoApellido,
		COALESCE(pac.PrimerNombre,'') PrimerNombre,
		COALESCE(pac.SegundoNombre,'') SegundoNombre,
		pac.FechaNacimiento,
		[dbo].[Fun_ORI_CalculaEdad](pac.FechaNacimiento,Fecha) Edad,
		1 Unidad,
		substring(Genero,1,1) Sexo,
		substring(ciudad.Referencia1,1,2) Depertamento,
		substring(ciudad.Referencia1,3,3) Ciudad,
		'U' Zona,
		case when pais.Valor = 'Colombia' or TABLA.[Tipo Documento] in ('CC','TI') then 'Colombia'

		else pais.Valor end Nacionalidad,
		case when pais.Valor = 'Colombia' or TABLA.[Tipo Documento] in ('CC','TI') then 'Nacional'

		else 'Extranjera' end TipoNacionalidad
FROM (
SELECT  '110010893004' [Codigo Prestador],
		'FUNDACION ORIENTAME' [Razon Social],
		'NI' [TipoNit],
		'860049972' Nit,
		SUBSTRING(pac.TipoIdentificacion,1,2) [Tipo Documento],
		pac.Identificacion Documento,
		DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		CAST(FechaFactura AS DATE) Fecha,
		CAST(@desde AS DATE) FechaInicial,
		CAST(@hasta AS DATE) FechaDesde,
		CASE
			WHEN M.CodigoResponsable = '900156264' THEN 'EPS037'
			WHEN M.CodigoResponsable = '830003564' THEN 'EPS017'
			WHEN M.CodigoResponsable = '800088702' THEN 'EPS018'
			WHEN M.CodigoResponsable = '830002272' THEN 'EPS019'
			WHEN M.CodigoResponsable = '901127521' THEN 'EPS020'
			ELSE 'SDS001' END CodigoResponsable,

		CASE
			WHEN M.CodigoResponsable = '900156264' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '830003564' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '800088702' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '830002272' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '901127521' THEN A.NombreResponsable
			ELSE 'FUNDACION ORIENTAME' END NombreResponsable,
		'' [# Contrato],
		'IVE' [Plan Beneficios],
		'' [# Poliza],
		'' [Valor Copago],
		'' [Valor Descuento],
		((B.ValorFactura * B.IndicadorFacturable ) - B.ValorIvaFactura ) * A.EstadoActivo ValorFactura1
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('CONSULTA')
AND A.EmpresaId = @empresa
and  NOT EXISTS (SELECT  *
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaId = A.FacturaId AND NOTAS.FechaFactura > @desde)
UNION ALL
select  '110010893004' [Codigo Prestador],
		'FUNDACION ORIENTAME' [Razon Social],
		'NI' [TipoNit],
		'860049972' Nit,
		[Tipo Documento],
		Documento,
		Factura,
		Fecha,
		CAST(@desde AS DATE) FechaInicial,
		CAST(@hasta AS DATE) FechaDesde,
				CASE

			WHEN CodigoResponsable = '900156264' THEN 'EPS037'
			WHEN CodigoResponsable = '830003564' THEN 'EPS017'
			WHEN CodigoResponsable = '800088702' THEN 'EPS018'
			WHEN CodigoResponsable = '830002272' THEN 'EPS019'
			WHEN CodigoResponsable = '901127521' THEN 'EPS020' ELSE 'SDS001' END CodigoResponsable, --	P6.Valor ,
				CASE

			WHEN CodigoResponsable = '900156264' THEN NombreResponsable
			WHEN CodigoResponsable = '830003564' THEN NombreResponsable
			WHEN CodigoResponsable = '800088702' THEN NombreResponsable
			WHEN CodigoResponsable = '830002272' THEN NombreResponsable
			WHEN CodigoResponsable = '901127521' THEN NombreResponsable ELSE 'FUNDACION ORIENTAME' END NombreResponsable	,
		'' [# Contrato],
		'IVE' [Plan Beneficios],
		'' [# Poliza],
		'' [Valor Copago],
		'' [Valor Descuento],
		ValorFactura1
from(
 SELECT	Row_number() over(partition by pac.Identificacion,ACT.ActividadId order by pac.Identificacion,A.Numero ASC) fila,
		DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		'110010893004' [Codigo Prestador],
		SUBSTRING(pac.TipoIdentificacion,1,2) [Tipo Documento],
		pac.Identificacion Documento,
		CAST(FechaFactura AS DATE) Fecha,
		'' [# Autorizaci�n],
		ACT.CodigoNacional [Codigo CUP],
		1 Ambito,
		2 Finalidad,
		'' Personal,
		'' [DX Principal],
		'' [DX Relacionado 1],
		2  [DX Complicaciones],
		1 [Realizacion],
			M.CodigoResponsable, --	P6.Valor ,
		A.NombreResponsable	,
		 ((B.ValorFactura*B.IndicadorFacturable)-B.ValorIvaFactura)*A.EstadoActivo ValorFactura1,
		 ACT.Nombre
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('PROCEDIMIENTOS')
AND M.EmpresaId = @empresa
and  NOT EXISTS (SELECT  NOTAS.FacturaId,NOtas.Numero,  B.FacturaAfectadaId
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					LEFT JOIN FAC_MOV_FacturasAnexos B WITH (NOLOCK) ON NOTAS.FacturaId = B.FacturaEncabezadoId
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaAfectadaId = A.FacturaId  AND NOTAS.FechaFactura> @desde )
				) tabla
				where fila =@fila
	)	TABLA
	INNER JOIN ADT_VIS_Pacientes pac with (nolock) ON pac.Identificacion = TABLA.Documento and SUBSTRING(pac.TipoIdentificacion,1,2) = TABLA.[Tipo Documento]
	left join GEN_CNF_Parametros ciudad on ciudad.ParametroId = MunicipioResidenciaId
	left join GEN_CNF_Parametros nacimiento on nacimiento.ParametroId = LugarNacimientoId
	left join GEN_CNF_Parametros departamento on departamento.ParametroId = nacimiento.PadreId
	left join GEN_CNF_Parametros pais on pais.ParametroId = departamento.PadreId
'''

# Information converted into a DataFrame with pandas
df = pd.read_sql(sql=sqlQuery, con=connection)

# Export the data on the excel into a Desktop
df.to_csv(os.environ["userprofile"] + "\\Documents\\Rips\\Empresas\\Sura\\" + "Sura_Usuarios " +
            datetime.now().strftime("%d-%b-%Y %H%M%S") + ".csv", index=False)

#Display notification to user
notification.notify(title="✅Report Status!!!",
                    message= f"Queries of Sura has been succesfully saved into .csv\
                        \n Queries Executed:  \n 1. Consultas, 2.Facturas, 3.Procedimientos, 4.Usuarios",
                        timeout=10)

C:\Users\ddrodriguez\AppData\Local\Temp\ipykernel_19832\1965793434.py:240: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=sqlQuery, con=connection)


##
<font color='Wheat' size="6"> SERVIMED </font>

###
<font color='Wheat' size="5"> Consultas Servimed </font>

In [ ]:
# SQL command to read the data
sqlQuery = '''

--CONSULTAS Servimed

DECLARE @hoy DATETIME = GETDATE()
DECLARE @hasta DATETIME = EOMONTH(@hoy, -1);
DECLARE @desde DATETIME =  DATETRUNC(month,@hasta),
		@empresa INT = 10
/*
1	FAMISANAR SAS
2	NUEVA EPS SA
4	SERVICIO OCCIDENTAL DE SALUD S.O.S
5	SURA EPS
6	UT SERVISALUD SAN JOSE
7	UNISALUD
10	SERVIMED IPS SA
13	EPS COMFAMILIAR NARI�O
16	DEPARTAMENTO DE CUNDINAMARCA
20	WOMENS LINK WORLDWIDE
25	FONDO DE POBLACIONES DE LAS NACIONES UNIDAS
29	COLEGIO COLOMBIANO DE PSICOLOGIA
43	I E D LLANO ORIENTAL
*/



SELECT	DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		'110010893004' [Codigo Prestador],
		SUBSTRING(pac.TipoIdentificacion,1,2) [Tipo Documento],
		pac.Identificacion Documento,
		CAST(FechaFactura AS DATE) Fecha,
		C.NumeroAutorizacion [# Autorizaci�n],
		ACT.CodigoNacional [Codigo CUP],
		10 Finalidad,
		13 Causa,
		'' [DX Principal],
		'' [DX Relacionado 1],
		'' [DX Relacionado 2],
		'' [DX Relacionado 3],
		2  [Tipo DX],
		 ((B.ValorFactura*B.IndicadorFacturable)-B.ValorIvaFactura)*A.EstadoActivo ValorFactura1,
		'' ValorCopago,
		 ((B.ValorFactura*B.IndicadorFacturable)-B.ValorIvaFactura)*A.EstadoActivo ValorFactura,
		 ACT.Nombre
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('CONSULTA')
AND A.EmpresaId = @empresa
and  NOT EXISTS (SELECT  *
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaId = A.FacturaId AND NOTAS.FechaFactura > @desde)
'''

# Information converted into a DataFrame with pandas
df = pd.read_sql(sql=sqlQuery, con=connection)

# Export the data on the excel into a Desktop
df.to_csv(os.environ["userprofile"] + "\\Documents\\Rips\\Empresas\\Servimed\\" + "Servimed_Consultas " +
            datetime.now().strftime("%d-%b-%Y %H%M%S") + ".csv", index=False)

C:\Users\ddrodriguez\AppData\Local\Temp\ipykernel_19832\2140648366.py:97: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=sqlQuery, con=connection)


###
<font color='Wheat' size="5"> Facturas Servimed </font>

In [ ]:
# SQL command to read the data
sqlQuery = '''

-- FACTURAS SERVIMED

DECLARE @hoy DATETIME = GETDATE()
DECLARE @hasta DATETIME = EOMONTH(@hoy, -1);
DECLARE @desde DATETIME =  DATETRUNC(month,@hasta),
		@empresa INT = 10,
		@fila int = 1
/*
1	FAMISANAR SAS
2	NUEVA EPS SA
4	SERVICIO OCCIDENTAL DE SALUD S.O.S
5	SURA EPS
6	UT SERVISALUD SAN JOSE
7	UNISALUD
10	SERVIMED IPS SA
13	EPS COMFAMILIAR NARIÑO
16	DEPARTAMENTO DE CUNDINAMARCA
20	WOMENS LINK WORLDWIDE
25	FONDO DE POBLACIONES DE LAS NACIONES UNIDAS
29	COLEGIO COLOMBIANO DE PSICOLOGIA
43	I E D LLANO ORIENTAL
*/


SELECT
		[Codigo Prestador],
		[Razon Social],
		[Tipo Identificacion],
		[Identificacion],
		Factura,
		Fecha,
		FechaInicial,
		FechaDesde,
		CodigoResponsable,
		NombreResponsable,
		[# Contrato],
		[Plan Beneficios],
		[# Poliza],
		[Valor Copago],
		[Valor Descuento],
		sum(ValorFactura1) Valor
FROM (
SELECT  '110010893004' [Codigo Prestador],
		'FUNDACION ORIENTAME' [Razon Social],
		'NI' [Tipo Identificacion],
		'860049972' [Identificacion],
		DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		CAST(FechaFactura AS DATE) Fecha,
		CAST(@desde AS DATE) FechaInicial,
		CAST(@hasta AS DATE) FechaDesde,
		CASE
			WHEN M.CodigoResponsable = '900156264' THEN 'EPS037'
			WHEN M.CodigoResponsable = '830003564' THEN 'EPS017'
			WHEN M.CodigoResponsable = '800088702' THEN 'EPS018'
			WHEN M.CodigoResponsable = '830002272' THEN 'EPS019'
			WHEN M.CodigoResponsable = '901127521' THEN 'EPS020'
			ELSE 'SDS001' END CodigoResponsable,

		CASE
			WHEN M.CodigoResponsable = '900156264' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '830003564' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '800088702' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '830002272' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '901127521' THEN A.NombreResponsable
			ELSE 'FUNDACION ORIENTAME' END NombreResponsable,
		'' [# Contrato],
		'IVE' [Plan Beneficios],
		'' [# Poliza],
		'' [Valor Copago],
		'' [Valor Descuento],
		((B.ValorFactura * B.IndicadorFacturable ) - B.ValorIvaFactura ) * A.EstadoActivo ValorFactura1
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('CONSULTA')
AND A.EmpresaId = @empresa
and  NOT EXISTS (SELECT  *
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaId = A.FacturaId AND NOTAS.FechaFactura > @desde)
UNION ALL
select  '110010893004' [Codigo Prestador],
		'FUNDACION ORIENTAME' [Razon Social],
		'NI' [Tipo Identificacion],
		'860049972' [Identificacion],
		Factura,
		Fecha,
		CAST(@desde AS DATE) FechaInicial,
		CAST(@hasta AS DATE) FechaDesde,
				CASE

			WHEN CodigoResponsable = '900156264' THEN 'EPS037'
			WHEN CodigoResponsable = '830003564' THEN 'EPS017'
			WHEN CodigoResponsable = '800088702' THEN 'EPS018'
			WHEN CodigoResponsable = '830002272' THEN 'EPS019'
			WHEN CodigoResponsable = '901127521' THEN 'EPS020' ELSE 'SDS001' END CodigoResponsable, --	P6.Valor ,
				CASE

			WHEN CodigoResponsable = '900156264' THEN NombreResponsable
			WHEN CodigoResponsable = '830003564' THEN NombreResponsable
			WHEN CodigoResponsable = '800088702' THEN NombreResponsable
			WHEN CodigoResponsable = '830002272' THEN NombreResponsable
			WHEN CodigoResponsable = '901127521' THEN NombreResponsable ELSE 'FUNDACION ORIENTAME' END NombreResponsable	,
		'' [# Contrato],
		'IVE' [Plan Beneficios],
		'' [# Poliza],
		'' [Valor Copago],
		'' [Valor Descuento],
		ValorFactura1
from(
 SELECT	Row_number() over(partition by pac.Identificacion,ACT.ActividadId order by pac.Identificacion,A.Numero ASC) fila,
		DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		'110010893004' [Codigo Prestador],
		SUBSTRING(pac.TipoIdentificacion,1,2) [Tipo Documento],
		pac.Identificacion Documento,
		CAST(FechaFactura AS DATE) Fecha,
		'' [# Autorizaci�n],
		ACT.CodigoNacional [Codigo CUP],
		1 Ambito,
		2 Finalidad,
		'' Personal,
		'' [DX Principal],
		'' [DX Relacionado 1],
		2  [DX Complicaciones],
		1 [Realizacion],
		M.CodigoResponsable,
		A.NombreResponsable	,
		 ((B.ValorFactura*B.IndicadorFacturable)-B.ValorIvaFactura)*A.EstadoActivo ValorFactura1,
		 ACT.Nombre
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('PROCEDIMIENTOS')
AND M.EmpresaId = @empresa
and  NOT EXISTS (SELECT  NOTAS.FacturaId,NOtas.Numero,  B.FacturaAfectadaId
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					LEFT JOIN FAC_MOV_FacturasAnexos B WITH (NOLOCK) ON NOTAS.FacturaId = B.FacturaEncabezadoId
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaAfectadaId = A.FacturaId  AND NOTAS.FechaFactura> @desde )
				) tabla
				where fila =@fila
	)	TABLA
group by
		[Codigo Prestador],
		[Razon Social],
		[Tipo Identificacion],
		[Identificacion],
		Factura,
		Fecha,
		FechaInicial,
		FechaDesde,
		CodigoResponsable,
		NombreResponsable,
		[# Contrato],
		[Plan Beneficios],
		[# Poliza],
		[Valor Copago],
		[Valor Descuento]
'''

# Information converted into a DataFrame with pandas
df = pd.read_sql(sql=sqlQuery, con=connection)

# Export the data on the excel into a Desktop
df.to_csv(os.environ["userprofile"] + "\\Documents\\Rips\\Empresas\\Servimed\\" + "Servimed_Facturas " +
            datetime.now().strftime("%d-%b-%Y %H%M%S") + ".csv", index=False)

C:\Users\ddrodriguez\AppData\Local\Temp\ipykernel_19832\688390570.py:243: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=sqlQuery, con=connection)


###
<font color='Wheat' size="5"> Procedimientos Servimed </font>

In [ ]:
# SQL command to read the data
sqlQuery = '''

--PROCEDIMIENTOS SERVIMED

DECLARE @hoy DATETIME = GETDATE()
DECLARE @hasta DATETIME = EOMONTH(@hoy, -1);
DECLARE @desde DATETIME =  DATETRUNC(month,@hasta),
		@empresa INT = 10,
		@fila int = 1
/*
1	FAMISANAR SAS
2	NUEVA EPS SA
4	SERVICIO OCCIDENTAL DE SALUD S.O.S
5	SURA EPS
6	UT SERVISALUD SAN JOSE
7	UNISALUD
10	SERVIMED IPS SA
13	EPS COMFAMILIAR NARIÑO
16	DEPARTAMENTO DE CUNDINAMARCA
20	WOMENS LINK WORLDWIDE
25	FONDO DE POBLACIONES DE LAS NACIONES UNIDAS
29	COLEGIO COLOMBIANO DE PSICOLOGIA
43	I E D LLANO ORIENTAL
*/

select *
from(
SELECT	Row_number() over(partition by pac.Identificacion,ACT.ActividadId order by pac.Identificacion,FechaFactura) fila,
		DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		'110010893004' [Codigo Prestador],
		SUBSTRING(pac.TipoIdentificacion,1,2) [Tipo Documento],
		pac.Identificacion Documento,
		CAST(FechaFactura AS DATE) Fecha,
		C.NumeroAutorizacion [# Autorizaci�n],
		ACT.CodigoNacional [Codigo CUP],
		1 Ambito,
		2 Finalidad,
		'' Personal,
		'' [DX Principal],
		'' [DX Relacionado 1],
		2  [DX Complicaciones],
		1 [Realizacion],
		 ((B.ValorFactura*B.IndicadorFacturable)-B.ValorIvaFactura)*A.EstadoActivo ValorFactura1,
		 ACT.Nombre
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('PROCEDIMIENTOS')
AND M.EmpresaId = @empresa
and  NOT EXISTS (SELECT  NOTAS.FacturaId,NOtas.Numero,  B.FacturaAfectadaId
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					LEFT JOIN FAC_MOV_FacturasAnexos B WITH (NOLOCK) ON NOTAS.FacturaId = B.FacturaEncabezadoId
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaAfectadaId = A.FacturaId  AND NOTAS.FechaFactura> @desde )
				) tabla
				where fila =@fila
'''

# Information converted into a DataFrame with pandas
df = pd.read_sql(sql=sqlQuery, con=connection)

# Export the data on the excel into a Desktop
df.to_csv(os.environ["userprofile"] + "\\Documents\\Rips\\Empresas\\Servimed\\" + "Servimed_Procedimientos " +
            datetime.now().strftime("%d-%b-%Y %H%M%S") + ".csv", index=False)

C:\Users\ddrodriguez\AppData\Local\Temp\ipykernel_19832\3726592833.py:100: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=sqlQuery, con=connection)


###
<font color='Wheat' size="5"> Usuarios Servimed </font>

In [ ]:
# SQL command to read the data
sqlQuery = '''

--USUARIOS SERVIMED

DECLARE @hoy DATETIME = GETDATE()
DECLARE @hasta DATETIME = EOMONTH(@hoy, -1);
DECLARE @desde DATETIME =  DATETRUNC(month,@hasta),
		@empresa INT = 10,
		@fila int = 1
/*
1	FAMISANAR SAS
2	NUEVA EPS SA
4	SERVICIO OCCIDENTAL DE SALUD S.O.S
5	SURA EPS
6	UT SERVISALUD SAN JOSE
7	UNISALUD
13	EPS COMFAMILIAR NARIÑO
16	DEPARTAMENTO DE CUNDINAMARCA
20	WOMENS LINK WORLDWIDE
25	FONDO DE POBLACIONES DE LAS NACIONES UNIDAS
29	COLEGIO COLOMBIANO DE PSICOLOGIA
43	I E D LLANO ORIENTAL
*/

SELECT
	DISTINCT
		TABLA.[Tipo Documento],
		TABLA.Documento,
		CodigoResponsable,
		case when CodigoResponsable = 'SDS001' THEN 4 ELSE 1 END TipoUsuaurio,
		COALESCE(pac.PrimerApellido,'') PrimerApellido,
		COALESCE(pac.SegundoApellido,'') SegundoApellido,
		COALESCE(pac.PrimerNombre,'') PrimerNombre,
		COALESCE(pac.SegundoNombre,'') SegundoNombre,
		pac.FechaNacimiento,
		[dbo].[Fun_ORI_CalculaEdad](pac.FechaNacimiento,Fecha) Edad,
		1 Unidad,
		substring(Genero,1,1) Sexo,
		substring(ciudad.Referencia1,1,2) Depertamento,
		substring(ciudad.Referencia1,3,3) Ciudad,
		'U' Zona,
		case when pais.Valor = 'Colombia' or TABLA.[Tipo Documento] in ('CC','TI') then 'Colombia'

		else pais.Valor end Nacionalidad,
		case when pais.Valor = 'Colombia' or TABLA.[Tipo Documento] in ('CC','TI') then 'Nacional'

		else 'Extranjera' end TipoNacionalidad
FROM (
SELECT  '110010893004' [Codigo Prestador],
		'FUNDACION ORIENTAME' [Razon Social],
		'NI' [TipoNit],
		'860049972' Nit,
		SUBSTRING(pac.TipoIdentificacion,1,2) [Tipo Documento],
		pac.Identificacion Documento,
		DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		CAST(FechaFactura AS DATE) Fecha,
		CAST(@desde AS DATE) FechaInicial,
		CAST(@hasta AS DATE) FechaDesde,
		CASE
			WHEN M.CodigoResponsable = '900156264' THEN 'EPS037'
			WHEN M.CodigoResponsable = '830003564' THEN 'EPS017'
			WHEN M.CodigoResponsable = '800088702' THEN 'EPS018'
			WHEN M.CodigoResponsable = '830002272' THEN 'EPS019'
			WHEN M.CodigoResponsable = '901127521' THEN 'EPS020'
			ELSE 'SDS001' END CodigoResponsable,

		CASE
			WHEN M.CodigoResponsable = '900156264' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '830003564' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '800088702' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '830002272' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '901127521' THEN A.NombreResponsable
			ELSE 'FUNDACION ORIENTAME' END NombreResponsable,
		'' [# Contrato],
		'IVE' [Plan Beneficios],
		'' [# Poliza],
		'' [Valor Copago],
		'' [Valor Descuento],
		((B.ValorFactura * B.IndicadorFacturable ) - B.ValorIvaFactura ) * A.EstadoActivo ValorFactura1
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('CONSULTA')
AND A.EmpresaId = @empresa
and  NOT EXISTS (SELECT  *
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaId = A.FacturaId AND NOTAS.FechaFactura > @desde)
UNION ALL
select  '110010893004' [Codigo Prestador],
		'FUNDACION ORIENTAME' [Razon Social],
		'NI' [TipoNit],
		'860049972' Nit,
		[Tipo Documento],
		Documento,
		Factura,
		Fecha,
		CAST(@desde AS DATE) FechaInicial,
		CAST(@hasta AS DATE) FechaDesde,
				CASE

			WHEN CodigoResponsable = '900156264' THEN 'EPS037'
			WHEN CodigoResponsable = '830003564' THEN 'EPS017'
			WHEN CodigoResponsable = '800088702' THEN 'EPS018'
			WHEN CodigoResponsable = '830002272' THEN 'EPS019'
			WHEN CodigoResponsable = '901127521' THEN 'EPS020' ELSE 'SDS001' END CodigoResponsable, --	P6.Valor ,
				CASE

			WHEN CodigoResponsable = '900156264' THEN NombreResponsable
			WHEN CodigoResponsable = '830003564' THEN NombreResponsable
			WHEN CodigoResponsable = '800088702' THEN NombreResponsable
			WHEN CodigoResponsable = '830002272' THEN NombreResponsable
			WHEN CodigoResponsable = '901127521' THEN NombreResponsable ELSE 'FUNDACION ORIENTAME' END NombreResponsable	,
		'' [# Contrato],
		'IVE' [Plan Beneficios],
		'' [# Poliza],
		'' [Valor Copago],
		'' [Valor Descuento],
		ValorFactura1
from(
 SELECT	Row_number() over(partition by pac.Identificacion,ACT.ActividadId order by pac.Identificacion,A.Numero ASC) fila,
		DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		'110010893004' [Codigo Prestador],
		SUBSTRING(pac.TipoIdentificacion,1,2) [Tipo Documento],
		pac.Identificacion Documento,
		CAST(FechaFactura AS DATE) Fecha,
		'' [# Autorizaci�n],
		ACT.CodigoNacional [Codigo CUP],
		1 Ambito,
		2 Finalidad,
		'' Personal,
		'' [DX Principal],
		'' [DX Relacionado 1],
		2  [DX Complicaciones],
		1 [Realizacion],
			M.CodigoResponsable, --	P6.Valor ,
		A.NombreResponsable	,
		 ((B.ValorFactura*B.IndicadorFacturable)-B.ValorIvaFactura)*A.EstadoActivo ValorFactura1,
		 ACT.Nombre
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('PROCEDIMIENTOS')
AND M.EmpresaId = @empresa
and  NOT EXISTS (SELECT  NOTAS.FacturaId,NOtas.Numero,  B.FacturaAfectadaId
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					LEFT JOIN FAC_MOV_FacturasAnexos B WITH (NOLOCK) ON NOTAS.FacturaId = B.FacturaEncabezadoId
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaAfectadaId = A.FacturaId  AND NOTAS.FechaFactura> @desde )
				) tabla
				where fila =@fila
	)	TABLA
	INNER JOIN ADT_VIS_Pacientes pac with (nolock) ON pac.Identificacion = TABLA.Documento and SUBSTRING(pac.TipoIdentificacion,1,2) = TABLA.[Tipo Documento]
	left join GEN_CNF_Parametros ciudad on ciudad.ParametroId = MunicipioResidenciaId
	left join GEN_CNF_Parametros nacimiento on nacimiento.ParametroId = LugarNacimientoId
	left join GEN_CNF_Parametros departamento on departamento.ParametroId = nacimiento.PadreId
	left join GEN_CNF_Parametros pais on pais.ParametroId = departamento.PadreId
'''

# Information converted into a DataFrame with pandas
df = pd.read_sql(sql=sqlQuery, con=connection)

# Export the data on the excel into a Desktop
df.to_csv(os.environ["userprofile"] + "\\Documents\\Rips\\Empresas\\Servimed\\" + "Servimed_Usuarios " +
            datetime.now().strftime("%d-%b-%Y %H%M%S") + ".csv", index=False)

#Display notification to user
notification.notify(title="✅Report Status!!!",
                    message= f"Queries of Servimed has been succesfully saved into .csv\
                        \n Queries Executed:  \n 1. Consultas, 2.Facturas, 3.Procedimientos, 4.Usuarios",
                        timeout=10)

C:\Users\ddrodriguez\AppData\Local\Temp\ipykernel_19832\3413327178.py:240: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=sqlQuery, con=connection)


# **Queries of Sedes 🏥**

En esta sección se encuentran las consultas correspondientes a "SEDES" cada una de las consultas se guardan como un archivo .csv por aparte en la carpeta C:\Users\ddrodriguez\Documents\RIPS\Sedes. y separadas por ciudades

##
<font color='Gold' size="6">MEDELLIN</font>

###
<font color='Gold' size="5"> Consultas Medellin</font>

In [ ]:
# SQL command to read the data
sqlQuery = '''
--CONSULTAS MEDELLN

DECLARE @hoy DATETIME = GETDATE()
DECLARE @hasta DATETIME = EOMONTH(@hoy, -1);
DECLARE @desde DATETIME =  DATETRUNC(month,@hasta),
		@codigo_ciudad VARCHAR(5) = '05001'

/*
05001	MEDELLIN
08001	BARRANQUILLA
66170	DOSQUEBRADAS
54001	CUCUTA
52356	IPIALES
50001	VILLAVICENCIO
20001	VALLEDUPAR
*/

SELECT	DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		S.CodigoHabilitacion [Codigo Prestador],
		SUBSTRING(pac.TipoIdentificacion,1,2) [Tipo Documento],
		pac.Identificacion Documento,
		CAST(FechaFactura AS DATE) Fecha,
		C.NumeroAutorizacion [# Autorización],
		ACT.CodigoNacional [Codigo CUP],
		10 Finalidad,
		13 Causa,
		'' [DX Principal],
		'' [DX Relacionado 1],
		'' [DX Relacionado 2],
		'' [DX Relacionado 3],
		2  [Tipo DX],
		 ((B.ValorFactura*B.IndicadorFacturable)-B.ValorIvaFactura)*A.EstadoActivo ValorFactura1,
		'' ValorCopago,
		 ((B.ValorFactura*B.IndicadorFacturable)-B.ValorIvaFactura)*A.EstadoActivo ValorFactura,
		 ACT.Nombre
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('CONSULTA')
AND S.Codigo in (SELECT Codigo FROM GEN_CNF_Sedes WHERE CodigoTributarioCiudad = @codigo_ciudad)
and  NOT EXISTS (SELECT  *
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaId = A.FacturaId AND NOTAS.FechaFactura > @desde)
'''

# Information converted into a DataFrame with pandas
df = pd.read_sql(sql=sqlQuery, con=connection)

# Export the data on the excel into a Desktop
df.to_csv(os.environ["userprofile"] + "\\Documents\\Rips\\Sedes\\Medellin\\" + "Medellin_Consultas " +
            datetime.now().strftime("%d-%b-%Y %H%M%S") + ".csv", index=False)

C:\Users\ddrodriguez\AppData\Local\Temp\ipykernel_19832\3616990060.py:89: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=sqlQuery, con=connection)


###
<font color='Gold' size="5"> Facturas Medellin</font>

In [ ]:
# SQL command to read the data
sqlQuery = '''

--FACTURAS MEDELLIN

DECLARE @hoy DATETIME = GETDATE()
DECLARE @hasta DATETIME = EOMONTH(@hoy, -1);
DECLARE @desde DATETIME =  DATETRUNC(month,@hasta),
		@codigo_ciudad VARCHAR(5) = '05001'
/*
05001	MEDELLIN
08001	BARRANQUILLA
66170	DOSQUEBRADAS
54001	CUCUTA
52356	IPIALES
50001	VILLAVICENCIO
20001	VALLEDUPAR
*/


SELECT
		[Codigo Prestador],
		[Razon Social],
		[Tipo Identificacion],
		[Identificacion],
		Factura,
		Fecha,
		FechaInicial,
		FechaDesde,
		CodigoResponsable,
		NombreResponsable,
		[# Contrato],
		[Plan Beneficios],
		[# Poliza],
		[Valor Copago],
		[Valor Descuento],
		sum(ValorFactura1) Valor
FROM (
SELECT  S.CodigoHabilitacion [Codigo Prestador],
		'FUNDACION ORIENTAME' [Razon Social],
		'NI' [Tipo Identificacion],
		'860049972' [Identificacion],
		DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		CAST(FechaFactura AS DATE) Fecha,
		CAST(@desde AS DATE) FechaInicial,
		CAST(@hasta AS DATE) FechaDesde,
		CASE
			WHEN M.CodigoResponsable = '900156264' THEN 'EPS037'
			WHEN M.CodigoResponsable = '830003564' THEN 'EPS017'
			WHEN M.CodigoResponsable = '800088702' THEN 'EPS018'
			WHEN M.CodigoResponsable = '830002272' THEN 'EPS019'
			WHEN M.CodigoResponsable = '901127521' THEN 'EPS020'
			ELSE 'SDS001' END CodigoResponsable,

		CASE
			WHEN M.CodigoResponsable = '900156264' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '830003564' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '800088702' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '830002272' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '901127521' THEN A.NombreResponsable
			ELSE 'FUNDACION ORIENTAME' END NombreResponsable,
		'' [# Contrato],
		'IVE' [Plan Beneficios],
		'' [# Poliza],
		'' [Valor Copago],
		'' [Valor Descuento],
		((B.ValorFactura * B.IndicadorFacturable ) - B.ValorIvaFactura ) * A.EstadoActivo ValorFactura1
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('CONSULTA')
AND S.Codigo in (SELECT Codigo FROM GEN_CNF_Sedes WHERE CodigoTributarioCiudad = @codigo_ciudad)
and  NOT EXISTS (SELECT  *
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaId = A.FacturaId AND NOTAS.FechaFactura > @desde)
UNION ALL
select   [Codigo Prestador],
		'FUNDACION ORIENTAME' [Razon Social],
		'NI' [Tipo Identificacion],
		'860049972' [Identificacion],
		Factura,
		Fecha,
		CAST(@desde AS DATE) FechaInicial,
		CAST(@hasta AS DATE) FechaDesde,
				CASE

			WHEN CodigoResponsable = '900156264' THEN 'EPS037'
			WHEN CodigoResponsable = '830003564' THEN 'EPS017'
			WHEN CodigoResponsable = '800088702' THEN 'EPS018'
			WHEN CodigoResponsable = '830002272' THEN 'EPS019'
			WHEN CodigoResponsable = '901127521' THEN 'EPS020' ELSE 'SDS001' END CodigoResponsable, --	P6.Valor ,
				CASE

			WHEN CodigoResponsable = '900156264' THEN NombreResponsable
			WHEN CodigoResponsable = '830003564' THEN NombreResponsable
			WHEN CodigoResponsable = '800088702' THEN NombreResponsable
			WHEN CodigoResponsable = '830002272' THEN NombreResponsable
			WHEN CodigoResponsable = '901127521' THEN NombreResponsable ELSE 'FUNDACION ORIENTAME' END NombreResponsable	,
		'' [# Contrato],
		'IVE' [Plan Beneficios],
		'' [# Poliza],
		'' [Valor Copago],
		'' [Valor Descuento],
		ValorFactura1
from(
 SELECT	Row_number() over(partition by pac.Identificacion,ACT.ActividadId order by pac.Identificacion,A.Numero ASC) fila,
		DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		S.CodigoHabilitacion [Codigo Prestador],
		SUBSTRING(pac.TipoIdentificacion,1,2) [Tipo Documento],
		pac.Identificacion Documento,
		CAST(FechaFactura AS DATE) Fecha,
		'' [# Autorización],
		ACT.CodigoNacional [Codigo CUP],
		1 Ambito,
		2 Finalidad,
		'' Personal,
		'' [DX Principal],
		'' [DX Relacionado 1],
		2  [DX Complicaciones],
		1 [Realizacion],
		M.CodigoResponsable,
		A.NombreResponsable	,
		 ((B.ValorFactura*B.IndicadorFacturable)-B.ValorIvaFactura)*A.EstadoActivo ValorFactura1,
		 ACT.Nombre
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('PROCEDIMIENTOS')
AND S.Codigo in (SELECT Codigo FROM GEN_CNF_Sedes WHERE CodigoTributarioCiudad = @codigo_ciudad)
and  NOT EXISTS (SELECT  NOTAS.FacturaId,NOtas.Numero,  B.FacturaAfectadaId
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					LEFT JOIN FAC_MOV_FacturasAnexos B WITH (NOLOCK) ON NOTAS.FacturaId = B.FacturaEncabezadoId
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaAfectadaId = A.FacturaId  AND NOTAS.FechaFactura> @desde )
				) tabla
				where fila in (1,2)
	)	TABLA
group by
		[Codigo Prestador],
		[Razon Social],
		[Tipo Identificacion],
		[Identificacion],
		Factura,
		Fecha,
		FechaInicial,
		FechaDesde,
		CodigoResponsable,
		NombreResponsable,
		[# Contrato],
		[Plan Beneficios],
		[# Poliza],
		[Valor Copago],
		[Valor Descuento]
'''

# Information converted into a DataFrame with pandas
df = pd.read_sql(sql=sqlQuery, con=connection)

# Export the data on the excel into a Desktop
df.to_csv(os.environ["userprofile"] + "\\Documents\\Rips\\Sedes\\Medellin\\" + "Medellin_Facturas " +
            datetime.now().strftime("%d-%b-%Y %H%M%S") + ".csv", index=False)

C:\Users\ddrodriguez\AppData\Local\Temp\ipykernel_19832\798372824.py:236: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=sqlQuery, con=connection)


###
<font color='Gold' size="5"> Procedimientos Medellin</font>

In [ ]:
# SQL command to read the data
sqlQuery = '''

--PROCEDIMIENTOS MEDELLIN

DECLARE @hoy DATETIME = GETDATE()
DECLARE @hasta DATETIME = EOMONTH(@hoy, -1);
DECLARE @desde DATETIME =  DATETRUNC(month,@hasta),
		@codigo_ciudad VARCHAR(5) = '05001'
/*
05001	MEDELLIN
08001	BARRANQUILLA
66170	DOSQUEBRADAS
54001	CUCUTA
52356	IPIALES
50001	VILLAVICENCIO
20001	VALLEDUPAR
*/


select *
from(
SELECT	Row_number() over(partition by pac.Identificacion,ACT.ActividadId order by pac.Identificacion,FechaFactura) fila,
		DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		 S.CodigoHabilitacion [Codigo Prestador],
		SUBSTRING(pac.TipoIdentificacion,1,2) [Tipo Documento],
		pac.Identificacion Documento,
		CAST(FechaFactura AS DATE) Fecha,
		C.NumeroAutorizacion [# Autorizaci�n],
		ACT.CodigoNacional [Codigo CUP],
		1 Ambito,
		2 Finalidad,
		'' Personal,
		'' [DX Principal],
		'' [DX Relacionado 1],
		2  [DX Complicaciones],
		1 [Realizacion],
		 ((B.ValorFactura*B.IndicadorFacturable)-B.ValorIvaFactura)*A.EstadoActivo ValorFactura1,
		 ACT.Nombre
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('PROCEDIMIENTOS')
AND S.Codigo in (SELECT Codigo FROM GEN_CNF_Sedes WHERE CodigoTributarioCiudad = @codigo_ciudad)
and  NOT EXISTS (SELECT  NOTAS.FacturaId,NOtas.Numero,  B.FacturaAfectadaId
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					LEFT JOIN FAC_MOV_FacturasAnexos B WITH (NOLOCK) ON NOTAS.FacturaId = B.FacturaEncabezadoId
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaAfectadaId = A.FacturaId  AND NOTAS.FechaFactura> @desde )
				) tabla
				where fila IN (1,2)
'''

# Information converted into a DataFrame with pandas
df = pd.read_sql(sql=sqlQuery, con=connection)

# Export the data on the excel into a Desktop
df.to_csv(os.environ["userprofile"] + "\\Documents\\Rips\\Sedes\\Medellin\\" + "Medellin_Procedimientos " +
            datetime.now().strftime("%d-%b-%Y %H%M%S") + ".csv", index=False)

C:\Users\ddrodriguez\AppData\Local\Temp\ipykernel_19832\955150917.py:94: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=sqlQuery, con=connection)


###
<font color='Gold' size="5"> Usuarios Medellin</font>

In [ ]:
# SQL command to read the data
sqlQuery = '''

--USUARIOS MEDELLIN

DECLARE @hoy DATETIME = GETDATE()
DECLARE @hasta DATETIME = EOMONTH(@hoy, -1);
DECLARE @desde DATETIME =  DATETRUNC(month,@hasta),
		@codigo_ciudad VARCHAR(5) = '05001'
/*
05001	MEDELLIN
08001	BARRANQUILLA
66170	DOSQUEBRADAS
54001	CUCUTA
52356	IPIALES
50001	VILLAVICENCIO
20001	VALLEDUPAR
*/


SELECT
	DISTINCT
		TABLA.[Tipo Documento],
		TABLA.Documento,
		CodigoResponsable,
		case when CodigoResponsable = 'SDS001' THEN 4 ELSE 1 END TipoUsuaurio,
		COALESCE(pac.PrimerApellido,'') PrimerApellido,
		COALESCE(pac.SegundoApellido,'') SegundoApellido,
		COALESCE(pac.PrimerNombre,'') PrimerNombre,
		COALESCE(pac.SegundoNombre,'') SegundoNombre,
		pac.FechaNacimiento,
		[dbo].[Fun_ORI_CalculaEdad](pac.FechaNacimiento,Fecha) Edad,
		1 Unidad,
		substring(Genero,1,1) Sexo,
		substring(ciudad.Referencia1,1,2) Depertamento,
		substring(ciudad.Referencia1,3,3) Ciudad,
		'U' Zona,
		case when pais.Valor = 'Colombia' or TABLA.[Tipo Documento] in ('CC','TI') then 'Colombia'

		else pais.Valor end Nacionalidad,
		case when pais.Valor = 'Colombia' or TABLA.[Tipo Documento] in ('CC','TI') then 'Nacional'

		else 'Extranjera' end TipoNacionalidad
FROM (
SELECT   S.CodigoHabilitacion [Codigo Prestador],
		'FUNDACION ORIENTAME' [Razon Social],
		'NI' [TipoNit],
		'860049972' Nit,
		SUBSTRING(pac.TipoIdentificacion,1,2) [Tipo Documento],
		pac.Identificacion Documento,
		DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		CAST(FechaFactura AS DATE) Fecha,
		CAST(@desde AS DATE) FechaInicial,
		CAST(@hasta AS DATE) FechaDesde,
		CASE
			WHEN M.CodigoResponsable = '900156264' THEN 'EPS037'
			WHEN M.CodigoResponsable = '830003564' THEN 'EPS017'
			WHEN M.CodigoResponsable = '800088702' THEN 'EPS018'
			WHEN M.CodigoResponsable = '830002272' THEN 'EPS019'
			WHEN M.CodigoResponsable = '901127521' THEN 'EPS020'
			ELSE 'SDS001' END CodigoResponsable,

		CASE
			WHEN M.CodigoResponsable = '900156264' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '830003564' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '800088702' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '830002272' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '901127521' THEN A.NombreResponsable
			ELSE 'FUNDACION ORIENTAME' END NombreResponsable,
		'' [# Contrato],
		'IVE' [Plan Beneficios],
		'' [# Poliza],
		'' [Valor Copago],
		'' [Valor Descuento],
		((B.ValorFactura * B.IndicadorFacturable ) - B.ValorIvaFactura ) * A.EstadoActivo ValorFactura1
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('CONSULTA')
AND S.Codigo in (SELECT Codigo FROM GEN_CNF_Sedes WHERE CodigoTributarioCiudad = @codigo_ciudad)
and  NOT EXISTS (SELECT  *
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaId = A.FacturaId AND NOTAS.FechaFactura > @desde)
UNION ALL
select  [Codigo Prestador],
		'FUNDACION ORIENTAME' [Razon Social],
		'NI' [TipoNit],
		'860049972' Nit,
		[Tipo Documento],
		Documento,
		Factura,
		Fecha,
		CAST(@desde AS DATE) FechaInicial,
		CAST(@hasta AS DATE) FechaDesde,
				CASE

			WHEN CodigoResponsable = '900156264' THEN 'EPS037'
			WHEN CodigoResponsable = '830003564' THEN 'EPS017'
			WHEN CodigoResponsable = '800088702' THEN 'EPS018'
			WHEN CodigoResponsable = '830002272' THEN 'EPS019'
			WHEN CodigoResponsable = '901127521' THEN 'EPS020' ELSE 'SDS001' END CodigoResponsable, --	P6.Valor ,
				CASE

			WHEN CodigoResponsable = '900156264' THEN NombreResponsable
			WHEN CodigoResponsable = '830003564' THEN NombreResponsable
			WHEN CodigoResponsable = '800088702' THEN NombreResponsable
			WHEN CodigoResponsable = '830002272' THEN NombreResponsable
			WHEN CodigoResponsable = '901127521' THEN NombreResponsable ELSE 'FUNDACION ORIENTAME' END NombreResponsable	,
		'' [# Contrato],
		'IVE' [Plan Beneficios],
		'' [# Poliza],
		'' [Valor Copago],
		'' [Valor Descuento],
		ValorFactura1
from(
 SELECT	Row_number() over(partition by pac.Identificacion,ACT.ActividadId order by pac.Identificacion,A.Numero ASC) fila,
		DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		 S.CodigoHabilitacion [Codigo Prestador],
		SUBSTRING(pac.TipoIdentificacion,1,2) [Tipo Documento],
		pac.Identificacion Documento,
		CAST(FechaFactura AS DATE) Fecha,
		'' [# Autorizaci�n],
		ACT.CodigoNacional [Codigo CUP],
		1 Ambito,
		2 Finalidad,
		'' Personal,
		'' [DX Principal],
		'' [DX Relacionado 1],
		2  [DX Complicaciones],
		1 [Realizacion],
			M.CodigoResponsable, --	P6.Valor ,
		A.NombreResponsable	,
		 ((B.ValorFactura*B.IndicadorFacturable)-B.ValorIvaFactura)*A.EstadoActivo ValorFactura1,
		 ACT.Nombre
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('PROCEDIMIENTOS')
AND S.Codigo in (SELECT Codigo FROM GEN_CNF_Sedes WHERE CodigoTributarioCiudad = @codigo_ciudad)
and  NOT EXISTS (SELECT  NOTAS.FacturaId,NOtas.Numero,  B.FacturaAfectadaId
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					LEFT JOIN FAC_MOV_FacturasAnexos B WITH (NOLOCK) ON NOTAS.FacturaId = B.FacturaEncabezadoId
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaAfectadaId = A.FacturaId  AND NOTAS.FechaFactura> @desde )
				) tabla
					where fila in (1,2)
	)	TABLA
	INNER JOIN ADT_VIS_Pacientes pac with (nolock) ON pac.Identificacion = TABLA.Documento and SUBSTRING(pac.TipoIdentificacion,1,2) = TABLA.[Tipo Documento]
	left join GEN_CNF_Parametros ciudad on ciudad.ParametroId = MunicipioResidenciaId
	left join GEN_CNF_Parametros nacimiento on nacimiento.ParametroId = LugarNacimientoId
	left join GEN_CNF_Parametros departamento on departamento.ParametroId = nacimiento.PadreId
	left join GEN_CNF_Parametros pais on pais.ParametroId = departamento.PadreId
'''

# Information converted into a DataFrame with pandas
df = pd.read_sql(sql=sqlQuery, con=connection)

# Export the data on the excel into a Desktop
df.to_csv(os.environ["userprofile"] + "\\Documents\\Rips\\Sedes\\Medellin\\" + "Medellin_Usuarios " +
            datetime.now().strftime("%d-%b-%Y %H%M%S") + ".csv", index=False)

#Display notification to user
notification.notify(title="✅Report Status!!!",
                    message= f"Queries of Medellin has been succesfully saved into .csv\
                        \n Queries Executed:  \n 1. Consultas, 2.Facturas, 3.Procedimientos, 4.Usuarios",
                        timeout=10)

C:\Users\ddrodriguez\AppData\Local\Temp\ipykernel_19832\26927035.py:235: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=sqlQuery, con=connection)


##
<font color='Tomato' size="6"> BARRANQUILLA </font>

###
<font color='Tomato' size="5"> Consultas Barranquilla </font>

In [ ]:
# SQL command to read the data
sqlQuery = '''
--CONSULTAS BARRANQUILLA

DECLARE @hoy DATETIME = GETDATE()
DECLARE @hasta DATETIME = EOMONTH(@hoy, -1);
DECLARE @desde DATETIME =  DATETRUNC(month,@hasta),
		@codigo_ciudad VARCHAR(5) = '08001'
/*
05001	MEDELLIN
08001	BARRANQUILLA
66170	DOSQUEBRADAS
54001	CUCUTA
52356	IPIALES
50001	VILLAVICENCIO
20001	VALLEDUPAR
*/

SELECT	DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		S.CodigoHabilitacion [Codigo Prestador],
		SUBSTRING(pac.TipoIdentificacion,1,2) [Tipo Documento],
		pac.Identificacion Documento,
		CAST(FechaFactura AS DATE) Fecha,
		C.NumeroAutorizacion [# Autorización],
		ACT.CodigoNacional [Codigo CUP],
		10 Finalidad,
		13 Causa,
		'' [DX Principal],
		'' [DX Relacionado 1],
		'' [DX Relacionado 2],
		'' [DX Relacionado 3],
		2  [Tipo DX],
		 ((B.ValorFactura*B.IndicadorFacturable)-B.ValorIvaFactura)*A.EstadoActivo ValorFactura1,
		'' ValorCopago,
		 ((B.ValorFactura*B.IndicadorFacturable)-B.ValorIvaFactura)*A.EstadoActivo ValorFactura,
		 ACT.Nombre
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('CONSULTA')
AND S.Codigo in (SELECT Codigo FROM GEN_CNF_Sedes WHERE CodigoTributarioCiudad = @codigo_ciudad)
and  NOT EXISTS (SELECT  *
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaId = A.FacturaId AND NOTAS.FechaFactura > @desde)

'''

# Information converted into a DataFrame with pandas
df = pd.read_sql(sql=sqlQuery, con=connection)

# Export the data on the excel into a Desktop
df.to_csv(os.environ["userprofile"] + "\\Documents\\Rips\\Sedes\\Barranquilla\\" + "Barranquilla_Consultas " +
            datetime.now().strftime("%d-%b-%Y %H%M%S") + ".csv", index=False)

C:\Users\ddrodriguez\AppData\Local\Temp\ipykernel_19832\3357245222.py:89: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=sqlQuery, con=connection)


###
<font color='Tomato' size="5"> Facturas Barranquilla </font>

In [ ]:
# SQL command to read the data
sqlQuery = '''
--FACTURAS BARRANQUILLA

DECLARE @hoy DATETIME = GETDATE()
DECLARE @hasta DATETIME = EOMONTH(@hoy, -1);
DECLARE @desde DATETIME =  DATETRUNC(month,@hasta),
		@codigo_ciudad VARCHAR(5) = '08001'
/*
05001	MEDELLIN
08001	BARRANQUILLA
66170	DOSQUEBRADAS
54001	CUCUTA
52356	IPIALES
50001	VILLAVICENCIO
20001	VALLEDUPAR
*/


SELECT
		[Codigo Prestador],
		[Razon Social],
		[Tipo Identificacion],
		[Identificacion],
		Factura,
		Fecha,
		FechaInicial,
		FechaDesde,
		CodigoResponsable,
		NombreResponsable,
		[# Contrato],
		[Plan Beneficios],
		[# Poliza],
		[Valor Copago],
		[Valor Descuento],
		sum(ValorFactura1) Valor
FROM (
SELECT  S.CodigoHabilitacion [Codigo Prestador],
		'FUNDACION ORIENTAME' [Razon Social],
		'NI' [Tipo Identificacion],
		'860049972' [Identificacion],
		DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		CAST(FechaFactura AS DATE) Fecha,
		CAST(@desde AS DATE) FechaInicial,
		CAST(@hasta AS DATE) FechaDesde,
		CASE
			WHEN M.CodigoResponsable = '900156264' THEN 'EPS037'
			WHEN M.CodigoResponsable = '830003564' THEN 'EPS017'
			WHEN M.CodigoResponsable = '800088702' THEN 'EPS018'
			WHEN M.CodigoResponsable = '830002272' THEN 'EPS019'
			WHEN M.CodigoResponsable = '901127521' THEN 'EPS020'
			ELSE 'SDS001' END CodigoResponsable,

		CASE
			WHEN M.CodigoResponsable = '900156264' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '830003564' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '800088702' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '830002272' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '901127521' THEN A.NombreResponsable
			ELSE 'FUNDACION ORIENTAME' END NombreResponsable,
		'' [# Contrato],
		'IVE' [Plan Beneficios],
		'' [# Poliza],
		'' [Valor Copago],
		'' [Valor Descuento],
		((B.ValorFactura * B.IndicadorFacturable ) - B.ValorIvaFactura ) * A.EstadoActivo ValorFactura1
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('CONSULTA')
AND S.Codigo in (SELECT Codigo FROM GEN_CNF_Sedes WHERE CodigoTributarioCiudad = @codigo_ciudad)
and  NOT EXISTS (SELECT  *
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaId = A.FacturaId AND NOTAS.FechaFactura > @desde)
UNION ALL
select   [Codigo Prestador],
		'FUNDACION ORIENTAME' [Razon Social],
		'NI' [Tipo Identificacion],
		'860049972' [Identificacion],
		Factura,
		Fecha,
		CAST(@desde AS DATE) FechaInicial,
		CAST(@hasta AS DATE) FechaDesde,
				CASE

			WHEN CodigoResponsable = '900156264' THEN 'EPS037'
			WHEN CodigoResponsable = '830003564' THEN 'EPS017'
			WHEN CodigoResponsable = '800088702' THEN 'EPS018'
			WHEN CodigoResponsable = '830002272' THEN 'EPS019'
			WHEN CodigoResponsable = '901127521' THEN 'EPS020' ELSE 'SDS001' END CodigoResponsable, --	P6.Valor ,
				CASE

			WHEN CodigoResponsable = '900156264' THEN NombreResponsable
			WHEN CodigoResponsable = '830003564' THEN NombreResponsable
			WHEN CodigoResponsable = '800088702' THEN NombreResponsable
			WHEN CodigoResponsable = '830002272' THEN NombreResponsable
			WHEN CodigoResponsable = '901127521' THEN NombreResponsable ELSE 'FUNDACION ORIENTAME' END NombreResponsable	,
		'' [# Contrato],
		'IVE' [Plan Beneficios],
		'' [# Poliza],
		'' [Valor Copago],
		'' [Valor Descuento],
		ValorFactura1
from(
 SELECT	Row_number() over(partition by pac.Identificacion,ACT.ActividadId order by pac.Identificacion,A.Numero ASC) fila,
		DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		S.CodigoHabilitacion [Codigo Prestador],
		SUBSTRING(pac.TipoIdentificacion,1,2) [Tipo Documento],
		pac.Identificacion Documento,
		CAST(FechaFactura AS DATE) Fecha,
		'' [# Autorización],
		ACT.CodigoNacional [Codigo CUP],
		1 Ambito,
		2 Finalidad,
		'' Personal,
		'' [DX Principal],
		'' [DX Relacionado 1],
		2  [DX Complicaciones],
		1 [Realizacion],
		M.CodigoResponsable,
		A.NombreResponsable	,
		 ((B.ValorFactura*B.IndicadorFacturable)-B.ValorIvaFactura)*A.EstadoActivo ValorFactura1,
		 ACT.Nombre
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('PROCEDIMIENTOS')
AND S.Codigo in (SELECT Codigo FROM GEN_CNF_Sedes WHERE CodigoTributarioCiudad = @codigo_ciudad)
and  NOT EXISTS (SELECT  NOTAS.FacturaId,NOtas.Numero,  B.FacturaAfectadaId
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					LEFT JOIN FAC_MOV_FacturasAnexos B WITH (NOLOCK) ON NOTAS.FacturaId = B.FacturaEncabezadoId
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaAfectadaId = A.FacturaId  AND NOTAS.FechaFactura> @desde )
				) tabla
				where fila in (1,2)
	)	TABLA
group by
		[Codigo Prestador],
		[Razon Social],
		[Tipo Identificacion],
		[Identificacion],
		Factura,
		Fecha,
		FechaInicial,
		FechaDesde,
		CodigoResponsable,
		NombreResponsable,
		[# Contrato],
		[Plan Beneficios],
		[# Poliza],
		[Valor Copago],
		[Valor Descuento]
'''

# Information converted into a DataFrame with pandas
df = pd.read_sql(sql=sqlQuery, con=connection)

# Export the data on the excel into a Desktop
df.to_csv(os.environ["userprofile"] + "\\Documents\\Rips\\Sedes\\Barranquilla\\" + "Barranquilla_Facturas " +
            datetime.now().strftime("%d-%b-%Y %H%M%S") + ".csv", index=False)

C:\Users\ddrodriguez\AppData\Local\Temp\ipykernel_19832\777631719.py:235: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=sqlQuery, con=connection)


###
<font color='Tomato' size="5"> Procedimientos Barranquilla </font>

In [ ]:
# SQL command to read the data
sqlQuery = '''

--PROCEDIMIENTOS

DECLARE @hoy DATETIME = GETDATE()
DECLARE @hasta DATETIME = EOMONTH(@hoy, -1);
DECLARE @desde DATETIME =  DATETRUNC(month,@hasta),
		@codigo_ciudad VARCHAR(5) = '08001'
/*
05001	MEDELLIN
08001	BARRANQUILLA
66170	DOSQUEBRADAS
54001	CUCUTA
52356	IPIALES
50001	VILLAVICENCIO
20001	VALLEDUPAR
*/


select *
from(
SELECT	Row_number() over(partition by pac.Identificacion,ACT.ActividadId order by pac.Identificacion,FechaFactura) fila,
		DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		 S.CodigoHabilitacion [Codigo Prestador],
		SUBSTRING(pac.TipoIdentificacion,1,2) [Tipo Documento],
		pac.Identificacion Documento,
		CAST(FechaFactura AS DATE) Fecha,
		C.NumeroAutorizacion [# Autorizaci�n],
		ACT.CodigoNacional [Codigo CUP],
		1 Ambito,
		2 Finalidad,
		'' Personal,
		'' [DX Principal],
		'' [DX Relacionado 1],
		2  [DX Complicaciones],
		1 [Realizacion],
		 ((B.ValorFactura*B.IndicadorFacturable)-B.ValorIvaFactura)*A.EstadoActivo ValorFactura1,
		 ACT.Nombre
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('PROCEDIMIENTOS')
AND S.Codigo in (SELECT Codigo FROM GEN_CNF_Sedes WHERE CodigoTributarioCiudad = @codigo_ciudad)
and  NOT EXISTS (SELECT  NOTAS.FacturaId,NOtas.Numero,  B.FacturaAfectadaId
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					LEFT JOIN FAC_MOV_FacturasAnexos B WITH (NOLOCK) ON NOTAS.FacturaId = B.FacturaEncabezadoId
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaAfectadaId = A.FacturaId  AND NOTAS.FechaFactura> @desde )
				) tabla
				where fila IN (1,2)
'''

# Information converted into a DataFrame with pandas
df = pd.read_sql(sql=sqlQuery, con=connection)

# Export the data on the excel into a Desktop
df.to_csv(os.environ["userprofile"] + "\\Documents\\Rips\\Sedes\\Barranquilla\\" + "Barranquilla_Procedimientos " +
            datetime.now().strftime("%d-%b-%Y %H%M%S") + ".csv", index=False)

C:\Users\ddrodriguez\AppData\Local\Temp\ipykernel_19832\3749085056.py:94: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=sqlQuery, con=connection)


###
<font color='Tomato' size="5"> Usuarios Barranquilla </font>

In [ ]:
# SQL command to read the data
sqlQuery = '''

--USUARIOS BARRANQUILLA

DECLARE @hoy DATETIME = GETDATE()
DECLARE @hasta DATETIME = EOMONTH(@hoy, -1);
DECLARE @desde DATETIME =  DATETRUNC(month,@hasta),
		@codigo_ciudad VARCHAR(5) = '08001'
/*
05001	MEDELLIN
08001	BARRANQUILLA
66170	DOSQUEBRADAS
54001	CUCUTA
52356	IPIALES
50001	VILLAVICENCIO
20001	VALLEDUPAR
*/


SELECT
	DISTINCT
		TABLA.[Tipo Documento],
		TABLA.Documento,
		CodigoResponsable,
		case when CodigoResponsable = 'SDS001' THEN 4 ELSE 1 END TipoUsuaurio,
		COALESCE(pac.PrimerApellido,'') PrimerApellido,
		COALESCE(pac.SegundoApellido,'') SegundoApellido,
		COALESCE(pac.PrimerNombre,'') PrimerNombre,
		COALESCE(pac.SegundoNombre,'') SegundoNombre,
		pac.FechaNacimiento,
		[dbo].[Fun_ORI_CalculaEdad](pac.FechaNacimiento,Fecha) Edad,
		1 Unidad,
		substring(Genero,1,1) Sexo,
		substring(ciudad.Referencia1,1,2) Depertamento,
		substring(ciudad.Referencia1,3,3) Ciudad,
		'U' Zona,
		case when pais.Valor = 'Colombia' or TABLA.[Tipo Documento] in ('CC','TI') then 'Colombia'

		else pais.Valor end Nacionalidad,
		case when pais.Valor = 'Colombia' or TABLA.[Tipo Documento] in ('CC','TI') then 'Nacional'

		else 'Extranjera' end TipoNacionalidad
FROM (
SELECT   S.CodigoHabilitacion [Codigo Prestador],
		'FUNDACION ORIENTAME' [Razon Social],
		'NI' [TipoNit],
		'860049972' Nit,
		SUBSTRING(pac.TipoIdentificacion,1,2) [Tipo Documento],
		pac.Identificacion Documento,
		DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		CAST(FechaFactura AS DATE) Fecha,
		CAST(@desde AS DATE) FechaInicial,
		CAST(@hasta AS DATE) FechaDesde,
		CASE
			WHEN M.CodigoResponsable = '900156264' THEN 'EPS037'
			WHEN M.CodigoResponsable = '830003564' THEN 'EPS017'
			WHEN M.CodigoResponsable = '800088702' THEN 'EPS018'
			WHEN M.CodigoResponsable = '830002272' THEN 'EPS019'
			WHEN M.CodigoResponsable = '901127521' THEN 'EPS020'
			ELSE 'SDS001' END CodigoResponsable,

		CASE
			WHEN M.CodigoResponsable = '900156264' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '830003564' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '800088702' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '830002272' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '901127521' THEN A.NombreResponsable
			ELSE 'FUNDACION ORIENTAME' END NombreResponsable,
		'' [# Contrato],
		'IVE' [Plan Beneficios],
		'' [# Poliza],
		'' [Valor Copago],
		'' [Valor Descuento],
		((B.ValorFactura * B.IndicadorFacturable ) - B.ValorIvaFactura ) * A.EstadoActivo ValorFactura1
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('CONSULTA')
AND S.Codigo in (SELECT Codigo FROM GEN_CNF_Sedes WHERE CodigoTributarioCiudad = @codigo_ciudad)
and  NOT EXISTS (SELECT  *
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaId = A.FacturaId AND NOTAS.FechaFactura > @desde)
UNION ALL
select  [Codigo Prestador],
		'FUNDACION ORIENTAME' [Razon Social],
		'NI' [TipoNit],
		'860049972' Nit,
		[Tipo Documento],
		Documento,
		Factura,
		Fecha,
		CAST(@desde AS DATE) FechaInicial,
		CAST(@hasta AS DATE) FechaDesde,
				CASE

			WHEN CodigoResponsable = '900156264' THEN 'EPS037'
			WHEN CodigoResponsable = '830003564' THEN 'EPS017'
			WHEN CodigoResponsable = '800088702' THEN 'EPS018'
			WHEN CodigoResponsable = '830002272' THEN 'EPS019'
			WHEN CodigoResponsable = '901127521' THEN 'EPS020' ELSE 'SDS001' END CodigoResponsable, --	P6.Valor ,
				CASE

			WHEN CodigoResponsable = '900156264' THEN NombreResponsable
			WHEN CodigoResponsable = '830003564' THEN NombreResponsable
			WHEN CodigoResponsable = '800088702' THEN NombreResponsable
			WHEN CodigoResponsable = '830002272' THEN NombreResponsable
			WHEN CodigoResponsable = '901127521' THEN NombreResponsable ELSE 'FUNDACION ORIENTAME' END NombreResponsable	,
		'' [# Contrato],
		'IVE' [Plan Beneficios],
		'' [# Poliza],
		'' [Valor Copago],
		'' [Valor Descuento],
		ValorFactura1
from(
 SELECT	Row_number() over(partition by pac.Identificacion,ACT.ActividadId order by pac.Identificacion,A.Numero ASC) fila,
		DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		 S.CodigoHabilitacion [Codigo Prestador],
		SUBSTRING(pac.TipoIdentificacion,1,2) [Tipo Documento],
		pac.Identificacion Documento,
		CAST(FechaFactura AS DATE) Fecha,
		'' [# Autorizaci�n],
		ACT.CodigoNacional [Codigo CUP],
		1 Ambito,
		2 Finalidad,
		'' Personal,
		'' [DX Principal],
		'' [DX Relacionado 1],
		2  [DX Complicaciones],
		1 [Realizacion],
			M.CodigoResponsable, --	P6.Valor ,
		A.NombreResponsable	,
		 ((B.ValorFactura*B.IndicadorFacturable)-B.ValorIvaFactura)*A.EstadoActivo ValorFactura1,
		 ACT.Nombre
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('PROCEDIMIENTOS')
AND S.Codigo in (SELECT Codigo FROM GEN_CNF_Sedes WHERE CodigoTributarioCiudad = @codigo_ciudad)
and  NOT EXISTS (SELECT  NOTAS.FacturaId,NOtas.Numero,  B.FacturaAfectadaId
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					LEFT JOIN FAC_MOV_FacturasAnexos B WITH (NOLOCK) ON NOTAS.FacturaId = B.FacturaEncabezadoId
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaAfectadaId = A.FacturaId  AND NOTAS.FechaFactura> @desde )
				) tabla
					where fila in (1,2)
	)	TABLA
	INNER JOIN ADT_VIS_Pacientes pac with (nolock) ON pac.Identificacion = TABLA.Documento and SUBSTRING(pac.TipoIdentificacion,1,2) = TABLA.[Tipo Documento]
	left join GEN_CNF_Parametros ciudad on ciudad.ParametroId = MunicipioResidenciaId
	left join GEN_CNF_Parametros nacimiento on nacimiento.ParametroId = LugarNacimientoId
	left join GEN_CNF_Parametros departamento on departamento.ParametroId = nacimiento.PadreId
	left join GEN_CNF_Parametros pais on pais.ParametroId = departamento.PadreId
'''

# Information converted into a DataFrame with pandas
df = pd.read_sql(sql=sqlQuery, con=connection)

# Export the data on the excel into a Desktop
df.to_csv(os.environ["userprofile"] + "\\Documents\\Rips\\Sedes\\Barranquilla\\" + "Barranquilla_Usuarios " +
            datetime.now().strftime("%d-%b-%Y %H%M%S") + ".csv", index=False)

#Display notification to user
notification.notify(title="✅Report Status!!!",
                    message= f"Queries of Barranquilla has been succesfully saved into .csv\
                        \n Queries Executed:  \n 1. Consultas, 2.Facturas, 3.Procedimientos, 4.Usuarios",
                        timeout=10)

C:\Users\ddrodriguez\AppData\Local\Temp\ipykernel_19832\2496996649.py:235: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=sqlQuery, con=connection)


##
<font color='SpringGreen' size="6"> DOSQUEBRADAS </font>

###
<font color='SpringGreen' size="5"> Consultas Dosquebradas </font>

In [ ]:
# SQL command to read the data
sqlQuery = '''
--CONSULTAS DOSQUEDRADAS

DECLARE @hoy DATETIME = GETDATE()
DECLARE @hasta DATETIME = EOMONTH(@hoy, -1);
DECLARE @desde DATETIME =  DATETRUNC(month,@hasta),
		@codigo_ciudad VARCHAR(5) = '66170'
/*
05001	MEDELLIN
08001	BARRANQUILLA
66170	DOSQUEBRADAS
54001	CUCUTA
52356	IPIALES
50001	VILLAVICENCIO
20001	VALLEDUPAR
*/

SELECT	DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		S.CodigoHabilitacion [Codigo Prestador],
		SUBSTRING(pac.TipoIdentificacion,1,2) [Tipo Documento],
		pac.Identificacion Documento,
		CAST(FechaFactura AS DATE) Fecha,
		C.NumeroAutorizacion [# Autorización],
		ACT.CodigoNacional [Codigo CUP],
		10 Finalidad,
		13 Causa,
		'' [DX Principal],
		'' [DX Relacionado 1],
		'' [DX Relacionado 2],
		'' [DX Relacionado 3],
		2  [Tipo DX],
		 ((B.ValorFactura*B.IndicadorFacturable)-B.ValorIvaFactura)*A.EstadoActivo ValorFactura1,
		'' ValorCopago,
		 ((B.ValorFactura*B.IndicadorFacturable)-B.ValorIvaFactura)*A.EstadoActivo ValorFactura,
		 ACT.Nombre
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('CONSULTA')
AND S.Codigo in (SELECT Codigo FROM GEN_CNF_Sedes WHERE CodigoTributarioCiudad = @codigo_ciudad)
and  NOT EXISTS (SELECT  *
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaId = A.FacturaId AND NOTAS.FechaFactura > @desde)
'''

# Information converted into a DataFrame with pandas
df = pd.read_sql(sql=sqlQuery, con=connection)

# Export the data on the excel into a Desktop
df.to_csv(os.environ["userprofile"] + "\\Documents\\Rips\\Sedes\\Dosquebradas\\" + "Dosquebradas_Consultas " +
            datetime.now().strftime("%d-%b-%Y %H%M%S") + ".csv", index=False)

C:\Users\ddrodriguez\AppData\Local\Temp\ipykernel_19832\2985184662.py:88: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=sqlQuery, con=connection)


###
<font color='SpringGreen' size="5"> Facturas Dosquebradas </font>

In [ ]:
# SQL command to read the data
sqlQuery = '''
--Facturas

DECLARE @hoy DATETIME = GETDATE()
DECLARE @hasta DATETIME = EOMONTH(@hoy, -1);
DECLARE @desde DATETIME =  DATETRUNC(month,@hasta),
		@codigo_ciudad VARCHAR(5) = '66170'
/*
05001	MEDELLIN
08001	BARRANQUILLA
66170	DOSQUEBRADAS
54001	CUCUTA
52356	IPIALES
50001	VILLAVICENCIO
20001	VALLEDUPAR
*/


SELECT
		[Codigo Prestador],
		[Razon Social],
		[Tipo Identificacion],
		[Identificacion],
		Factura,
		Fecha,
		FechaInicial,
		FechaDesde,
		CodigoResponsable,
		NombreResponsable,
		[# Contrato],
		[Plan Beneficios],
		[# Poliza],
		[Valor Copago],
		[Valor Descuento],
		sum(ValorFactura1) Valor
FROM (
SELECT  S.CodigoHabilitacion [Codigo Prestador],
		'FUNDACION ORIENTAME' [Razon Social],
		'NI' [Tipo Identificacion],
		'860049972' [Identificacion],
		DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		CAST(FechaFactura AS DATE) Fecha,
		CAST(@desde AS DATE) FechaInicial,
		CAST(@hasta AS DATE) FechaDesde,
		CASE
			WHEN M.CodigoResponsable = '900156264' THEN 'EPS037'
			WHEN M.CodigoResponsable = '830003564' THEN 'EPS017'
			WHEN M.CodigoResponsable = '800088702' THEN 'EPS018'
			WHEN M.CodigoResponsable = '830002272' THEN 'EPS019'
			WHEN M.CodigoResponsable = '901127521' THEN 'EPS020'
			ELSE 'SDS001' END CodigoResponsable,

		CASE
			WHEN M.CodigoResponsable = '900156264' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '830003564' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '800088702' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '830002272' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '901127521' THEN A.NombreResponsable
			ELSE 'FUNDACION ORIENTAME' END NombreResponsable,
		'' [# Contrato],
		'IVE' [Plan Beneficios],
		'' [# Poliza],
		'' [Valor Copago],
		'' [Valor Descuento],
		((B.ValorFactura * B.IndicadorFacturable ) - B.ValorIvaFactura ) * A.EstadoActivo ValorFactura1
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('CONSULTA')
AND S.Codigo in (SELECT Codigo FROM GEN_CNF_Sedes WHERE CodigoTributarioCiudad = @codigo_ciudad)
and  NOT EXISTS (SELECT  *
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaId = A.FacturaId AND NOTAS.FechaFactura > @desde)
UNION ALL
select   [Codigo Prestador],
		'FUNDACION ORIENTAME' [Razon Social],
		'NI' [Tipo Identificacion],
		'860049972' [Identificacion],
		Factura,
		Fecha,
		CAST(@desde AS DATE) FechaInicial,
		CAST(@hasta AS DATE) FechaDesde,
				CASE

			WHEN CodigoResponsable = '900156264' THEN 'EPS037'
			WHEN CodigoResponsable = '830003564' THEN 'EPS017'
			WHEN CodigoResponsable = '800088702' THEN 'EPS018'
			WHEN CodigoResponsable = '830002272' THEN 'EPS019'
			WHEN CodigoResponsable = '901127521' THEN 'EPS020' ELSE 'SDS001' END CodigoResponsable, --	P6.Valor ,
				CASE

			WHEN CodigoResponsable = '900156264' THEN NombreResponsable
			WHEN CodigoResponsable = '830003564' THEN NombreResponsable
			WHEN CodigoResponsable = '800088702' THEN NombreResponsable
			WHEN CodigoResponsable = '830002272' THEN NombreResponsable
			WHEN CodigoResponsable = '901127521' THEN NombreResponsable ELSE 'FUNDACION ORIENTAME' END NombreResponsable	,
		'' [# Contrato],
		'IVE' [Plan Beneficios],
		'' [# Poliza],
		'' [Valor Copago],
		'' [Valor Descuento],
		ValorFactura1
from(
 SELECT	Row_number() over(partition by pac.Identificacion,ACT.ActividadId order by pac.Identificacion,A.Numero ASC) fila,
		DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		S.CodigoHabilitacion [Codigo Prestador],
		SUBSTRING(pac.TipoIdentificacion,1,2) [Tipo Documento],
		pac.Identificacion Documento,
		CAST(FechaFactura AS DATE) Fecha,
		'' [# Autorización],
		ACT.CodigoNacional [Codigo CUP],
		1 Ambito,
		2 Finalidad,
		'' Personal,
		'' [DX Principal],
		'' [DX Relacionado 1],
		2  [DX Complicaciones],
		1 [Realizacion],
		M.CodigoResponsable,
		A.NombreResponsable	,
		 ((B.ValorFactura*B.IndicadorFacturable)-B.ValorIvaFactura)*A.EstadoActivo ValorFactura1,
		 ACT.Nombre
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('PROCEDIMIENTOS')
AND S.Codigo in (SELECT Codigo FROM GEN_CNF_Sedes WHERE CodigoTributarioCiudad = @codigo_ciudad)
and  NOT EXISTS (SELECT  NOTAS.FacturaId,NOtas.Numero,  B.FacturaAfectadaId
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					LEFT JOIN FAC_MOV_FacturasAnexos B WITH (NOLOCK) ON NOTAS.FacturaId = B.FacturaEncabezadoId
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaAfectadaId = A.FacturaId  AND NOTAS.FechaFactura> @desde )
				) tabla
				where fila in (1,2)
	)	TABLA
group by
		[Codigo Prestador],
		[Razon Social],
		[Tipo Identificacion],
		[Identificacion],
		Factura,
		Fecha,
		FechaInicial,
		FechaDesde,
		CodigoResponsable,
		NombreResponsable,
		[# Contrato],
		[Plan Beneficios],
		[# Poliza],
		[Valor Copago],
		[Valor Descuento]
'''

# Information converted into a DataFrame with pandas
df = pd.read_sql(sql=sqlQuery, con=connection)

# Export the data on the excel into a Desktop
df.to_csv(os.environ["userprofile"] + "\\Documents\\Rips\\Sedes\\Dosquebradas\\" + "Dosquebradas_Facturas " +
            datetime.now().strftime("%d-%b-%Y %H%M%S") + ".csv", index=False)

C:\Users\ddrodriguez\AppData\Local\Temp\ipykernel_19832\2138371779.py:235: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=sqlQuery, con=connection)


###
<font color='SpringGreen' size="5"> Procedimientos Dosquebradas </font>

In [ ]:
# SQL command to read the data
sqlQuery = '''
--PROCEDIMIENTOS Dosquebradas

DECLARE @hoy DATETIME = GETDATE()
DECLARE @hasta DATETIME = EOMONTH(@hoy, -1);
DECLARE @desde DATETIME =  DATETRUNC(month,@hasta),
		@codigo_ciudad VARCHAR(5) = '66170'
/*
05001	MEDELLIN
08001	BARRANQUILLA
66170	DOSQUEBRADAS
54001	CUCUTA
52356	IPIALES
50001	VILLAVICENCIO
20001	VALLEDUPAR
*/


select *
from(
SELECT	Row_number() over(partition by pac.Identificacion,ACT.ActividadId order by pac.Identificacion,FechaFactura) fila,
		DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		 S.CodigoHabilitacion [Codigo Prestador],
		SUBSTRING(pac.TipoIdentificacion,1,2) [Tipo Documento],
		pac.Identificacion Documento,
		CAST(FechaFactura AS DATE) Fecha,
		C.NumeroAutorizacion [# Autorizaci�n],
		ACT.CodigoNacional [Codigo CUP],
		1 Ambito,
		2 Finalidad,
		'' Personal,
		'' [DX Principal],
		'' [DX Relacionado 1],
		2  [DX Complicaciones],
		1 [Realizacion],
		 ((B.ValorFactura*B.IndicadorFacturable)-B.ValorIvaFactura)*A.EstadoActivo ValorFactura1,
		 ACT.Nombre
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('PROCEDIMIENTOS')
AND S.Codigo in (SELECT Codigo FROM GEN_CNF_Sedes WHERE CodigoTributarioCiudad = @codigo_ciudad)
and  NOT EXISTS (SELECT  NOTAS.FacturaId,NOtas.Numero,  B.FacturaAfectadaId
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					LEFT JOIN FAC_MOV_FacturasAnexos B WITH (NOLOCK) ON NOTAS.FacturaId = B.FacturaEncabezadoId
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaAfectadaId = A.FacturaId  AND NOTAS.FechaFactura> @desde )
				) tabla
				where fila IN (1,2)
'''

# Information converted into a DataFrame with pandas
df = pd.read_sql(sql=sqlQuery, con=connection)

# Export the data on the excel into a Desktop
df.to_csv(os.environ["userprofile"] + "\\Documents\\Rips\\Sedes\\Dosquebradas\\" + "Dosquebradas_Procedimientos " +
            datetime.now().strftime("%d-%b-%Y %H%M%S") + ".csv", index=False)

C:\Users\ddrodriguez\AppData\Local\Temp\ipykernel_19832\1437577848.py:93: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=sqlQuery, con=connection)


###
<font color='SpringGreen' size="5"> Usuarios Dosquebradas </font>

In [ ]:
# SQL command to read the data
sqlQuery = '''
--USUARIOS DOSQUEBRADAS

DECLARE @hoy DATETIME = GETDATE()
DECLARE @hasta DATETIME = EOMONTH(@hoy, -1);
DECLARE @desde DATETIME =  DATETRUNC(month,@hasta),
		@codigo_ciudad VARCHAR(5) = '66170'
/*
05001	MEDELLIN
08001	BARRANQUILLA
66170	DOSQUEBRADAS
54001	CUCUTA
52356	IPIALES
50001	VILLAVICENCIO
20001	VALLEDUPAR
*/


SELECT
	DISTINCT
		TABLA.[Tipo Documento],
		TABLA.Documento,
		CodigoResponsable,
		case when CodigoResponsable = 'SDS001' THEN 4 ELSE 1 END TipoUsuaurio,
		COALESCE(pac.PrimerApellido,'') PrimerApellido,
		COALESCE(pac.SegundoApellido,'') SegundoApellido,
		COALESCE(pac.PrimerNombre,'') PrimerNombre,
		COALESCE(pac.SegundoNombre,'') SegundoNombre,
		pac.FechaNacimiento,
		[dbo].[Fun_ORI_CalculaEdad](pac.FechaNacimiento,Fecha) Edad,
		1 Unidad,
		substring(Genero,1,1) Sexo,
		substring(ciudad.Referencia1,1,2) Depertamento,
		substring(ciudad.Referencia1,3,3) Ciudad,
		'U' Zona,
		case when pais.Valor = 'Colombia' or TABLA.[Tipo Documento] in ('CC','TI') then 'Colombia'

		else pais.Valor end Nacionalidad,
		case when pais.Valor = 'Colombia' or TABLA.[Tipo Documento] in ('CC','TI') then 'Nacional'

		else 'Extranjera' end TipoNacionalidad
FROM (
SELECT   S.CodigoHabilitacion [Codigo Prestador],
		'FUNDACION ORIENTAME' [Razon Social],
		'NI' [TipoNit],
		'860049972' Nit,
		SUBSTRING(pac.TipoIdentificacion,1,2) [Tipo Documento],
		pac.Identificacion Documento,
		DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		CAST(FechaFactura AS DATE) Fecha,
		CAST(@desde AS DATE) FechaInicial,
		CAST(@hasta AS DATE) FechaDesde,
		CASE
			WHEN M.CodigoResponsable = '900156264' THEN 'EPS037'
			WHEN M.CodigoResponsable = '830003564' THEN 'EPS017'
			WHEN M.CodigoResponsable = '800088702' THEN 'EPS018'
			WHEN M.CodigoResponsable = '830002272' THEN 'EPS019'
			WHEN M.CodigoResponsable = '901127521' THEN 'EPS020'
			ELSE 'SDS001' END CodigoResponsable,

		CASE
			WHEN M.CodigoResponsable = '900156264' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '830003564' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '800088702' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '830002272' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '901127521' THEN A.NombreResponsable
			ELSE 'FUNDACION ORIENTAME' END NombreResponsable,
		'' [# Contrato],
		'IVE' [Plan Beneficios],
		'' [# Poliza],
		'' [Valor Copago],
		'' [Valor Descuento],
		((B.ValorFactura * B.IndicadorFacturable ) - B.ValorIvaFactura ) * A.EstadoActivo ValorFactura1
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('CONSULTA')
AND S.Codigo in (SELECT Codigo FROM GEN_CNF_Sedes WHERE CodigoTributarioCiudad = @codigo_ciudad)
and  NOT EXISTS (SELECT  *
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaId = A.FacturaId AND NOTAS.FechaFactura > @desde)
UNION ALL
select  [Codigo Prestador],
		'FUNDACION ORIENTAME' [Razon Social],
		'NI' [TipoNit],
		'860049972' Nit,
		[Tipo Documento],
		Documento,
		Factura,
		Fecha,
		CAST(@desde AS DATE) FechaInicial,
		CAST(@hasta AS DATE) FechaDesde,
				CASE

			WHEN CodigoResponsable = '900156264' THEN 'EPS037'
			WHEN CodigoResponsable = '830003564' THEN 'EPS017'
			WHEN CodigoResponsable = '800088702' THEN 'EPS018'
			WHEN CodigoResponsable = '830002272' THEN 'EPS019'
			WHEN CodigoResponsable = '901127521' THEN 'EPS020' ELSE 'SDS001' END CodigoResponsable, --	P6.Valor ,
				CASE

			WHEN CodigoResponsable = '900156264' THEN NombreResponsable
			WHEN CodigoResponsable = '830003564' THEN NombreResponsable
			WHEN CodigoResponsable = '800088702' THEN NombreResponsable
			WHEN CodigoResponsable = '830002272' THEN NombreResponsable
			WHEN CodigoResponsable = '901127521' THEN NombreResponsable ELSE 'FUNDACION ORIENTAME' END NombreResponsable	,
		'' [# Contrato],
		'IVE' [Plan Beneficios],
		'' [# Poliza],
		'' [Valor Copago],
		'' [Valor Descuento],
		ValorFactura1
from(
 SELECT	Row_number() over(partition by pac.Identificacion,ACT.ActividadId order by pac.Identificacion,A.Numero ASC) fila,
		DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		 S.CodigoHabilitacion [Codigo Prestador],
		SUBSTRING(pac.TipoIdentificacion,1,2) [Tipo Documento],
		pac.Identificacion Documento,
		CAST(FechaFactura AS DATE) Fecha,
		'' [# Autorizaci�n],
		ACT.CodigoNacional [Codigo CUP],
		1 Ambito,
		2 Finalidad,
		'' Personal,
		'' [DX Principal],
		'' [DX Relacionado 1],
		2  [DX Complicaciones],
		1 [Realizacion],
			M.CodigoResponsable, --	P6.Valor ,
		A.NombreResponsable	,
		 ((B.ValorFactura*B.IndicadorFacturable)-B.ValorIvaFactura)*A.EstadoActivo ValorFactura1,
		 ACT.Nombre
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('PROCEDIMIENTOS')
AND S.Codigo in (SELECT Codigo FROM GEN_CNF_Sedes WHERE CodigoTributarioCiudad = @codigo_ciudad)
and  NOT EXISTS (SELECT  NOTAS.FacturaId,NOtas.Numero,  B.FacturaAfectadaId
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					LEFT JOIN FAC_MOV_FacturasAnexos B WITH (NOLOCK) ON NOTAS.FacturaId = B.FacturaEncabezadoId
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaAfectadaId = A.FacturaId  AND NOTAS.FechaFactura> @desde )
				) tabla
					where fila in (1,2)
	)	TABLA
	INNER JOIN ADT_VIS_Pacientes pac with (nolock) ON pac.Identificacion = TABLA.Documento and SUBSTRING(pac.TipoIdentificacion,1,2) = TABLA.[Tipo Documento]
	left join GEN_CNF_Parametros ciudad on ciudad.ParametroId = MunicipioResidenciaId
	left join GEN_CNF_Parametros nacimiento on nacimiento.ParametroId = LugarNacimientoId
	left join GEN_CNF_Parametros departamento on departamento.ParametroId = nacimiento.PadreId
	left join GEN_CNF_Parametros pais on pais.ParametroId = departamento.PadreId
'''

# Information converted into a DataFrame with pandas
df = pd.read_sql(sql=sqlQuery, con=connection)

# Export the data on the excel into a Desktop
df.to_csv(os.environ["userprofile"] + "\\Documents\\Rips\\Sedes\\Dosquebradas\\" + "Dosquebradas_Usuarios " +
            datetime.now().strftime("%d-%b-%Y %H%M%S") + ".csv", index=False)

#Display notification to user
notification.notify(title="✅Report Status!!!",
                    message= f"Queries of Dosquebradas has been succesfully saved into .csv\
                        \n Queries Executed:  \n 1. Consultas, 2.Facturas, 3.Procedimientos, 4.Usuarios",
                        timeout=10)

C:\Users\ddrodriguez\AppData\Local\Temp\ipykernel_19832\3956365043.py:234: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=sqlQuery, con=connection)


##
<font color='SteelBlue' size="6">  CUCUTA </font>

###
<font color='SteelBlue' size="5"> Consultas Cucuta </font>

In [ ]:
# SQL command to read the data
sqlQuery = '''
--CONSULTAS

DECLARE @hoy DATETIME = GETDATE()
DECLARE @hasta DATETIME = EOMONTH(@hoy, -1);
DECLARE @desde DATETIME =  DATETRUNC(month,@hasta),
		@codigo_ciudad VARCHAR(5) = '54001'

/*
05001	MEDELLIN
08001	BARRANQUILLA
66170	DOSQUEBRADAS
54001	CUCUTA
52356	IPIALES
50001	VILLAVICENCIO
20001	VALLEDUPAR
*/

SELECT	DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		S.CodigoHabilitacion [Codigo Prestador],
		SUBSTRING(pac.TipoIdentificacion,1,2) [Tipo Documento],
		pac.Identificacion Documento,
		CAST(FechaFactura AS DATE) Fecha,
		C.NumeroAutorizacion [# Autorización],
		ACT.CodigoNacional [Codigo CUP],
		10 Finalidad,
		13 Causa,
		'' [DX Principal],
		'' [DX Relacionado 1],
		'' [DX Relacionado 2],
		'' [DX Relacionado 3],
		2  [Tipo DX],
		 ((B.ValorFactura*B.IndicadorFacturable)-B.ValorIvaFactura)*A.EstadoActivo ValorFactura1,
		'' ValorCopago,
		 ((B.ValorFactura*B.IndicadorFacturable)-B.ValorIvaFactura)*A.EstadoActivo ValorFactura,
		 ACT.Nombre
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('CONSULTA')
AND S.Codigo in (SELECT Codigo FROM GEN_CNF_Sedes WHERE CodigoTributarioCiudad = @codigo_ciudad)
and  NOT EXISTS (SELECT  *
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaId = A.FacturaId AND NOTAS.FechaFactura > @desde)
'''

# Information converted into a DataFrame with pandas
df = pd.read_sql(sql=sqlQuery, con=connection)

# Export the data on the excel into a Desktop
df.to_csv(os.environ["userprofile"] + "\\Documents\\Rips\\Sedes\\Cucuta\\" + "Cucuta_Consultas " +
            datetime.now().strftime("%d-%b-%Y %H%M%S") + ".csv", index=False)

C:\Users\ddrodriguez\AppData\Local\Temp\ipykernel_19832\762936481.py:89: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=sqlQuery, con=connection)


###
<font color='SteelBlue' size="5"> Facturas Cucuta </font>

In [ ]:
# SQL command to read the data
sqlQuery = '''
--FACTURAS CUCUTA

DECLARE @hoy DATETIME = GETDATE()
DECLARE @hasta DATETIME = EOMONTH(@hoy, -1);
DECLARE @desde DATETIME =  DATETRUNC(month,@hasta),
		@codigo_ciudad VARCHAR(5) = '54001'
/*
05001	MEDELLIN
08001	BARRANQUILLA
66170	DOSQUEBRADAS
54001	CUCUTA
52356	IPIALES
50001	VILLAVICENCIO
20001	VALLEDUPAR
*/


SELECT
		[Codigo Prestador],
		[Razon Social],
		[Tipo Identificacion],
		[Identificacion],
		Factura,
		Fecha,
		FechaInicial,
		FechaDesde,
		CodigoResponsable,
		NombreResponsable,
		[# Contrato],
		[Plan Beneficios],
		[# Poliza],
		[Valor Copago],
		[Valor Descuento],
		sum(ValorFactura1) Valor
FROM (
SELECT  S.CodigoHabilitacion [Codigo Prestador],
		'FUNDACION ORIENTAME' [Razon Social],
		'NI' [Tipo Identificacion],
		'860049972' [Identificacion],
		DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		CAST(FechaFactura AS DATE) Fecha,
		CAST(@desde AS DATE) FechaInicial,
		CAST(@hasta AS DATE) FechaDesde,
		CASE
			WHEN M.CodigoResponsable = '900156264' THEN 'EPS037'
			WHEN M.CodigoResponsable = '830003564' THEN 'EPS017'
			WHEN M.CodigoResponsable = '800088702' THEN 'EPS018'
			WHEN M.CodigoResponsable = '830002272' THEN 'EPS019'
			WHEN M.CodigoResponsable = '901127521' THEN 'EPS020'
			ELSE 'SDS001' END CodigoResponsable,

		CASE
			WHEN M.CodigoResponsable = '900156264' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '830003564' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '800088702' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '830002272' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '901127521' THEN A.NombreResponsable
			ELSE 'FUNDACION ORIENTAME' END NombreResponsable,
		'' [# Contrato],
		'IVE' [Plan Beneficios],
		'' [# Poliza],
		'' [Valor Copago],
		'' [Valor Descuento],
		((B.ValorFactura * B.IndicadorFacturable ) - B.ValorIvaFactura ) * A.EstadoActivo ValorFactura1
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('CONSULTA')
AND S.Codigo in (SELECT Codigo FROM GEN_CNF_Sedes WHERE CodigoTributarioCiudad = @codigo_ciudad)
and  NOT EXISTS (SELECT  *
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaId = A.FacturaId AND NOTAS.FechaFactura > @desde)
UNION ALL
select   [Codigo Prestador],
		'FUNDACION ORIENTAME' [Razon Social],
		'NI' [Tipo Identificacion],
		'860049972' [Identificacion],
		Factura,
		Fecha,
		CAST(@desde AS DATE) FechaInicial,
		CAST(@hasta AS DATE) FechaDesde,
				CASE

			WHEN CodigoResponsable = '900156264' THEN 'EPS037'
			WHEN CodigoResponsable = '830003564' THEN 'EPS017'
			WHEN CodigoResponsable = '800088702' THEN 'EPS018'
			WHEN CodigoResponsable = '830002272' THEN 'EPS019'
			WHEN CodigoResponsable = '901127521' THEN 'EPS020' ELSE 'SDS001' END CodigoResponsable, --	P6.Valor ,
				CASE

			WHEN CodigoResponsable = '900156264' THEN NombreResponsable
			WHEN CodigoResponsable = '830003564' THEN NombreResponsable
			WHEN CodigoResponsable = '800088702' THEN NombreResponsable
			WHEN CodigoResponsable = '830002272' THEN NombreResponsable
			WHEN CodigoResponsable = '901127521' THEN NombreResponsable ELSE 'FUNDACION ORIENTAME' END NombreResponsable	,
		'' [# Contrato],
		'IVE' [Plan Beneficios],
		'' [# Poliza],
		'' [Valor Copago],
		'' [Valor Descuento],
		ValorFactura1
from(
 SELECT	Row_number() over(partition by pac.Identificacion,ACT.ActividadId order by pac.Identificacion,A.Numero ASC) fila,
		DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		S.CodigoHabilitacion [Codigo Prestador],
		SUBSTRING(pac.TipoIdentificacion,1,2) [Tipo Documento],
		pac.Identificacion Documento,
		CAST(FechaFactura AS DATE) Fecha,
		'' [# Autorización],
		ACT.CodigoNacional [Codigo CUP],
		1 Ambito,
		2 Finalidad,
		'' Personal,
		'' [DX Principal],
		'' [DX Relacionado 1],
		2  [DX Complicaciones],
		1 [Realizacion],
		M.CodigoResponsable,
		A.NombreResponsable	,
		 ((B.ValorFactura*B.IndicadorFacturable)-B.ValorIvaFactura)*A.EstadoActivo ValorFactura1,
		 ACT.Nombre
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('PROCEDIMIENTOS')
AND S.Codigo in (SELECT Codigo FROM GEN_CNF_Sedes WHERE CodigoTributarioCiudad = @codigo_ciudad)
and  NOT EXISTS (SELECT  NOTAS.FacturaId,NOtas.Numero,  B.FacturaAfectadaId
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					LEFT JOIN FAC_MOV_FacturasAnexos B WITH (NOLOCK) ON NOTAS.FacturaId = B.FacturaEncabezadoId
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaAfectadaId = A.FacturaId  AND NOTAS.FechaFactura> @desde )
				) tabla
				where fila in (1,2)
	)	TABLA
group by
		[Codigo Prestador],
		[Razon Social],
		[Tipo Identificacion],
		[Identificacion],
		Factura,
		Fecha,
		FechaInicial,
		FechaDesde,
		CodigoResponsable,
		NombreResponsable,
		[# Contrato],
		[Plan Beneficios],
		[# Poliza],
		[Valor Copago],
		[Valor Descuento]
'''

# Information converted into a DataFrame with pandas
df = pd.read_sql(sql=sqlQuery, con=connection)

# Export the data on the excel into a Desktop
df.to_csv(os.environ["userprofile"] + "\\Documents\\Rips\\Sedes\\Cucuta\\" + "Cucuta_Facturas " +
            datetime.now().strftime("%d-%b-%Y %H%M%S") + ".csv", index=False)

C:\Users\ddrodriguez\AppData\Local\Temp\ipykernel_19832\4056309626.py:235: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=sqlQuery, con=connection)


###
<font color='SteelBlue' size="5"> Procedimientos Cucuta </font>

In [ ]:
# SQL command to read the data
sqlQuery = '''
--PROCEDIMIENTOS

DECLARE @hoy DATETIME = GETDATE()
DECLARE @hasta DATETIME = EOMONTH(@hoy, -1);
DECLARE @desde DATETIME =  DATETRUNC(month,@hasta),
		@codigo_ciudad VARCHAR(5) = '54001'
/*
05001	MEDELLIN
08001	BARRANQUILLA
66170	DOSQUEBRADAS
54001	CUCUTA
52356	IPIALES
50001	VILLAVICENCIO
20001	VALLEDUPAR
*/


select *
from(
SELECT	Row_number() over(partition by pac.Identificacion,ACT.ActividadId order by pac.Identificacion,FechaFactura) fila,
		DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		 S.CodigoHabilitacion [Codigo Prestador],
		SUBSTRING(pac.TipoIdentificacion,1,2) [Tipo Documento],
		pac.Identificacion Documento,
		CAST(FechaFactura AS DATE) Fecha,
		C.NumeroAutorizacion [# Autorizaci�n],
		ACT.CodigoNacional [Codigo CUP],
		1 Ambito,
		2 Finalidad,
		'' Personal,
		'' [DX Principal],
		'' [DX Relacionado 1],
		2  [DX Complicaciones],
		1 [Realizacion],
		 ((B.ValorFactura*B.IndicadorFacturable)-B.ValorIvaFactura)*A.EstadoActivo ValorFactura1,
		 ACT.Nombre
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('PROCEDIMIENTOS')
AND S.Codigo in (SELECT Codigo FROM GEN_CNF_Sedes WHERE CodigoTributarioCiudad = @codigo_ciudad)
and  NOT EXISTS (SELECT  NOTAS.FacturaId,NOtas.Numero,  B.FacturaAfectadaId
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					LEFT JOIN FAC_MOV_FacturasAnexos B WITH (NOLOCK) ON NOTAS.FacturaId = B.FacturaEncabezadoId
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaAfectadaId = A.FacturaId  AND NOTAS.FechaFactura> @desde )
				) tabla
				where fila IN (1,2)
'''

# Information converted into a DataFrame with pandas
df = pd.read_sql(sql=sqlQuery, con=connection)

# Export the data on the excel into a Desktop
df.to_csv(os.environ["userprofile"] + "\\Documents\\Rips\\Sedes\\Cucuta\\" + "Cucuta_Procedimientos " +
            datetime.now().strftime("%d-%b-%Y %H%M%S") + ".csv", index=False)

C:\Users\ddrodriguez\AppData\Local\Temp\ipykernel_19832\23072604.py:93: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=sqlQuery, con=connection)


###
<font color='SteelBlue' size="5"> Usuarios Cucuta </font>

In [ ]:
# SQL command to read the data
sqlQuery = '''
--Usuarios Cucuta

DECLARE @hoy DATETIME = GETDATE()
DECLARE @hasta DATETIME = EOMONTH(@hoy, -1);
DECLARE @desde DATETIME =  DATETRUNC(month,@hasta),
		@codigo_ciudad VARCHAR(5) = '54001'
/*
05001	MEDELLIN
08001	BARRANQUILLA
66170	DOSQUEBRADAS
54001	CUCUTA
52356	IPIALES
50001	VILLAVICENCIO
20001	VALLEDUPAR
*/


SELECT
	DISTINCT
		TABLA.[Tipo Documento],
		TABLA.Documento,
		CodigoResponsable,
		case when CodigoResponsable = 'SDS001' THEN 4 ELSE 1 END TipoUsuaurio,
		COALESCE(pac.PrimerApellido,'') PrimerApellido,
		COALESCE(pac.SegundoApellido,'') SegundoApellido,
		COALESCE(pac.PrimerNombre,'') PrimerNombre,
		COALESCE(pac.SegundoNombre,'') SegundoNombre,
		pac.FechaNacimiento,
		[dbo].[Fun_ORI_CalculaEdad](pac.FechaNacimiento,Fecha) Edad,
		1 Unidad,
		substring(Genero,1,1) Sexo,
		substring(ciudad.Referencia1,1,2) Depertamento,
		substring(ciudad.Referencia1,3,3) Ciudad,
		'U' Zona,
		case when pais.Valor = 'Colombia' or TABLA.[Tipo Documento] in ('CC','TI') then 'Colombia'

		else pais.Valor end Nacionalidad,
		case when pais.Valor = 'Colombia' or TABLA.[Tipo Documento] in ('CC','TI') then 'Nacional'

		else 'Extranjera' end TipoNacionalidad
FROM (
SELECT   S.CodigoHabilitacion [Codigo Prestador],
		'FUNDACION ORIENTAME' [Razon Social],
		'NI' [TipoNit],
		'860049972' Nit,
		SUBSTRING(pac.TipoIdentificacion,1,2) [Tipo Documento],
		pac.Identificacion Documento,
		DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		CAST(FechaFactura AS DATE) Fecha,
		CAST(@desde AS DATE) FechaInicial,
		CAST(@hasta AS DATE) FechaDesde,
		CASE
			WHEN M.CodigoResponsable = '900156264' THEN 'EPS037'
			WHEN M.CodigoResponsable = '830003564' THEN 'EPS017'
			WHEN M.CodigoResponsable = '800088702' THEN 'EPS018'
			WHEN M.CodigoResponsable = '830002272' THEN 'EPS019'
			WHEN M.CodigoResponsable = '901127521' THEN 'EPS020'
			ELSE 'SDS001' END CodigoResponsable,

		CASE
			WHEN M.CodigoResponsable = '900156264' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '830003564' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '800088702' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '830002272' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '901127521' THEN A.NombreResponsable
			ELSE 'FUNDACION ORIENTAME' END NombreResponsable,
		'' [# Contrato],
		'IVE' [Plan Beneficios],
		'' [# Poliza],
		'' [Valor Copago],
		'' [Valor Descuento],
		((B.ValorFactura * B.IndicadorFacturable ) - B.ValorIvaFactura ) * A.EstadoActivo ValorFactura1
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('CONSULTA')
AND S.Codigo in (SELECT Codigo FROM GEN_CNF_Sedes WHERE CodigoTributarioCiudad = @codigo_ciudad)
and  NOT EXISTS (SELECT  *
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaId = A.FacturaId AND NOTAS.FechaFactura > @desde)
UNION ALL
select  [Codigo Prestador],
		'FUNDACION ORIENTAME' [Razon Social],
		'NI' [TipoNit],
		'860049972' Nit,
		[Tipo Documento],
		Documento,
		Factura,
		Fecha,
		CAST(@desde AS DATE) FechaInicial,
		CAST(@hasta AS DATE) FechaDesde,
				CASE

			WHEN CodigoResponsable = '900156264' THEN 'EPS037'
			WHEN CodigoResponsable = '830003564' THEN 'EPS017'
			WHEN CodigoResponsable = '800088702' THEN 'EPS018'
			WHEN CodigoResponsable = '830002272' THEN 'EPS019'
			WHEN CodigoResponsable = '901127521' THEN 'EPS020' ELSE 'SDS001' END CodigoResponsable, --	P6.Valor ,
				CASE

			WHEN CodigoResponsable = '900156264' THEN NombreResponsable
			WHEN CodigoResponsable = '830003564' THEN NombreResponsable
			WHEN CodigoResponsable = '800088702' THEN NombreResponsable
			WHEN CodigoResponsable = '830002272' THEN NombreResponsable
			WHEN CodigoResponsable = '901127521' THEN NombreResponsable ELSE 'FUNDACION ORIENTAME' END NombreResponsable	,
		'' [# Contrato],
		'IVE' [Plan Beneficios],
		'' [# Poliza],
		'' [Valor Copago],
		'' [Valor Descuento],
		ValorFactura1
from(
 SELECT	Row_number() over(partition by pac.Identificacion,ACT.ActividadId order by pac.Identificacion,A.Numero ASC) fila,
		DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		 S.CodigoHabilitacion [Codigo Prestador],
		SUBSTRING(pac.TipoIdentificacion,1,2) [Tipo Documento],
		pac.Identificacion Documento,
		CAST(FechaFactura AS DATE) Fecha,
		'' [# Autorizaci�n],
		ACT.CodigoNacional [Codigo CUP],
		1 Ambito,
		2 Finalidad,
		'' Personal,
		'' [DX Principal],
		'' [DX Relacionado 1],
		2  [DX Complicaciones],
		1 [Realizacion],
			M.CodigoResponsable, --	P6.Valor ,
		A.NombreResponsable	,
		 ((B.ValorFactura*B.IndicadorFacturable)-B.ValorIvaFactura)*A.EstadoActivo ValorFactura1,
		 ACT.Nombre
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('PROCEDIMIENTOS')
AND S.Codigo in (SELECT Codigo FROM GEN_CNF_Sedes WHERE CodigoTributarioCiudad = @codigo_ciudad)
and  NOT EXISTS (SELECT  NOTAS.FacturaId,NOtas.Numero,  B.FacturaAfectadaId
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					LEFT JOIN FAC_MOV_FacturasAnexos B WITH (NOLOCK) ON NOTAS.FacturaId = B.FacturaEncabezadoId
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaAfectadaId = A.FacturaId  AND NOTAS.FechaFactura> @desde )
				) tabla
					where fila in (1,2)
	)	TABLA
	INNER JOIN ADT_VIS_Pacientes pac with (nolock) ON pac.Identificacion = TABLA.Documento and SUBSTRING(pac.TipoIdentificacion,1,2) = TABLA.[Tipo Documento]
	left join GEN_CNF_Parametros ciudad on ciudad.ParametroId = MunicipioResidenciaId
	left join GEN_CNF_Parametros nacimiento on nacimiento.ParametroId = LugarNacimientoId
	left join GEN_CNF_Parametros departamento on departamento.ParametroId = nacimiento.PadreId
	left join GEN_CNF_Parametros pais on pais.ParametroId = departamento.PadreId
'''

# Information converted into a DataFrame with pandas
df = pd.read_sql(sql=sqlQuery, con=connection)

# Export the data on the excel into a Desktop
df.to_csv(os.environ["userprofile"] + "\\Documents\\Rips\\Sedes\\Cucuta\\" + "Cucuta_Usuarios " +
            datetime.now().strftime("%d-%b-%Y %H%M%S") + ".csv", index=False)

#Display notification to user
notification.notify(title="✅Report Status!!!",
                    message= f"Queries of Cucuta has been succesfully saved into .csv\
                        \n Queries Executed:  \n 1. Consultas, 2.Facturas, 3.Procedimientos, 4.Usuarios",
                        timeout=10)

C:\Users\ddrodriguez\AppData\Local\Temp\ipykernel_19832\2870544186.py:234: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=sqlQuery, con=connection)


##
<font color='Wheat' size="6"> IPIALES </font>

###
<font color='Wheat' size="5"> Consultas Ipiales </font>

In [ ]:
# SQL command to read the data
sqlQuery = '''
--CONSULTAS

DECLARE @hoy DATETIME = GETDATE()
DECLARE @hasta DATETIME = EOMONTH(@hoy, -1);
DECLARE @desde DATETIME =  DATETRUNC(month,@hasta),
		@codigo_ciudad VARCHAR(5) = '52356'
/*
05001	MEDELLIN
08001	BARRANQUILLA
66170	DOSQUEBRADAS
54001	CUCUTA
52356	IPIALES
50001	VILLAVICENCIO
20001	VALLEDUPAR
*/

SELECT	DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		S.CodigoHabilitacion [Codigo Prestador],
		SUBSTRING(pac.TipoIdentificacion,1,2) [Tipo Documento],
		pac.Identificacion Documento,
		CAST(FechaFactura AS DATE) Fecha,
		C.NumeroAutorizacion [# Autorización],
		ACT.CodigoNacional [Codigo CUP],
		10 Finalidad,
		13 Causa,
		'' [DX Principal],
		'' [DX Relacionado 1],
		'' [DX Relacionado 2],
		'' [DX Relacionado 3],
		2  [Tipo DX],
		 ((B.ValorFactura*B.IndicadorFacturable)-B.ValorIvaFactura)*A.EstadoActivo ValorFactura1,
		'' ValorCopago,
		 ((B.ValorFactura*B.IndicadorFacturable)-B.ValorIvaFactura)*A.EstadoActivo ValorFactura,
		 ACT.Nombre
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('CONSULTA')
AND S.Codigo in (SELECT Codigo FROM GEN_CNF_Sedes WHERE CodigoTributarioCiudad = @codigo_ciudad)
and  NOT EXISTS (SELECT  *
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaId = A.FacturaId AND NOTAS.FechaFactura > @desde)
'''

# Information converted into a DataFrame with pandas
df = pd.read_sql(sql=sqlQuery, con=connection)

# Export the data on the excel into a Desktop
df.to_csv(os.environ["userprofile"] + "\\Documents\\Rips\\Sedes\\Ipiales\\" + "Ipiales_Consultas " +
            datetime.now().strftime("%d-%b-%Y %H%M%S") + ".csv", index=False)

C:\Users\ddrodriguez\AppData\Local\Temp\ipykernel_19832\37802394.py:88: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=sqlQuery, con=connection)


###
<font color='Wheat' size="5"> Facturas Ipiales </font>

In [ ]:
# SQL command to read the data
sqlQuery = '''
--FACTURAS IPIALES

DECLARE @hoy DATETIME = GETDATE()
DECLARE @hasta DATETIME = EOMONTH(@hoy, -1);
DECLARE @desde DATETIME =  DATETRUNC(month,@hasta),
		@codigo_ciudad VARCHAR(5) = '52356'
/*
05001	MEDELLIN
08001	BARRANQUILLA
66170	DOSQUEBRADAS
54001	CUCUTA
52356	IPIALES
50001	VILLAVICENCIO
20001	VALLEDUPAR
*/


SELECT
		[Codigo Prestador],
		[Razon Social],
		[Tipo Identificacion],
		[Identificacion],
		Factura,
		Fecha,
		FechaInicial,
		FechaDesde,
		CodigoResponsable,
		NombreResponsable,
		[# Contrato],
		[Plan Beneficios],
		[# Poliza],
		[Valor Copago],
		[Valor Descuento],
		sum(ValorFactura1) Valor
FROM (
SELECT  S.CodigoHabilitacion [Codigo Prestador],
		'FUNDACION ORIENTAME' [Razon Social],
		'NI' [Tipo Identificacion],
		'860049972' [Identificacion],
		DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		CAST(FechaFactura AS DATE) Fecha,
		CAST(@desde AS DATE) FechaInicial,
		CAST(@hasta AS DATE) FechaDesde,
		CASE
			WHEN M.CodigoResponsable = '900156264' THEN 'EPS037'
			WHEN M.CodigoResponsable = '830003564' THEN 'EPS017'
			WHEN M.CodigoResponsable = '800088702' THEN 'EPS018'
			WHEN M.CodigoResponsable = '830002272' THEN 'EPS019'
			WHEN M.CodigoResponsable = '901127521' THEN 'EPS020'
			ELSE 'SDS001' END CodigoResponsable,

		CASE
			WHEN M.CodigoResponsable = '900156264' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '830003564' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '800088702' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '830002272' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '901127521' THEN A.NombreResponsable
			ELSE 'FUNDACION ORIENTAME' END NombreResponsable,
		'' [# Contrato],
		'IVE' [Plan Beneficios],
		'' [# Poliza],
		'' [Valor Copago],
		'' [Valor Descuento],
		((B.ValorFactura * B.IndicadorFacturable ) - B.ValorIvaFactura ) * A.EstadoActivo ValorFactura1
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('CONSULTA')
AND S.Codigo in (SELECT Codigo FROM GEN_CNF_Sedes WHERE CodigoTributarioCiudad = @codigo_ciudad)
and  NOT EXISTS (SELECT  *
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaId = A.FacturaId AND NOTAS.FechaFactura > @desde)
UNION ALL
select   [Codigo Prestador],
		'FUNDACION ORIENTAME' [Razon Social],
		'NI' [Tipo Identificacion],
		'860049972' [Identificacion],
		Factura,
		Fecha,
		CAST(@desde AS DATE) FechaInicial,
		CAST(@hasta AS DATE) FechaDesde,
				CASE

			WHEN CodigoResponsable = '900156264' THEN 'EPS037'
			WHEN CodigoResponsable = '830003564' THEN 'EPS017'
			WHEN CodigoResponsable = '800088702' THEN 'EPS018'
			WHEN CodigoResponsable = '830002272' THEN 'EPS019'
			WHEN CodigoResponsable = '901127521' THEN 'EPS020' ELSE 'SDS001' END CodigoResponsable, --	P6.Valor ,
				CASE

			WHEN CodigoResponsable = '900156264' THEN NombreResponsable
			WHEN CodigoResponsable = '830003564' THEN NombreResponsable
			WHEN CodigoResponsable = '800088702' THEN NombreResponsable
			WHEN CodigoResponsable = '830002272' THEN NombreResponsable
			WHEN CodigoResponsable = '901127521' THEN NombreResponsable ELSE 'FUNDACION ORIENTAME' END NombreResponsable	,
		'' [# Contrato],
		'IVE' [Plan Beneficios],
		'' [# Poliza],
		'' [Valor Copago],
		'' [Valor Descuento],
		ValorFactura1
from(
 SELECT	Row_number() over(partition by pac.Identificacion,ACT.ActividadId order by pac.Identificacion,A.Numero ASC) fila,
		DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		S.CodigoHabilitacion [Codigo Prestador],
		SUBSTRING(pac.TipoIdentificacion,1,2) [Tipo Documento],
		pac.Identificacion Documento,
		CAST(FechaFactura AS DATE) Fecha,
		'' [# Autorización],
		ACT.CodigoNacional [Codigo CUP],
		1 Ambito,
		2 Finalidad,
		'' Personal,
		'' [DX Principal],
		'' [DX Relacionado 1],
		2  [DX Complicaciones],
		1 [Realizacion],
		M.CodigoResponsable,
		A.NombreResponsable	,
		 ((B.ValorFactura*B.IndicadorFacturable)-B.ValorIvaFactura)*A.EstadoActivo ValorFactura1,
		 ACT.Nombre
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('PROCEDIMIENTOS')
AND S.Codigo in (SELECT Codigo FROM GEN_CNF_Sedes WHERE CodigoTributarioCiudad = @codigo_ciudad)
and  NOT EXISTS (SELECT  NOTAS.FacturaId,NOtas.Numero,  B.FacturaAfectadaId
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					LEFT JOIN FAC_MOV_FacturasAnexos B WITH (NOLOCK) ON NOTAS.FacturaId = B.FacturaEncabezadoId
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaAfectadaId = A.FacturaId  AND NOTAS.FechaFactura> @desde )
				) tabla
				where fila in (1,2)
	)	TABLA
group by
		[Codigo Prestador],
		[Razon Social],
		[Tipo Identificacion],
		[Identificacion],
		Factura,
		Fecha,
		FechaInicial,
		FechaDesde,
		CodigoResponsable,
		NombreResponsable,
		[# Contrato],
		[Plan Beneficios],
		[# Poliza],
		[Valor Copago],
		[Valor Descuento]
'''

# Information converted into a DataFrame with pandas
df = pd.read_sql(sql=sqlQuery, con=connection)

# Export the data on the excel into a Desktop
df.to_csv(os.environ["userprofile"] + "\\Documents\\Rips\\Sedes\\Ipiales\\" + "Ipiales_Facturas " +
            datetime.now().strftime("%d-%b-%Y %H%M%S") + ".csv", index=False)

C:\Users\ddrodriguez\AppData\Local\Temp\ipykernel_19832\2141245363.py:235: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=sqlQuery, con=connection)


###
<font color='Wheat' size="5"> Procedimientos Ipiales </font>

In [ ]:
# SQL command to read the data
sqlQuery = '''

--PROCEDIMIENTOS IPIALES

DECLARE @hoy DATETIME = GETDATE()
DECLARE @hasta DATETIME = EOMONTH(@hoy, -1);
DECLARE @desde DATETIME =  DATETRUNC(month,@hasta),
		@codigo_ciudad VARCHAR(5) = '52356'
/*
05001	MEDELLIN
08001	BARRANQUILLA
66170	DOSQUEBRADAS
54001	CUCUTA
52356	IPIALES
50001	VILLAVICENCIO
20001	VALLEDUPAR
*/


select *
from(
SELECT	Row_number() over(partition by pac.Identificacion,ACT.ActividadId order by pac.Identificacion,FechaFactura) fila,
		DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		 S.CodigoHabilitacion [Codigo Prestador],
		SUBSTRING(pac.TipoIdentificacion,1,2) [Tipo Documento],
		pac.Identificacion Documento,
		CAST(FechaFactura AS DATE) Fecha,
		C.NumeroAutorizacion [# Autorizaci�n],
		ACT.CodigoNacional [Codigo CUP],
		1 Ambito,
		2 Finalidad,
		'' Personal,
		'' [DX Principal],
		'' [DX Relacionado 1],
		2  [DX Complicaciones],
		1 [Realizacion],
		 ((B.ValorFactura*B.IndicadorFacturable)-B.ValorIvaFactura)*A.EstadoActivo ValorFactura1,
		 ACT.Nombre
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('PROCEDIMIENTOS')
AND S.Codigo in (SELECT Codigo FROM GEN_CNF_Sedes WHERE CodigoTributarioCiudad = @codigo_ciudad)
and  NOT EXISTS (SELECT  NOTAS.FacturaId,NOtas.Numero,  B.FacturaAfectadaId
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					LEFT JOIN FAC_MOV_FacturasAnexos B WITH (NOLOCK) ON NOTAS.FacturaId = B.FacturaEncabezadoId
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaAfectadaId = A.FacturaId  AND NOTAS.FechaFactura> @desde )
				) tabla
				where fila IN (1,2)
'''

# Information converted into a DataFrame with pandas
df = pd.read_sql(sql=sqlQuery, con=connection)

# Export the data on the excel into a Desktop
df.to_csv(os.environ["userprofile"] + "\\Documents\\Rips\\Sedes\\Ipiales\\" + "Ipiales_Procedimientos " +
            datetime.now().strftime("%d-%b-%Y %H%M%S") + ".csv", index=False)

C:\Users\ddrodriguez\AppData\Local\Temp\ipykernel_19832\2542664408.py:94: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=sqlQuery, con=connection)


###
<font color='Wheat' size="5"> Usuarios Ipiales </font>

In [ ]:
# SQL command to read the data
sqlQuery = '''
--USUARIOS IPIALES

DECLARE @hoy DATETIME = GETDATE()
DECLARE @hasta DATETIME = EOMONTH(@hoy, -1);
DECLARE @desde DATETIME =  DATETRUNC(month,@hasta),
		@codigo_ciudad VARCHAR(5) = '52356'
/*
05001	MEDELLIN
08001	BARRANQUILLA
66170	DOSQUEBRADAS
54001	CUCUTA
52356	IPIALES
50001	VILLAVICENCIO
20001	VALLEDUPAR
*/


SELECT
	DISTINCT
		TABLA.[Tipo Documento],
		TABLA.Documento,
		CodigoResponsable,
		case when CodigoResponsable = 'SDS001' THEN 4 ELSE 1 END TipoUsuaurio,
		COALESCE(pac.PrimerApellido,'') PrimerApellido,
		COALESCE(pac.SegundoApellido,'') SegundoApellido,
		COALESCE(pac.PrimerNombre,'') PrimerNombre,
		COALESCE(pac.SegundoNombre,'') SegundoNombre,
		pac.FechaNacimiento,
		[dbo].[Fun_ORI_CalculaEdad](pac.FechaNacimiento,Fecha) Edad,
		1 Unidad,
		substring(Genero,1,1) Sexo,
		substring(ciudad.Referencia1,1,2) Depertamento,
		substring(ciudad.Referencia1,3,3) Ciudad,
		'U' Zona,
		case when pais.Valor = 'Colombia' or TABLA.[Tipo Documento] in ('CC','TI') then 'Colombia'

		else pais.Valor end Nacionalidad,
		case when pais.Valor = 'Colombia' or TABLA.[Tipo Documento] in ('CC','TI') then 'Nacional'

		else 'Extranjera' end TipoNacionalidad
FROM (
SELECT   S.CodigoHabilitacion [Codigo Prestador],
		'FUNDACION ORIENTAME' [Razon Social],
		'NI' [TipoNit],
		'860049972' Nit,
		SUBSTRING(pac.TipoIdentificacion,1,2) [Tipo Documento],
		pac.Identificacion Documento,
		DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		CAST(FechaFactura AS DATE) Fecha,
		CAST(@desde AS DATE) FechaInicial,
		CAST(@hasta AS DATE) FechaDesde,
		CASE
			WHEN M.CodigoResponsable = '900156264' THEN 'EPS037'
			WHEN M.CodigoResponsable = '830003564' THEN 'EPS017'
			WHEN M.CodigoResponsable = '800088702' THEN 'EPS018'
			WHEN M.CodigoResponsable = '830002272' THEN 'EPS019'
			WHEN M.CodigoResponsable = '901127521' THEN 'EPS020'
			ELSE 'SDS001' END CodigoResponsable,

		CASE
			WHEN M.CodigoResponsable = '900156264' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '830003564' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '800088702' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '830002272' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '901127521' THEN A.NombreResponsable
			ELSE 'FUNDACION ORIENTAME' END NombreResponsable,
		'' [# Contrato],
		'IVE' [Plan Beneficios],
		'' [# Poliza],
		'' [Valor Copago],
		'' [Valor Descuento],
		((B.ValorFactura * B.IndicadorFacturable ) - B.ValorIvaFactura ) * A.EstadoActivo ValorFactura1
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('CONSULTA')
AND S.Codigo in (SELECT Codigo FROM GEN_CNF_Sedes WHERE CodigoTributarioCiudad = @codigo_ciudad)
and  NOT EXISTS (SELECT  *
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaId = A.FacturaId AND NOTAS.FechaFactura > @desde)
UNION ALL
select  [Codigo Prestador],
		'FUNDACION ORIENTAME' [Razon Social],
		'NI' [TipoNit],
		'860049972' Nit,
		[Tipo Documento],
		Documento,
		Factura,
		Fecha,
		CAST(@desde AS DATE) FechaInicial,
		CAST(@hasta AS DATE) FechaDesde,
				CASE

			WHEN CodigoResponsable = '900156264' THEN 'EPS037'
			WHEN CodigoResponsable = '830003564' THEN 'EPS017'
			WHEN CodigoResponsable = '800088702' THEN 'EPS018'
			WHEN CodigoResponsable = '830002272' THEN 'EPS019'
			WHEN CodigoResponsable = '901127521' THEN 'EPS020' ELSE 'SDS001' END CodigoResponsable, --	P6.Valor ,
				CASE

			WHEN CodigoResponsable = '900156264' THEN NombreResponsable
			WHEN CodigoResponsable = '830003564' THEN NombreResponsable
			WHEN CodigoResponsable = '800088702' THEN NombreResponsable
			WHEN CodigoResponsable = '830002272' THEN NombreResponsable
			WHEN CodigoResponsable = '901127521' THEN NombreResponsable ELSE 'FUNDACION ORIENTAME' END NombreResponsable	,
		'' [# Contrato],
		'IVE' [Plan Beneficios],
		'' [# Poliza],
		'' [Valor Copago],
		'' [Valor Descuento],
		ValorFactura1
from(
 SELECT	Row_number() over(partition by pac.Identificacion,ACT.ActividadId order by pac.Identificacion,A.Numero ASC) fila,
		DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		 S.CodigoHabilitacion [Codigo Prestador],
		SUBSTRING(pac.TipoIdentificacion,1,2) [Tipo Documento],
		pac.Identificacion Documento,
		CAST(FechaFactura AS DATE) Fecha,
		'' [# Autorizaci�n],
		ACT.CodigoNacional [Codigo CUP],
		1 Ambito,
		2 Finalidad,
		'' Personal,
		'' [DX Principal],
		'' [DX Relacionado 1],
		2  [DX Complicaciones],
		1 [Realizacion],
			M.CodigoResponsable, --	P6.Valor ,
		A.NombreResponsable	,
		 ((B.ValorFactura*B.IndicadorFacturable)-B.ValorIvaFactura)*A.EstadoActivo ValorFactura1,
		 ACT.Nombre
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('PROCEDIMIENTOS')
AND S.Codigo in (SELECT Codigo FROM GEN_CNF_Sedes WHERE CodigoTributarioCiudad = @codigo_ciudad)
and  NOT EXISTS (SELECT  NOTAS.FacturaId,NOtas.Numero,  B.FacturaAfectadaId
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					LEFT JOIN FAC_MOV_FacturasAnexos B WITH (NOLOCK) ON NOTAS.FacturaId = B.FacturaEncabezadoId
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaAfectadaId = A.FacturaId  AND NOTAS.FechaFactura> @desde )
				) tabla
					where fila in (1,2)
	)	TABLA
	INNER JOIN ADT_VIS_Pacientes pac with (nolock) ON pac.Identificacion = TABLA.Documento and SUBSTRING(pac.TipoIdentificacion,1,2) = TABLA.[Tipo Documento]
	left join GEN_CNF_Parametros ciudad on ciudad.ParametroId = MunicipioResidenciaId
	left join GEN_CNF_Parametros nacimiento on nacimiento.ParametroId = LugarNacimientoId
	left join GEN_CNF_Parametros departamento on departamento.ParametroId = nacimiento.PadreId
	left join GEN_CNF_Parametros pais on pais.ParametroId = departamento.PadreId
'''

# Information converted into a DataFrame with pandas
df = pd.read_sql(sql=sqlQuery, con=connection)

# Export the data on the excel into a Desktop
df.to_csv(os.environ["userprofile"] + "\\Documents\\Rips\\Sedes\\Ipiales\\" + "Ipiales_Usuarios " +
            datetime.now().strftime("%d-%b-%Y %H%M%S") + ".csv", index=False)

#Display notification to user
notification.notify(title="✅Report Status!!!",
                    message= f"Queries of Ipiales has been succesfully saved into .csv\
                        \n Queries Executed:  \n 1. Consultas, 2.Facturas, 3.Procedimientos, 4.Usuarios",
                        timeout=10)

C:\Users\ddrodriguez\AppData\Local\Temp\ipykernel_19832\2555543433.py:234: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=sqlQuery, con=connection)


##
 <font color='MediumVioletRed' size="6"> VILLAVICENCIO </font>

###
 <font color='MediumVioletRed' size="5"> Consultas Villavicencio </font>

In [ ]:
# SQL command to read the data
sqlQuery = '''

--CONSULTAS VILLAVICENCIO

DECLARE @hoy DATETIME = GETDATE()
DECLARE @hasta DATETIME = EOMONTH(@hoy, -1);
DECLARE @desde DATETIME =  DATETRUNC(month,@hasta),
		@codigo_ciudad VARCHAR(5) = '50001'
/*
05001	MEDELLIN
08001	BARRANQUILLA
66170	DOSQUEBRADAS
54001	CUCUTA
52356	IPIALES
50001	VILLAVICENCIO
20001	VALLEDUPAR
*/

SELECT	DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		S.CodigoHabilitacion [Codigo Prestador],
		SUBSTRING(pac.TipoIdentificacion,1,2) [Tipo Documento],
		pac.Identificacion Documento,
		CAST(FechaFactura AS DATE) Fecha,
		C.NumeroAutorizacion [# Autorización],
		ACT.CodigoNacional [Codigo CUP],
		10 Finalidad,
		13 Causa,
		'' [DX Principal],
		'' [DX Relacionado 1],
		'' [DX Relacionado 2],
		'' [DX Relacionado 3],
		2  [Tipo DX],
		 ((B.ValorFactura*B.IndicadorFacturable)-B.ValorIvaFactura)*A.EstadoActivo ValorFactura1,
		'' ValorCopago,
		 ((B.ValorFactura*B.IndicadorFacturable)-B.ValorIvaFactura)*A.EstadoActivo ValorFactura,
		 ACT.Nombre
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('CONSULTA')
AND S.Codigo in (SELECT Codigo FROM GEN_CNF_Sedes WHERE CodigoTributarioCiudad = @codigo_ciudad)
and  NOT EXISTS (SELECT  *
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaId = A.FacturaId AND NOTAS.FechaFactura > @desde)
'''

# Information converted into a DataFrame with pandas
df = pd.read_sql(sql=sqlQuery, con=connection)

# Export the data on the excel into a Desktop
df.to_csv(os.environ["userprofile"] + "\\Documents\\Rips\\Sedes\\Villavicencio\\" + "Villavicencio_Consultas " +
            datetime.now().strftime("%d-%b-%Y %H%M%S") + ".csv", index=False)

C:\Users\ddrodriguez\AppData\Local\Temp\ipykernel_19832\47845849.py:89: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=sqlQuery, con=connection)


###
 <font color='MediumVioletRed' size="5"> Facturas Villavicencio </font>

In [ ]:
# SQL command to read the data
sqlQuery = '''

--FACTURAS VILLAVICENCIO

DECLARE @hoy DATETIME = GETDATE()
DECLARE @hasta DATETIME = EOMONTH(@hoy, -1);
DECLARE @desde DATETIME =  DATETRUNC(month,@hasta),
		@codigo_ciudad VARCHAR(5) = '50001'
/*
05001	MEDELLIN
08001	BARRANQUILLA
66170	DOSQUEBRADAS
54001	CUCUTA
52356	IPIALES
50001	VILLAVICENCIO
20001	VALLEDUPAR
*/


SELECT
		[Codigo Prestador],
		[Razon Social],
		[Tipo Identificacion],
		[Identificacion],
		Factura,
		Fecha,
		FechaInicial,
		FechaDesde,
		CodigoResponsable,
		NombreResponsable,
		[# Contrato],
		[Plan Beneficios],
		[# Poliza],
		[Valor Copago],
		[Valor Descuento],
		sum(ValorFactura1) Valor
FROM (
SELECT  S.CodigoHabilitacion [Codigo Prestador],
		'FUNDACION ORIENTAME' [Razon Social],
		'NI' [Tipo Identificacion],
		'860049972' [Identificacion],
		DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		CAST(FechaFactura AS DATE) Fecha,
		CAST(@desde AS DATE) FechaInicial,
		CAST(@hasta AS DATE) FechaDesde,
		CASE
			WHEN M.CodigoResponsable = '900156264' THEN 'EPS037'
			WHEN M.CodigoResponsable = '830003564' THEN 'EPS017'
			WHEN M.CodigoResponsable = '800088702' THEN 'EPS018'
			WHEN M.CodigoResponsable = '830002272' THEN 'EPS019'
			WHEN M.CodigoResponsable = '901127521' THEN 'EPS020'
			ELSE 'SDS001' END CodigoResponsable,

		CASE
			WHEN M.CodigoResponsable = '900156264' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '830003564' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '800088702' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '830002272' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '901127521' THEN A.NombreResponsable
			ELSE 'FUNDACION ORIENTAME' END NombreResponsable,
		'' [# Contrato],
		'IVE' [Plan Beneficios],
		'' [# Poliza],
		'' [Valor Copago],
		'' [Valor Descuento],
		((B.ValorFactura * B.IndicadorFacturable ) - B.ValorIvaFactura ) * A.EstadoActivo ValorFactura1
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('CONSULTA')
AND S.Codigo in (SELECT Codigo FROM GEN_CNF_Sedes WHERE CodigoTributarioCiudad = @codigo_ciudad)
and  NOT EXISTS (SELECT  *
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaId = A.FacturaId AND NOTAS.FechaFactura > @desde)
UNION ALL
select   [Codigo Prestador],
		'FUNDACION ORIENTAME' [Razon Social],
		'NI' [Tipo Identificacion],
		'860049972' [Identificacion],
		Factura,
		Fecha,
		CAST(@desde AS DATE) FechaInicial,
		CAST(@hasta AS DATE) FechaDesde,
				CASE

			WHEN CodigoResponsable = '900156264' THEN 'EPS037'
			WHEN CodigoResponsable = '830003564' THEN 'EPS017'
			WHEN CodigoResponsable = '800088702' THEN 'EPS018'
			WHEN CodigoResponsable = '830002272' THEN 'EPS019'
			WHEN CodigoResponsable = '901127521' THEN 'EPS020' ELSE 'SDS001' END CodigoResponsable, --	P6.Valor ,
				CASE

			WHEN CodigoResponsable = '900156264' THEN NombreResponsable
			WHEN CodigoResponsable = '830003564' THEN NombreResponsable
			WHEN CodigoResponsable = '800088702' THEN NombreResponsable
			WHEN CodigoResponsable = '830002272' THEN NombreResponsable
			WHEN CodigoResponsable = '901127521' THEN NombreResponsable ELSE 'FUNDACION ORIENTAME' END NombreResponsable	,
		'' [# Contrato],
		'IVE' [Plan Beneficios],
		'' [# Poliza],
		'' [Valor Copago],
		'' [Valor Descuento],
		ValorFactura1
from(
 SELECT	Row_number() over(partition by pac.Identificacion,ACT.ActividadId order by pac.Identificacion,A.Numero ASC) fila,
		DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		S.CodigoHabilitacion [Codigo Prestador],
		SUBSTRING(pac.TipoIdentificacion,1,2) [Tipo Documento],
		pac.Identificacion Documento,
		CAST(FechaFactura AS DATE) Fecha,
		'' [# Autorización],
		ACT.CodigoNacional [Codigo CUP],
		1 Ambito,
		2 Finalidad,
		'' Personal,
		'' [DX Principal],
		'' [DX Relacionado 1],
		2  [DX Complicaciones],
		1 [Realizacion],
		M.CodigoResponsable,
		A.NombreResponsable	,
		 ((B.ValorFactura*B.IndicadorFacturable)-B.ValorIvaFactura)*A.EstadoActivo ValorFactura1,
		 ACT.Nombre
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('PROCEDIMIENTOS')
AND S.Codigo in (SELECT Codigo FROM GEN_CNF_Sedes WHERE CodigoTributarioCiudad = @codigo_ciudad)
and  NOT EXISTS (SELECT  NOTAS.FacturaId,NOtas.Numero,  B.FacturaAfectadaId
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					LEFT JOIN FAC_MOV_FacturasAnexos B WITH (NOLOCK) ON NOTAS.FacturaId = B.FacturaEncabezadoId
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaAfectadaId = A.FacturaId  AND NOTAS.FechaFactura> @desde )
				) tabla
				where fila in (1,2)
	)	TABLA
group by
		[Codigo Prestador],
		[Razon Social],
		[Tipo Identificacion],
		[Identificacion],
		Factura,
		Fecha,
		FechaInicial,
		FechaDesde,
		CodigoResponsable,
		NombreResponsable,
		[# Contrato],
		[Plan Beneficios],
		[# Poliza],
		[Valor Copago],
		[Valor Descuento]
'''

# Information converted into a DataFrame with pandas
df = pd.read_sql(sql=sqlQuery, con=connection)

# Export the data on the excel into a Desktop
df.to_csv(os.environ["userprofile"] + "\\Documents\\Rips\\Sedes\\Villavicencio\\" + "Villavicencio_Facturas " +
            datetime.now().strftime("%d-%b-%Y %H%M%S") + ".csv", index=False)

C:\Users\ddrodriguez\AppData\Local\Temp\ipykernel_19832\3315828275.py:236: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=sqlQuery, con=connection)


###
 <font color='MediumVioletRed' size="5"> Procedimientos Villavicencio </font>

In [ ]:
# SQL command to read the data
sqlQuery = '''

--PROCEDIMIENTOS VILLAVICENCIO

DECLARE @hoy DATETIME = GETDATE()
DECLARE @hasta DATETIME = EOMONTH(@hoy, -1);
DECLARE @desde DATETIME =  DATETRUNC(month,@hasta),
		@codigo_ciudad VARCHAR(5) = '50001'
/*
05001	MEDELLIN
08001	BARRANQUILLA
66170	DOSQUEBRADAS
54001	CUCUTA
52356	IPIALES
50001	VILLAVICENCIO
20001	VALLEDUPAR
*/


select *
from(
SELECT	Row_number() over(partition by pac.Identificacion,ACT.ActividadId order by pac.Identificacion,FechaFactura) fila,
		DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		 S.CodigoHabilitacion [Codigo Prestador],
		SUBSTRING(pac.TipoIdentificacion,1,2) [Tipo Documento],
		pac.Identificacion Documento,
		CAST(FechaFactura AS DATE) Fecha,
		C.NumeroAutorizacion [# Autorizaci�n],
		ACT.CodigoNacional [Codigo CUP],
		1 Ambito,
		2 Finalidad,
		'' Personal,
		'' [DX Principal],
		'' [DX Relacionado 1],
		2  [DX Complicaciones],
		1 [Realizacion],
		 ((B.ValorFactura*B.IndicadorFacturable)-B.ValorIvaFactura)*A.EstadoActivo ValorFactura1,
		 ACT.Nombre
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('PROCEDIMIENTOS')
AND S.Codigo in (SELECT Codigo FROM GEN_CNF_Sedes WHERE CodigoTributarioCiudad = @codigo_ciudad)
and  NOT EXISTS (SELECT  NOTAS.FacturaId,NOtas.Numero,  B.FacturaAfectadaId
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					LEFT JOIN FAC_MOV_FacturasAnexos B WITH (NOLOCK) ON NOTAS.FacturaId = B.FacturaEncabezadoId
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaAfectadaId = A.FacturaId  AND NOTAS.FechaFactura> @desde )
				) tabla
				where fila IN (1,2)
'''

# Information converted into a DataFrame with pandas
df = pd.read_sql(sql=sqlQuery, con=connection)

# Export the data on the excel into a Desktop
df.to_csv(os.environ["userprofile"] + "\\Documents\\Rips\\Sedes\\Villavicencio\\" + "Villavicencio_Procedimientos " +
            datetime.now().strftime("%d-%b-%Y %H%M%S") + ".csv", index=False)

C:\Users\ddrodriguez\AppData\Local\Temp\ipykernel_19832\1437342548.py:94: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=sqlQuery, con=connection)


###
 <font color='MediumVioletRed' size="5"> Usuarios Villavicencio </font>

In [ ]:
# SQL command to read the data
sqlQuery = '''

-- USUARIOS VILLAVICENCIO

DECLARE @hoy DATETIME = GETDATE()
DECLARE @hasta DATETIME = EOMONTH(@hoy, -1);
DECLARE @desde DATETIME =  DATETRUNC(month,@hasta),
		@codigo_ciudad VARCHAR(5) = '50001'
/*
05001	MEDELLIN
08001	BARRANQUILLA
66170	DOSQUEBRADAS
54001	CUCUTA
52356	IPIALES
50001	VILLAVICENCIO
20001	VALLEDUPAR
*/


SELECT
	DISTINCT
		TABLA.[Tipo Documento],
		TABLA.Documento,
		CodigoResponsable,
		case when CodigoResponsable = 'SDS001' THEN 4 ELSE 1 END TipoUsuaurio,
		COALESCE(pac.PrimerApellido,'') PrimerApellido,
		COALESCE(pac.SegundoApellido,'') SegundoApellido,
		COALESCE(pac.PrimerNombre,'') PrimerNombre,
		COALESCE(pac.SegundoNombre,'') SegundoNombre,
		pac.FechaNacimiento,
		[dbo].[Fun_ORI_CalculaEdad](pac.FechaNacimiento,Fecha) Edad,
		1 Unidad,
		substring(Genero,1,1) Sexo,
		substring(ciudad.Referencia1,1,2) Depertamento,
		substring(ciudad.Referencia1,3,3) Ciudad,
		'U' Zona,
		case when pais.Valor = 'Colombia' or TABLA.[Tipo Documento] in ('CC','TI') then 'Colombia'

		else pais.Valor end Nacionalidad,
		case when pais.Valor = 'Colombia' or TABLA.[Tipo Documento] in ('CC','TI') then 'Nacional'

		else 'Extranjera' end TipoNacionalidad
FROM (
SELECT   S.CodigoHabilitacion [Codigo Prestador],
		'FUNDACION ORIENTAME' [Razon Social],
		'NI' [TipoNit],
		'860049972' Nit,
		SUBSTRING(pac.TipoIdentificacion,1,2) [Tipo Documento],
		pac.Identificacion Documento,
		DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		CAST(FechaFactura AS DATE) Fecha,
		CAST(@desde AS DATE) FechaInicial,
		CAST(@hasta AS DATE) FechaDesde,
		CASE
			WHEN M.CodigoResponsable = '900156264' THEN 'EPS037'
			WHEN M.CodigoResponsable = '830003564' THEN 'EPS017'
			WHEN M.CodigoResponsable = '800088702' THEN 'EPS018'
			WHEN M.CodigoResponsable = '830002272' THEN 'EPS019'
			WHEN M.CodigoResponsable = '901127521' THEN 'EPS020'
			ELSE 'SDS001' END CodigoResponsable,

		CASE
			WHEN M.CodigoResponsable = '900156264' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '830003564' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '800088702' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '830002272' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '901127521' THEN A.NombreResponsable
			ELSE 'FUNDACION ORIENTAME' END NombreResponsable,
		'' [# Contrato],
		'IVE' [Plan Beneficios],
		'' [# Poliza],
		'' [Valor Copago],
		'' [Valor Descuento],
		((B.ValorFactura * B.IndicadorFacturable ) - B.ValorIvaFactura ) * A.EstadoActivo ValorFactura1
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('CONSULTA')
AND S.Codigo in (SELECT Codigo FROM GEN_CNF_Sedes WHERE CodigoTributarioCiudad = @codigo_ciudad)
and  NOT EXISTS (SELECT  *
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaId = A.FacturaId AND NOTAS.FechaFactura > @desde)
UNION ALL
select  [Codigo Prestador],
		'FUNDACION ORIENTAME' [Razon Social],
		'NI' [TipoNit],
		'860049972' Nit,
		[Tipo Documento],
		Documento,
		Factura,
		Fecha,
		CAST(@desde AS DATE) FechaInicial,
		CAST(@hasta AS DATE) FechaDesde,
				CASE

			WHEN CodigoResponsable = '900156264' THEN 'EPS037'
			WHEN CodigoResponsable = '830003564' THEN 'EPS017'
			WHEN CodigoResponsable = '800088702' THEN 'EPS018'
			WHEN CodigoResponsable = '830002272' THEN 'EPS019'
			WHEN CodigoResponsable = '901127521' THEN 'EPS020' ELSE 'SDS001' END CodigoResponsable, --	P6.Valor ,
				CASE

			WHEN CodigoResponsable = '900156264' THEN NombreResponsable
			WHEN CodigoResponsable = '830003564' THEN NombreResponsable
			WHEN CodigoResponsable = '800088702' THEN NombreResponsable
			WHEN CodigoResponsable = '830002272' THEN NombreResponsable
			WHEN CodigoResponsable = '901127521' THEN NombreResponsable ELSE 'FUNDACION ORIENTAME' END NombreResponsable	,
		'' [# Contrato],
		'IVE' [Plan Beneficios],
		'' [# Poliza],
		'' [Valor Copago],
		'' [Valor Descuento],
		ValorFactura1
from(
 SELECT	Row_number() over(partition by pac.Identificacion,ACT.ActividadId order by pac.Identificacion,A.Numero ASC) fila,
		DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		 S.CodigoHabilitacion [Codigo Prestador],
		SUBSTRING(pac.TipoIdentificacion,1,2) [Tipo Documento],
		pac.Identificacion Documento,
		CAST(FechaFactura AS DATE) Fecha,
		'' [# Autorizaci�n],
		ACT.CodigoNacional [Codigo CUP],
		1 Ambito,
		2 Finalidad,
		'' Personal,
		'' [DX Principal],
		'' [DX Relacionado 1],
		2  [DX Complicaciones],
		1 [Realizacion],
			M.CodigoResponsable, --	P6.Valor ,
		A.NombreResponsable	,
		 ((B.ValorFactura*B.IndicadorFacturable)-B.ValorIvaFactura)*A.EstadoActivo ValorFactura1,
		 ACT.Nombre
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('PROCEDIMIENTOS')
AND S.Codigo in (SELECT Codigo FROM GEN_CNF_Sedes WHERE CodigoTributarioCiudad = @codigo_ciudad)
and  NOT EXISTS (SELECT  NOTAS.FacturaId,NOtas.Numero,  B.FacturaAfectadaId
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					LEFT JOIN FAC_MOV_FacturasAnexos B WITH (NOLOCK) ON NOTAS.FacturaId = B.FacturaEncabezadoId
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaAfectadaId = A.FacturaId  AND NOTAS.FechaFactura> @desde )
				) tabla
					where fila in (1,2)
	)	TABLA
	INNER JOIN ADT_VIS_Pacientes pac with (nolock) ON pac.Identificacion = TABLA.Documento and SUBSTRING(pac.TipoIdentificacion,1,2) = TABLA.[Tipo Documento]
	left join GEN_CNF_Parametros ciudad on ciudad.ParametroId = MunicipioResidenciaId
	left join GEN_CNF_Parametros nacimiento on nacimiento.ParametroId = LugarNacimientoId
	left join GEN_CNF_Parametros departamento on departamento.ParametroId = nacimiento.PadreId
	left join GEN_CNF_Parametros pais on pais.ParametroId = departamento.PadreId
'''

# Information converted into a DataFrame with pandas
df = pd.read_sql(sql=sqlQuery, con=connection)

# Export the data on the excel into a Desktop
df.to_csv(os.environ["userprofile"] + "\\Documents\\Rips\\Sedes\\Villavicencio\\" + "Villavicencio_Usuarios " +
            datetime.now().strftime("%d-%b-%Y %H%M%S") + ".csv", index=False)

#Display notification to user
notification.notify(title="✅Report Status!!!",
                    message= f"Queries of Villavicencio has been succesfully saved into .csv\
                        \n Queries Executed:  \n 1. Consultas, 2.Facturas, 3.Procedimientos, 4.Usuarios",
                        timeout=10)

C:\Users\ddrodriguez\AppData\Local\Temp\ipykernel_19832\615278922.py:235: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=sqlQuery, con=connection)


##
 <font color='Cyan' size="6"> VALLEDUPAR </font>

###
 <font color='Cyan' size="5"> Consultas Valledupar </font>

In [ ]:
# SQL command to read the data
sqlQuery = '''
--CONSULTAS

DECLARE @hoy DATETIME = GETDATE()
DECLARE @hasta DATETIME = EOMONTH(@hoy, -1);
DECLARE @desde DATETIME =  DATETRUNC(month,@hasta),
		@codigo_ciudad VARCHAR(5) = '20001'
/*
05001	MEDELLIN
08001	BARRANQUILLA
66170	DOSQUEBRADAS
54001	CUCUTA
52356	IPIALES
50001	VILLAVICENCIO
20001	VALLEDUPAR
*/

SELECT	DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		S.CodigoHabilitacion [Codigo Prestador],
		SUBSTRING(pac.TipoIdentificacion,1,2) [Tipo Documento],
		pac.Identificacion Documento,
		CAST(FechaFactura AS DATE) Fecha,
		C.NumeroAutorizacion [# Autorización],
		ACT.CodigoNacional [Codigo CUP],
		10 Finalidad,
		13 Causa,
		'' [DX Principal],
		'' [DX Relacionado 1],
		'' [DX Relacionado 2],
		'' [DX Relacionado 3],
		2  [Tipo DX],
		 ((B.ValorFactura*B.IndicadorFacturable)-B.ValorIvaFactura)*A.EstadoActivo ValorFactura1,
		'' ValorCopago,
		 ((B.ValorFactura*B.IndicadorFacturable)-B.ValorIvaFactura)*A.EstadoActivo ValorFactura,
		 ACT.Nombre
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('CONSULTA')
AND S.Codigo in (SELECT Codigo FROM GEN_CNF_Sedes WHERE CodigoTributarioCiudad = @codigo_ciudad)
and  NOT EXISTS (SELECT  *
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaId = A.FacturaId AND NOTAS.FechaFactura > @desde)
'''

# Information converted into a DataFrame with pandas
df = pd.read_sql(sql=sqlQuery, con=connection)

# Export the data on the excel into a Desktop
df.to_csv(os.environ["userprofile"] + "\\Documents\\Rips\\Sedes\\Valledupar\\" + "Valledupar_Consultas " +
            datetime.now().strftime("%d-%b-%Y %H%M%S") + ".csv", index=False)

C:\Users\ddrodriguez\AppData\Local\Temp\ipykernel_19832\3261926668.py:88: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=sqlQuery, con=connection)


###
 <font color='Cyan' size="5"> Facturas Valledupar </font>

In [ ]:
# SQL command to read the data
sqlQuery = '''

-- FACTURAS VALLEDUPAR

DECLARE @hoy DATETIME = GETDATE()
DECLARE @hasta DATETIME = EOMONTH(@hoy, -1);
DECLARE @desde DATETIME =  DATETRUNC(month,@hasta),
		@codigo_ciudad VARCHAR(5) = '20001'
/*
05001	MEDELLIN
08001	BARRANQUILLA
66170	DOSQUEBRADAS
54001	CUCUTA
52356	IPIALES
50001	VILLAVICENCIO
20001	VALLEDUPAR
*/


SELECT
		[Codigo Prestador],
		[Razon Social],
		[Tipo Identificacion],
		[Identificacion],
		Factura,
		Fecha,
		FechaInicial,
		FechaDesde,
		CodigoResponsable,
		NombreResponsable,
		[# Contrato],
		[Plan Beneficios],
		[# Poliza],
		[Valor Copago],
		[Valor Descuento],
		sum(ValorFactura1) Valor
FROM (
SELECT  S.CodigoHabilitacion [Codigo Prestador],
		'FUNDACION ORIENTAME' [Razon Social],
		'NI' [Tipo Identificacion],
		'860049972' [Identificacion],
		DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		CAST(FechaFactura AS DATE) Fecha,
		CAST(@desde AS DATE) FechaInicial,
		CAST(@hasta AS DATE) FechaDesde,
		CASE
			WHEN M.CodigoResponsable = '900156264' THEN 'EPS037'
			WHEN M.CodigoResponsable = '830003564' THEN 'EPS017'
			WHEN M.CodigoResponsable = '800088702' THEN 'EPS018'
			WHEN M.CodigoResponsable = '830002272' THEN 'EPS019'
			WHEN M.CodigoResponsable = '901127521' THEN 'EPS020'
			ELSE 'SDS001' END CodigoResponsable,

		CASE
			WHEN M.CodigoResponsable = '900156264' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '830003564' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '800088702' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '830002272' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '901127521' THEN A.NombreResponsable
			ELSE 'FUNDACION ORIENTAME' END NombreResponsable,
		'' [# Contrato],
		'IVE' [Plan Beneficios],
		'' [# Poliza],
		'' [Valor Copago],
		'' [Valor Descuento],
		((B.ValorFactura * B.IndicadorFacturable ) - B.ValorIvaFactura ) * A.EstadoActivo ValorFactura1
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('CONSULTA')
AND S.Codigo in (SELECT Codigo FROM GEN_CNF_Sedes WHERE CodigoTributarioCiudad = @codigo_ciudad)
and  NOT EXISTS (SELECT  *
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaId = A.FacturaId AND NOTAS.FechaFactura > @desde)
UNION ALL
select   [Codigo Prestador],
		'FUNDACION ORIENTAME' [Razon Social],
		'NI' [Tipo Identificacion],
		'860049972' [Identificacion],
		Factura,
		Fecha,
		CAST(@desde AS DATE) FechaInicial,
		CAST(@hasta AS DATE) FechaDesde,
				CASE

			WHEN CodigoResponsable = '900156264' THEN 'EPS037'
			WHEN CodigoResponsable = '830003564' THEN 'EPS017'
			WHEN CodigoResponsable = '800088702' THEN 'EPS018'
			WHEN CodigoResponsable = '830002272' THEN 'EPS019'
			WHEN CodigoResponsable = '901127521' THEN 'EPS020' ELSE 'SDS001' END CodigoResponsable, --	P6.Valor ,
				CASE

			WHEN CodigoResponsable = '900156264' THEN NombreResponsable
			WHEN CodigoResponsable = '830003564' THEN NombreResponsable
			WHEN CodigoResponsable = '800088702' THEN NombreResponsable
			WHEN CodigoResponsable = '830002272' THEN NombreResponsable
			WHEN CodigoResponsable = '901127521' THEN NombreResponsable ELSE 'FUNDACION ORIENTAME' END NombreResponsable	,
		'' [# Contrato],
		'IVE' [Plan Beneficios],
		'' [# Poliza],
		'' [Valor Copago],
		'' [Valor Descuento],
		ValorFactura1
from(
 SELECT	Row_number() over(partition by pac.Identificacion,ACT.ActividadId order by pac.Identificacion,A.Numero ASC) fila,
		DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		S.CodigoHabilitacion [Codigo Prestador],
		SUBSTRING(pac.TipoIdentificacion,1,2) [Tipo Documento],
		pac.Identificacion Documento,
		CAST(FechaFactura AS DATE) Fecha,
		'' [# Autorización],
		ACT.CodigoNacional [Codigo CUP],
		1 Ambito,
		2 Finalidad,
		'' Personal,
		'' [DX Principal],
		'' [DX Relacionado 1],
		2  [DX Complicaciones],
		1 [Realizacion],
		M.CodigoResponsable,
		A.NombreResponsable	,
		 ((B.ValorFactura*B.IndicadorFacturable)-B.ValorIvaFactura)*A.EstadoActivo ValorFactura1,
		 ACT.Nombre
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('PROCEDIMIENTOS')
AND S.Codigo in (SELECT Codigo FROM GEN_CNF_Sedes WHERE CodigoTributarioCiudad = @codigo_ciudad)
and  NOT EXISTS (SELECT  NOTAS.FacturaId,NOtas.Numero,  B.FacturaAfectadaId
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					LEFT JOIN FAC_MOV_FacturasAnexos B WITH (NOLOCK) ON NOTAS.FacturaId = B.FacturaEncabezadoId
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaAfectadaId = A.FacturaId  AND NOTAS.FechaFactura> @desde )
				) tabla
				where fila in (1,2)
	)	TABLA
group by
		[Codigo Prestador],
		[Razon Social],
		[Tipo Identificacion],
		[Identificacion],
		Factura,
		Fecha,
		FechaInicial,
		FechaDesde,
		CodigoResponsable,
		NombreResponsable,
		[# Contrato],
		[Plan Beneficios],
		[# Poliza],
		[Valor Copago],
		[Valor Descuento]
'''

# Information converted into a DataFrame with pandas
df = pd.read_sql(sql=sqlQuery, con=connection)

# Export the data on the excel into a Desktop
df.to_csv(os.environ["userprofile"] + "\\Documents\\Rips\\Sedes\\Valledupar\\" + "Valledupar_Facturas " +
            datetime.now().strftime("%d-%b-%Y %H%M%S") + ".csv", index=False)

C:\Users\ddrodriguez\AppData\Local\Temp\ipykernel_19832\3682532658.py:236: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=sqlQuery, con=connection)


###
 <font color='Cyan' size="5"> Procedimientos Valledupar </font>

In [ ]:
# SQL command to read the data
sqlQuery = '''

--PROCEDIMIENTOS VALLEDUPAR

DECLARE @hoy DATETIME = GETDATE()
DECLARE @hasta DATETIME = EOMONTH(@hoy, -1);
DECLARE @desde DATETIME =  DATETRUNC(month,@hasta),
		@codigo_ciudad VARCHAR(5) = '20001'
/*
05001	MEDELLIN
08001	BARRANQUILLA
66170	DOSQUEBRADAS
54001	CUCUTA
52356	IPIALES
50001	VILLAVICENCIO
20001	VALLEDUPAR
*/


select *
from(
SELECT	Row_number() over(partition by pac.Identificacion,ACT.ActividadId order by pac.Identificacion,FechaFactura) fila,
		DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		 S.CodigoHabilitacion [Codigo Prestador],
		SUBSTRING(pac.TipoIdentificacion,1,2) [Tipo Documento],
		pac.Identificacion Documento,
		CAST(FechaFactura AS DATE) Fecha,
		C.NumeroAutorizacion [# Autorizaci�n],
		ACT.CodigoNacional [Codigo CUP],
		1 Ambito,
		2 Finalidad,
		'' Personal,
		'' [DX Principal],
		'' [DX Relacionado 1],
		2  [DX Complicaciones],
		1 [Realizacion],
		 ((B.ValorFactura*B.IndicadorFacturable)-B.ValorIvaFactura)*A.EstadoActivo ValorFactura1,
		 ACT.Nombre
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('PROCEDIMIENTOS')
AND S.Codigo in (SELECT Codigo FROM GEN_CNF_Sedes WHERE CodigoTributarioCiudad = @codigo_ciudad)
and  NOT EXISTS (SELECT  NOTAS.FacturaId,NOtas.Numero,  B.FacturaAfectadaId
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					LEFT JOIN FAC_MOV_FacturasAnexos B WITH (NOLOCK) ON NOTAS.FacturaId = B.FacturaEncabezadoId
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaAfectadaId = A.FacturaId  AND NOTAS.FechaFactura> @desde )
				) tabla
				where fila IN (1,2)
'''

# Information converted into a DataFrame with pandas
df = pd.read_sql(sql=sqlQuery, con=connection)

# Export the data on the excel into a Desktop
df.to_csv(os.environ["userprofile"] + "\\Documents\\Rips\\Sedes\\Valledupar\\" + "Valledupar_Procedimientos " +
            datetime.now().strftime("%d-%b-%Y %H%M%S") + ".csv", index=False)

C:\Users\ddrodriguez\AppData\Local\Temp\ipykernel_19832\4190811789.py:94: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=sqlQuery, con=connection)


###
 <font color='Cyan' size="5"> Usuarios Valledupar </font>

In [ ]:
# SQL command to read the data
sqlQuery = '''

--USUARIOS VALLEDUPAR

DECLARE @hoy DATETIME = GETDATE()
DECLARE @hasta DATETIME = EOMONTH(@hoy, -1);
DECLARE @desde DATETIME =  DATETRUNC(month,@hasta),
		@codigo_ciudad VARCHAR(5) = '20001'
/*
05001	MEDELLIN
08001	BARRANQUILLA
66170	DOSQUEBRADAS
54001	CUCUTA
52356	IPIALES
50001	VILLAVICENCIO
20001	VALLEDUPAR
*/


SELECT
	DISTINCT
		TABLA.[Tipo Documento],
		TABLA.Documento,
		CodigoResponsable,
		case when CodigoResponsable = 'SDS001' THEN 4 ELSE 1 END TipoUsuaurio,
		COALESCE(pac.PrimerApellido,'') PrimerApellido,
		COALESCE(pac.SegundoApellido,'') SegundoApellido,
		COALESCE(pac.PrimerNombre,'') PrimerNombre,
		COALESCE(pac.SegundoNombre,'') SegundoNombre,
		pac.FechaNacimiento,
		[dbo].[Fun_ORI_CalculaEdad](pac.FechaNacimiento,Fecha) Edad,
		1 Unidad,
		substring(Genero,1,1) Sexo,
		substring(ciudad.Referencia1,1,2) Depertamento,
		substring(ciudad.Referencia1,3,3) Ciudad,
		'U' Zona,
		case when pais.Valor = 'Colombia' or TABLA.[Tipo Documento] in ('CC','TI') then 'Colombia'

		else pais.Valor end Nacionalidad,
		case when pais.Valor = 'Colombia' or TABLA.[Tipo Documento] in ('CC','TI') then 'Nacional'

		else 'Extranjera' end TipoNacionalidad
FROM (
SELECT   S.CodigoHabilitacion [Codigo Prestador],
		'FUNDACION ORIENTAME' [Razon Social],
		'NI' [TipoNit],
		'860049972' Nit,
		SUBSTRING(pac.TipoIdentificacion,1,2) [Tipo Documento],
		pac.Identificacion Documento,
		DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		CAST(FechaFactura AS DATE) Fecha,
		CAST(@desde AS DATE) FechaInicial,
		CAST(@hasta AS DATE) FechaDesde,
		CASE
			WHEN M.CodigoResponsable = '900156264' THEN 'EPS037'
			WHEN M.CodigoResponsable = '830003564' THEN 'EPS017'
			WHEN M.CodigoResponsable = '800088702' THEN 'EPS018'
			WHEN M.CodigoResponsable = '830002272' THEN 'EPS019'
			WHEN M.CodigoResponsable = '901127521' THEN 'EPS020'
			ELSE 'SDS001' END CodigoResponsable,

		CASE
			WHEN M.CodigoResponsable = '900156264' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '830003564' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '800088702' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '830002272' THEN A.NombreResponsable
			WHEN M.CodigoResponsable = '901127521' THEN A.NombreResponsable
			ELSE 'FUNDACION ORIENTAME' END NombreResponsable,
		'' [# Contrato],
		'IVE' [Plan Beneficios],
		'' [# Poliza],
		'' [Valor Copago],
		'' [Valor Descuento],
		((B.ValorFactura * B.IndicadorFacturable ) - B.ValorIvaFactura ) * A.EstadoActivo ValorFactura1
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('CONSULTA')
AND S.Codigo in (SELECT Codigo FROM GEN_CNF_Sedes WHERE CodigoTributarioCiudad = @codigo_ciudad)
and  NOT EXISTS (SELECT  *
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaId = A.FacturaId AND NOTAS.FechaFactura > @desde)
UNION ALL
select  [Codigo Prestador],
		'FUNDACION ORIENTAME' [Razon Social],
		'NI' [TipoNit],
		'860049972' Nit,
		[Tipo Documento],
		Documento,
		Factura,
		Fecha,
		CAST(@desde AS DATE) FechaInicial,
		CAST(@hasta AS DATE) FechaDesde,
				CASE

			WHEN CodigoResponsable = '900156264' THEN 'EPS037'
			WHEN CodigoResponsable = '830003564' THEN 'EPS017'
			WHEN CodigoResponsable = '800088702' THEN 'EPS018'
			WHEN CodigoResponsable = '830002272' THEN 'EPS019'
			WHEN CodigoResponsable = '901127521' THEN 'EPS020' ELSE 'SDS001' END CodigoResponsable, --	P6.Valor ,
				CASE

			WHEN CodigoResponsable = '900156264' THEN NombreResponsable
			WHEN CodigoResponsable = '830003564' THEN NombreResponsable
			WHEN CodigoResponsable = '800088702' THEN NombreResponsable
			WHEN CodigoResponsable = '830002272' THEN NombreResponsable
			WHEN CodigoResponsable = '901127521' THEN NombreResponsable ELSE 'FUNDACION ORIENTAME' END NombreResponsable	,
		'' [# Contrato],
		'IVE' [Plan Beneficios],
		'' [# Poliza],
		'' [Valor Copago],
		'' [Valor Descuento],
		ValorFactura1
from(
 SELECT	Row_number() over(partition by pac.Identificacion,ACT.ActividadId order by pac.Identificacion,A.Numero ASC) fila,
		DC.Prefijo+''+CAST(A.Numero AS VARCHAR(10))  Factura,
		 S.CodigoHabilitacion [Codigo Prestador],
		SUBSTRING(pac.TipoIdentificacion,1,2) [Tipo Documento],
		pac.Identificacion Documento,
		CAST(FechaFactura AS DATE) Fecha,
		'' [# Autorizaci�n],
		ACT.CodigoNacional [Codigo CUP],
		1 Ambito,
		2 Finalidad,
		'' Personal,
		'' [DX Principal],
		'' [DX Relacionado 1],
		2  [DX Complicaciones],
		1 [Realizacion],
			M.CodigoResponsable, --	P6.Valor ,
		A.NombreResponsable	,
		 ((B.ValorFactura*B.IndicadorFacturable)-B.ValorIvaFactura)*A.EstadoActivo ValorFactura1,
		 ACT.Nombre
FROM FAC_MOV_Facturas A
JOIN FAC_MOV_FacturasOtrosDatos OT with (nolock) ON OT.FacturaId = A.FacturaId
inner join ADT_MOV_Ingresos ING with (nolock) on A.IngresoId = ING.IngresoId
inner join ADT_VIS_Pacientes pac with (nolock) on pac.PacienteId = ING.PacienteId
JOIN GEN_CNF_Parametros P5 with (nolock) ON P5.ParametroId = OT.TipoAtencionId
JOIN GEN_CNF_Companias O with (nolock) ON O.CompaniaId = A.CompaniaId
JOIN GEN_CNF_Sedes S with (nolock) ON S.SedeId = A.SedeId
JOIN GEN_CNF_DocumentosConsecutivos DC with (nolock) ON DC.DocumentoConsecutivoId = A.DocumentoConsecutivoId
JOIN ADT_CNF_Empresas M with (nolock) ON M.EmpresaId = A.EmpresaId
JOIN GEN_CNF_Parametros P6 with (nolock) ON P6.ParametroId = A.TipoEmpresaId
JOIN GEN_CNF_Parametros P7 with (nolock) ON P7.ParametroId = A.TipoIdentificacionId
JOIN FAC_CNF_ConceptosMovimientos AA with (nolock) ON AA.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P with (nolock) ON P.ParametroId = AA.ClaseDocumentoContableId AND P.Referencia1 = 'FAC'
JOIN FAC_CNF_ConceptosMovimientosEstados E with (nolock) ON E.ConceptoMovimientoId = A.ConceptoFACId
JOIN GEN_CNF_Parametros P8 ON P8.ParametroId = E.EstadoId AND P8.Referencia1 = 'GENERADA'
JOIN FAC_MOV_FacturasCargos B with (nolock) ON B.FacturaId = A.FacturaId AND B.IndicadorFacturable = 1
JOIN FAC_CNF_Actividades ACT ON ACT.ActividadId = B.ActividadId
JOIN GEN_CNF_Parametros P9 with (nolock) ON P9.ParametroId = ACT.TipoRipId
JOIN ADT_MOV_IngresosCargos C with (nolock) ON C.IngresoCargoId = B.IngresoCargoId
JOIN GEN_CNF_CentrosCostos N with (nolock) ON N.CentroCostosId = C.CentroCostosId
JOIN FAC_CNF_ConceptosMovimientos AAA with (nolock) ON AAA.ConceptoMovimientoId = C.ConceptoMovimientoId
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresas F with (nolock) ON F.ConceptoMovimientoId = C.ConceptoMovimientoId
             AND C.ConceptoIngresoId = F.ConceptoServicioId
             AND A.TipoEmpresaId = F.TipoEmpresaId
			 AND OT.TipoAtencionId = F.TipoAtencionId
JOIN CON_CNF_ConceptosMovimientosTiposEmpresas FF ON FF.ConceptoMovimientoId = A.ConceptoFACId
             AND A.TipoEmpresaId = FF.TipoEmpresaId
			 AND OT.TipoAtencionId = FF.TipoAtencionId
JOIN CON_CNF_InterfasesContables I  with (nolock) ON FF.DocumentoContable = I.DocumentoContable
JOIN CON_CNF_ConceptosMovimientosServiciosTiposEmpresasCuentas G with (nolock) ON
             G.ConceptoMovimientoServicioTipoEmpresaId = F.ConceptoMovimientoServicioTipoEmpresaId
			 AND G.CentroCostosId = C.CentroCostosId
JOIN GEN_CNF_Parametros P1 ON P1.ParametroId = G.TipoValorId AND P1.Referencia1 = 'VALORPROPIO'
JOIN GEN_CNF_Parametros P2 with (nolock) ON P2.ParametroId = G.NaturalezaId
JOIN GEN_CNF_Parametros P3 with (nolock) ON P3.ParametroId = G.IndicadorCCId
JOIN GEN_CNF_Parametros P4 with (nolock) ON P4.ParametroId = G.IndicadorNitId
JOIN CON_CNF_Cuentas CU with (nolock) ON CU.PlanCuentaId = G.PlanCuentaId AND CU.CuentaId = G.CuentaId
WHERE FechaFactura BETWEEN @desde AND @hasta
AND P9.Valor IN ('PROCEDIMIENTOS')
AND S.Codigo in (SELECT Codigo FROM GEN_CNF_Sedes WHERE CodigoTributarioCiudad = @codigo_ciudad)
and  NOT EXISTS (SELECT  NOTAS.FacturaId,NOtas.Numero,  B.FacturaAfectadaId
					FROM FAC_MOV_Facturas NOTAS WITH (NOLOCK)
					LEFT JOIN FAC_MOV_FacturasAnexos B WITH (NOLOCK) ON NOTAS.FacturaId = B.FacturaEncabezadoId
					INNER JOIN FAC_CNF_ConceptosMovimientos CM WITH (NOLOCK) ON CM.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P2 WITH (NOLOCK) ON	 P2.ParametroId = CM.ClaseDocumentoContableId
															AND (P2.Referencia1 = 'NOTADEBITOCARTERA' OR P2.Referencia1 = 'NOTACREDITOCARTERA')
					INNER JOIN FAC_CNF_ConceptosMovimientosEstados E WITH (NOLOCK) ON E.ConceptoMovimientoId = NOTAS.ConceptoFACId
					INNER JOIN GEN_CNF_Parametros P1 WITH (NOLOCK) ON P1.ParametroId = E.EstadoId AND P1.Referencia1 = 'GENERADA'
					WHERE NOTAS.EstadoGrabacion = 1 AND FacturaAfectadaId = A.FacturaId  AND NOTAS.FechaFactura> @desde )
				) tabla
					where fila in (1,2)
	)	TABLA
	INNER JOIN ADT_VIS_Pacientes pac with (nolock) ON pac.Identificacion = TABLA.Documento and SUBSTRING(pac.TipoIdentificacion,1,2) = TABLA.[Tipo Documento]
	left join GEN_CNF_Parametros ciudad on ciudad.ParametroId = MunicipioResidenciaId
	left join GEN_CNF_Parametros nacimiento on nacimiento.ParametroId = LugarNacimientoId
	left join GEN_CNF_Parametros departamento on departamento.ParametroId = nacimiento.PadreId
	left join GEN_CNF_Parametros pais on pais.ParametroId = departamento.PadreId
'''

# Information converted into a DataFrame with pandas
df = pd.read_sql(sql=sqlQuery, con=connection)

# Export the data on the excel into a Desktop
df.to_csv(os.environ["userprofile"] + "\\Documents\\Rips\\Sedes\\Valledupar\\" + "Valledupar_Usuarios " +
            datetime.now().strftime("%d-%b-%Y %H%M%S") + ".csv", index=False)

#Display notification to user
notification.notify(title="✅Report Status!!!",
                    message= f"Queries of Valledupar has been succesfully saved into .csv\
                        \n Queries Executed:  \n 1. Consultas, 2.Facturas, 3.Procedimientos, 4.Usuarios",
                        timeout=10)

#Display notification to user
notification.notify(title="RIPS QUERIES 👌",
                    message= f"All script had been executed sucessfully please validated csv files",
                        timeout=10)

C:\Users\ddrodriguez\AppData\Local\Temp\ipykernel_19832\1615192839.py:235: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=sqlQuery, con=connection)
